[![Open on GitHub](https://img.shields.io/badge/GitHub-View%20Source-181717?style=for-the-badge&logo=github)](https://github.com/SeenaKhosravi/NASS/blob/main/Analysis_NASS.ipynb)
[![Open In Colab](https://img.shields.io/badge/Colab-Open%20Notebook-F9AB00?style=for-the-badge&logo=google-colab)](https://colab.research.google.com/github/SeenaKhosravi/NASS/blob/main/Analysis_NASS.ipynb)
[![Launch on GCE](https://img.shields.io/badge/Google%20Cloud-Launch%20Instance-4285F4?style=for-the-badge&logo=google-cloud)](https://shell.cloud.google.com/cloudshell/editor?cloudshell_git_repo=https://github.com/SeenaKhosravi/NASS&cloudshell_tutorial=deploy/gce-tutorial.md)


# Socioeconomic and Demographic Drivers of Ambulatory Surgery Usage
### HCUP NASS 2020 - Reproducible Pipeline (Python + R)

**Script Author:** Seena Khosravi, MD  
**LLMs Utilized:** Claude Sonnet 4, Opus 4.1; ChatGPT 4o, o4; Deepseek 3.1; Gemini 2.5 Pro; Grok 5  
**Last Updated:** September 20, 2025  


**Data Source:**  
Department of Health & Human Services (HHS)  
Agency for Healthcare Research and Quality (AHRQ)  
Healthcare Cost and Utilization Project (HCUP)  
National Ambulatory Surgical Sample (NASS) 2020

 **[HCUP NASS 2020 Introduction](https://hcup-us.ahrq.gov/db/nation/nass/NASS_Introduction_2020.jsp)** - Official AHRQ documentation

---



## Overview
This notebook provides a reproducible analysis pipeline for examining socioeconomic and demographic factors influencing ambulatory surgery usage patterns. The analysis combines Python for data processing and R for data analysis and visualization.

### Data Usage Agreement
**DUA Compliant Online Implementation** - This notebook uses a simulated, artificial, smaller dataset with identical structure to the file created by [Raw_NASS_Processing.R](https://github.com/SeenaKhosravi/NASS/blob/a7764ce80be8a82fc449831821c27d957176c410/Raw%20NASS%20%20Processing.R). The simulated dataset production methodology is found in [Generate_Simulated_NASS.R](https://github.com/SeenaKhosravi/NASS/blob/161bf2b5c149da9654c0e887655b361fa2176db0/Generate_Simulated_NASS.R). If DUA signed and data purchased from HCUP, this notebook can run on full dataset in cloud storage (with provided passcode).

[Please see the DUA Agreement here.](https://hcup-us.ahrq.gov/team/NationwideDUA.jsp)

### Key Features
- **Multi-platform:** Works on Jupyter implementations via local environments, server, cloud VM instance, or platform as a service
- **Flexible Data Storage:** GitHub (simulated, static, open access), Google Drive, Google Cloud Storage, or local file
- **Reproducible:** All dependencies and environment setup included; Automated, Click-through VM set-up
- **Scalable:** Handles both simulated (0.2GB, 139k rows) and full dataset (12GB, 7.8M rows) with scalable cloud options

---

## Design Notes

### Architecture
- **Python primary, w/ R run via rpy2 python extension**
- **Python cells** handle "plumbing" (file I/O, environment setup, rpy2 configuration, data previews)
- **R cells** (prefixed by `%%R`) perform statistical analysis: survey weights, Census lookups, multilevel models, plots, classifiers, etc.

### Data Sources
- **Default:** Simulated dataset (0.2GB) from GitHub releases
- **Local:** Switch to locally stored files via configuration
- **Drive:** Google Drive (Only available in Colab)
- **Cloud:** Google Cloud Storage support for large datasets

### Environment Support
- **Local:** JupyterLab w/ Python 3.8+ kernel (requires R 4.0+, 4.4+ preferred)
- **Jupyter Server:** May require configuration depending on implementation (conda recommended)
- **Google Colab:** Pro recommended for full dataset, high-RAM runtime suggested
- **Virtual Machine Instance:** Automated GCE deployment via scripts, pre-configured R setup


---

# Setup


## 1. Configuration

**SETTINGS** 

Configure all settings here prior to run - data sources, debugging options, and file paths. Defaults to simulated dataset.

In [ ]:
# ==================== CONFIGURATION ====================
# Data Source Options
DATA_SOURCE = "github"      # Options: "github", "local", "gcs", "drive"
VERBOSE_PRINTS = True    # False → suppress debug output

# GitHub source (default - simulated data)
GITHUB_URL = "https://github.com/SeenaKhosravi/NASS/releases/download/v1.0.0/nass_2020_simulated.csv"

# Local file options
LOCAL_FILENAME = "nass_2020_local.csv"

# Google Cloud Storage options
GCS_BUCKET = "nass_2020"
GCS_BLOB = "nass_2020_all.csv"

# Google Drive options (for Colab)
DRIVE_PATH = "/content/drive/MyDrive/NASS/nass_2020_full.csv"


Configuration Loading

In [ ]:

# === CONFIGURATION LOADING ===
import getpass
import hashlib
import os
from typing import Optional

class SecurePasswordManager:
    """Secure password handling with memory cleanup."""
    
    def __init__(self):
        self._password_hash = None
    
    def get_password(self, prompt: str) -> Optional[str]:
        """Get password with enhanced security."""
        try:
            password = getpass.getpass(prompt)
            
            if not password.strip():
                return None
            
            # Store only hash for verification
            self._password_hash = hashlib.sha256(password.encode()).hexdigest()
            
            return password
            
        except (KeyboardInterrupt, EOFError):
            print("\nPassword entry cancelled")
            return None
    
    def clear_memory(self):
        """Attempt to clear sensitive data."""
        self._password_hash = None
        import gc
        gc.collect()

def configure_access():
    """Configure secure data source and Census access."""
    print(" NASS 2020 Research Analysis Pipeline")
    print(" Configuration Status ")
    print()
    
    password_manager = SecurePasswordManager()
    gcs_password = None
    census_password = None
    
    # Only show GCS prompts if GCS is selected
    if DATA_SOURCE == "gcs":
        print("🔒 Encrypted Dataset Access")
        gcs_password = password_manager.get_password(
            "   Enter GCS access key: "
        )
        
        if not gcs_password:
            print("   → Switching to public dataset")
        else:
            print("   ✓ Full dataset access enabled")
    
    # Streamlined Census API setup
    print("🌐 Census API Integration")
    census_password = password_manager.get_password(
        "   Enter Census API key: "
    )
    
    if census_password:
        print("   ✓ Encrypted Census integration ready")
    else:
        print("   → Manual API key entry required later")
    
    # Clear password manager
    password_manager.clear_memory()
    
    return gcs_password, census_password

# Configure access
GCS_PASSWORD, CENSUS_PASSWORD = configure_access()

# Set up encrypted URLs if passwords provided
if GCS_PASSWORD and DATA_SOURCE == "gcs":
    GCS_ENCRYPTED_KEY_URL = "https://github.com/SeenaKhosravi/NASS/releases/download/v1.0.0.0/gcs_key.enc"
elif DATA_SOURCE == "gcs":
    # Switch to simulated data if no GCS password
    DATA_SOURCE = "github"

if CENSUS_PASSWORD:
    CENSUS_ENCRYPTED_KEY_URL = "https://github.com/SeenaKhosravi/NASS/releases/download/v1.0.0.0/census_key.enc"

# Clean final status
print()
print("=" * 50)
data_status = {
    "github": "📊 Public Dataset (139K records)",
    "local": "💾 Local Dataset", 
    "gcs": "☁️  Full Dataset (7.8M records)",
    "drive": "📂 Google Drive Dataset"
}

census_status = "🔐 Encrypted" if CENSUS_PASSWORD else "🔑 Manual Entry"

print(f"✅ Ready: {data_status.get(DATA_SOURCE, DATA_SOURCE)} | Census: {census_status}")
print("=" * 50)

---
## 2. Python Environment Setup 

Detect environment and install Python packages.


In [ ]:
# === PYTHON ENVIRONMENT SETUP ===

import os
import sys
import subprocess
from pathlib import Path

class EnvironmentManager:
    def __init__(self):
        self.detect_environment()
        self.setup_packages()

    def detect_environment(self):
        """Detect runtime environment"""
        self.is_colab = 'COLAB_GPU' in os.environ or 'google.colab' in sys.modules
        self.is_vertex = 'DL_ANACONDA_HOME' in os.environ

        if self.is_colab:
            self.env_type = "Google Colab"
        elif self.is_vertex:
            self.env_type = "Vertex AI"
        else:
            self.env_type = "Local/Jupyter"

        print(f"Environment detected: {self.env_type}")

    def check_conda_available(self):
        """Check if conda is available"""
        try:
            subprocess.check_call(['conda', '--version'],
                                stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            return True
        except (subprocess.CalledProcessError, FileNotFoundError):
            return False

    def install_package(self, package, conda_name=None):
        """Smart package installation with fallback"""
        try:
            __import__(package)
            return True
        except ImportError:
            print(f"Installing {package}...")

            # Try conda first if available and not in Colab
            if conda_name and not self.is_colab and self.check_conda_available():
                try:
                    subprocess.check_call(['conda', 'install', '-y', conda_name],
                                        stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                    return True
                except subprocess.CalledProcessError:
                    print(f"  Conda install failed for {conda_name}, trying pip...")

            # Fallback to pip
            try:
                subprocess.check_call([sys.executable, '-m', 'pip', 'install', package],
                                    stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                return True
            except subprocess.CalledProcessError as e:
                print(f"  Pip install failed for {package}: {e}")
                return False

    def setup_packages(self):
        packages = {
            'pandas': 'pandas',
            'requests': 'requests',
            'rpy2': 'rpy2',
            'google.cloud.storage': 'google-cloud-storage',
            'pycryptodome': 'pycryptodome'  # For AES decryption
        }

        print("Installing and checking packages...")
        failed = []

        for pkg, install_name in packages.items():
            if not self.install_package(pkg, install_name):
                failed.append(pkg)

        # Store failed packages globally for recovery
        globals()['failed_packages'] = failed

        if failed:
            print(f"Warning: Failed to install: {', '.join(failed)}")
            print("Some features may not work")

            # Provide specific guidance for rpy2
            if 'rpy2' in failed:
                print("\nFor rpy2 installation issues:")
                if self.is_vertex:
                    print("   - Vertex AI: R may not be installed by default")
                    print("   - Run the next cell for automated R setup")
                else:
                    print("   - On Windows: May need Visual Studio Build Tools")
                    print("   - Try: conda install -c conda-forge rpy2")
                    print("   - Or: pip install rpy2 (requires R to be installed)")
        else:
            print("All packages ready")

        # Mount Google Drive if needed (check if DATA_SOURCE exists)
        try:
            if globals().get('DATA_SOURCE') == "drive" and self.is_colab:
                self.mount_drive()
        except NameError:
            pass  # DATA_SOURCE not defined yet

    def mount_drive(self):
        """Mount Google Drive in Colab"""
        try:
            from google.colab import drive
            drive.mount('/content/drive')
            print("Google Drive mounted successfully")
        except:
            print("Error: Failed to mount Google Drive")

# Initialize environment
env_manager = EnvironmentManager()

---
## 3. R Environment Setup

Load R integration and install R packages efficiently.

In [ ]:
# === R ENVIRONMENT SETUP ===

# Load rpy2 extension for R integration
try:
    %load_ext rpy2.ipython
    print("R integration loaded successfully")
    globals()['R_AVAILABLE'] = True
except Exception as e:
    print(f"Error: Failed to load R integration: {e}")

    # Windows-specific troubleshooting
    if "R.dll" in str(e) or "error 0x7e" in str(e):
        print("\nWindows R.dll loading issue detected:")
        print("   This is a common Windows + rpy2 compatibility issue")
        print("   Solutions:")
        print("   1. Restart Python kernel and try again")
        print("   2. Check R version compatibility with rpy2")
        print("   3. Try reinstalling R and rpy2")
        print("   4. Use Python-only analysis (fallback available)")
        globals()['R_AVAILABLE'] = False
    else:
        print("Install rpy2: pip install rpy2")
        globals()['R_AVAILABLE'] = False

Install essential R packages and attempt installation of optional packages.

**Note:** Other packages needed for specific analysis (advanced modeling packages) will be installed and called as needed later in the notebook.

In [ ]:
%%R -i VERBOSE_PRINTS # === R PACKAGE LOADING ===

# Environment-aware R package setup for Local, Colab, and GCE/Linux VMs

# Detect environment
is_colab <- Sys.getenv("COLAB_GPU") != ""
is_gce <- file.exists("/opt/nass") || file.exists("/etc/apt")  # GCE/Linux VM indicators

if(is_colab) {
  cat("Google Colab detected\n")
} else if(is_gce) {
  cat("GCE/Linux VM detected\n")
} else {
  cat("Local environment detected\n")
}

# Essential packages for all environments
essential_packages <- c(
  "data.table",    # Fast data manipulation
  "ggplot2",       # Plotting
  "scales"         # For ggplot2 percentage scales
)

optional_packages <- c(
  "survey"        # Survey statistics
)

# Fast installation settings
repos <- "https://cloud.r-project.org"
options(repos = repos)
Sys.setenv(MAKEFLAGS = paste0("-j", parallel::detectCores()))

# Package check and load functions
pkg_available <- function(pkg) {
  tryCatch({
    find.package(pkg, quiet = TRUE)
    TRUE
  }, error = function(e) FALSE)
}

load_pkg <- function(pkg) {
  tryCatch({
    suppressMessages(library(pkg, character.only = TRUE, quietly = TRUE))
    TRUE
  }, error = function(e) FALSE)
}

# Install missing essential packages
missing_essential <- essential_packages[!sapply(essential_packages, pkg_available)]

if(length(missing_essential) > 0) {
  cat("Installing essential packages:", paste(missing_essential, collapse = ", "), "\n")

  tryCatch({
    install.packages(missing_essential,
                    repos = repos,
                    type = getOption("pkgType"),
                    dependencies = FALSE,
                    quiet = !VERBOSE_PRINTS,
                    Ncpus = parallel::detectCores())
  }, error = function(e) {
    cat("Binary install failed, trying source...\n")
    install.packages(missing_essential,
                    repos = repos,
                    type = "source",
                    dependencies = FALSE,
                    quiet = !VERBOSE_PRINTS)
  })
}

# Load essential packages
essential_loaded <- sapply(essential_packages, load_pkg)
essential_success <- sum(essential_loaded)

cat("Essential packages loaded:", essential_success, "/", length(essential_packages), "\n")

# Quick install optional packages (30s timeout)
missing_optional <- optional_packages[!sapply(optional_packages, pkg_available)]

if(length(missing_optional) > 0) {
  cat("Installing optional packages...\n")

  for(pkg in missing_optional) {
    tryCatch({
      setTimeLimit(cpu = 30, elapsed = 30, transient = TRUE)
      install.packages(pkg, repos = repos,
                      type = getOption("pkgType"),
                      dependencies = FALSE,
                      quiet = TRUE)
      cat("Installed:", pkg, "\n")
    }, error = function(e) {
      cat("Skipped (timeout):", pkg, "\n")
    })

    setTimeLimit(cpu = Inf, elapsed = Inf, transient = FALSE)
  }
}

# Load optional packages
optional_loaded <- sapply(optional_packages, load_pkg)
optional_success <- sum(optional_loaded)

cat("Optional packages loaded:", optional_success, "/", length(optional_packages), "\n")

# Check core functionality
has_datatable <- require("data.table", quietly = TRUE)
has_ggplot <- require("ggplot2", quietly = TRUE)

# Enhanced GCE/Linux setup if packages are missing
if(is_gce && (!has_datatable || !has_ggplot)) {
  cat("\nGCE/Linux detected - attempting enhanced installation...\n")

  # System dependencies for Linux VMs
  if(Sys.which("apt-get") != "") {
    cat("Installing system dependencies...\n")
    system_deps <- c(
      "apt-get update -qq",
      "apt-get install -y libfontconfig1-dev libcairo2-dev",
      "apt-get install -y libxml2-dev libcurl4-openssl-dev libssl-dev",
      "apt-get install -y libharfbuzz-dev libfribidi-dev",
      "apt-get install -y libfreetype6-dev libpng-dev libtiff5-dev libjpeg-dev"
    )
    
    for(cmd in system_deps) {
      system(paste("sudo", cmd), ignore.stdout = TRUE, ignore.stderr = TRUE)
    }
  }

  # Retry failed packages with multiple repositories
  failed_packages <- c()
  if(!has_datatable) failed_packages <- c(failed_packages, "data.table")
  if(!has_ggplot) failed_packages <- c(failed_packages, "ggplot2", "scales")

  repos_gce <- c("https://cran.rstudio.com/", "https://cloud.r-project.org")

  for(pkg in failed_packages) {
    cat("Installing", pkg, "...")
    installed <- FALSE

    for(repo in repos_gce) {
      tryCatch({
        install.packages(pkg, repos = repo, dependencies = TRUE, quiet = TRUE)
        if(require(pkg, character.only = TRUE, quietly = TRUE)) {
          cat(" Success\n")
          installed <- TRUE
          break
        }
      }, error = function(e) NULL)
    }

    if(!installed) cat(" FAILED\n")
  }

  # Re-check after enhanced installation
  has_datatable <- require("data.table", quietly = TRUE)
  has_ggplot <- require("ggplot2", quietly = TRUE)

  cat("After enhanced installation: data.table =", has_datatable, "| ggplot2 =", has_ggplot, "\n")
}

# Final status check
if(has_datatable && has_ggplot) {
  cat("Core environment ready! (data.table + ggplot2)\n")
  setDTthreads(0)  # Use all cores

} else if(has_datatable) {
  cat("Warning: Partial setup - data.table ready, plotting may be limited\n")
  setDTthreads(0)

} else {
  cat("Error: Critical failure - data.table not available\n")
  stop("Cannot proceed without data.table")
}

Verify R setup is complete and ready for analysis.

In [ ]:
%%R # === R ENVIRONMENT VERIFICATION ===

# Quick verification and setup
cat("Verifying R environment...\n")

# Test core functionality
tryCatch({
  # Test data.table (essential)
  dt_test <- data.table(x = 1:3, y = letters[1:3])
  cat("data.table ready\n")

  # Test ggplot2 (optional)
  if(require("ggplot2", quietly = TRUE)) {
    cat("ggplot2 ready\n")
  } else {
    cat("Warning: ggplot2 not available (plots disabled)\n")
  }

  # Set up data.table options for performance
  setDTthreads(0)  # Use all cores

  cat("R environment optimized and ready!\n")

}, error = function(e) {
  cat("Error: R environment verification failed:", e$message, "\n")
  stop("R setup incomplete")
})

# Clean up test objects
rm(list = ls()[!ls() %in% c("VERBOSE_PRINTS")])
invisible(gc())

---
## 4. Data Loading

Config based data loader with error handling.

In [ ]:
# === DATA LOADING ===

import os
import sys
import time
import requests
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from typing import Optional, Dict, Any
import threading
import queue

class NASSDataLoader:
    """Secure data loader for NASS dataset with encryption support."""
    
    def __init__(self, verbose: bool = True):
        self.verbose = verbose
        self.data_dir = Path.home() / 'data'
        self.data_dir.mkdir(exist_ok=True)
        self._setup_environment()
        
        # Initialize encryption attributes
        self._gcs_encrypted_url = None
        self._gcs_password = None
        self._census_encrypted_url = None
        self._census_password = None
    
    def _setup_environment(self):
        """Detect environment and set appropriate settings."""
        self.is_colab = 'google.colab' in sys.modules
        self.is_local = not self.is_colab
        
        if self.verbose:
            env_name = "Google Colab" if self.is_colab else "Local/Jupyter"
            print(f"Environment: {env_name}")
    
    def set_encryption_config(self, gcs_url=None, gcs_password=None, census_url=None, census_password=None):
        """Set encryption configuration securely."""
        self._gcs_encrypted_url = gcs_url
        self._gcs_password = gcs_password
        self._census_encrypted_url = census_url
        self._census_password = census_password
    
    def _decrypt_key(self, encrypted_url: str, password: str) -> Optional[str]:
        """Decrypt key using pycryptodome with security measures."""
        decrypted_key = None
        
        try:
            from Crypto.Cipher import AES
            from Crypto.Hash import SHA256
            import base64
            import json
            
            if self.verbose:
                print("Downloading and decrypting access key...")
            
            # Download encrypted data
            response = requests.get(encrypted_url, timeout=30)
            response.raise_for_status()
            encrypted_data = base64.b64decode(response.content)
            
            # Derive key from password
            key = SHA256.new(password.encode()).digest()
            
            # Decrypt (assuming AES-CBC with IV)
            cipher = AES.new(key, AES.MODE_CBC, encrypted_data[:16])
            decrypted = cipher.decrypt(encrypted_data[16:])
            
            # Remove PKCS7 padding
            pad_len = decrypted[-1]
            decrypted_key = decrypted[:-pad_len].decode()
            
            # Validate it's valid JSON (basic integrity check)
            json.loads(decrypted_key)  # Will raise if invalid
            
            if self.verbose:
                print("✅ Successfully decrypted access key")
            
            return decrypted_key
            
        except ImportError:
            self._report_error("pycryptodome not installed - cannot decrypt keys")
            return None
        except Exception as e:
            if self.verbose:
                print("❌ Decryption failed: Invalid password or corrupted data")
            return None
            
        finally:
            # Security cleanup
            try:
                if 'password' in locals():
                    password = 'X' * len(password)
                if 'key' in locals():
                    key = b'X' * len(key) 
                if 'decrypted' in locals():
                    decrypted = b'X' * len(decrypted)
                
                import gc
                gc.collect()
            except:
                pass
    
    def load_data(self, source: str, **config) -> Optional[pd.DataFrame]:
        """
        Load data based on source configuration.
        
        Args:
            source: One of 'github', 'local', 'drive', 'gcs'
            **config: Source-specific configuration parameters
        
        Returns:
            DataFrame if successful, None if failed
        """
        if self.verbose:
            print(f"Loading data from source: {source}")
        
        try:
            if source == "github":
                return self._load_from_github(config.get('url'))
            elif source == "local":
                return self._load_from_local(config.get('filename'))
            elif source == "drive" and self.is_colab:
                return self._load_from_drive(config.get('path'))
            elif source == "gcs":
                return self._load_from_gcs(config)
            else:
                self._report_error(f"Unsupported source '{source}' for current environment")
                return None
                
        except Exception as e:
            self._report_error(f"Failed to load from {source}: {str(e)}")
            return None
    
    def _load_from_github(self, url: str) -> Optional[pd.DataFrame]:
        """Load data from GitHub public URL."""
        if not url:
            self._report_error("GitHub URL not provided in configuration")
            return None
        
        cache_file = self.data_dir / "nass_github_cache.csv"
        
        # Check cache first
        if cache_file.exists():
            cache_age = time.time() - cache_file.stat().st_mtime
            if cache_age < 3600:  # 1 hour cache
                if self.verbose:
                    print("✅ Using cached GitHub data")
                return pd.read_csv(cache_file)
        
        try:
            if self.verbose:
                print("Downloading from GitHub...")
            
            # Stream download with progress
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            total_size = int(response.headers.get('content-length', 0))
            
            with open(cache_file, 'wb') as f:
                downloaded = 0
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        if self.verbose and total_size > 0:
                            progress = (downloaded / total_size) * 100
                            print(f"\rProgress: {progress:.1f}%", end='', flush=True)
            
            if self.verbose:
                print(f"\n✅ Downloaded {downloaded / (1024*1024):.1f} MB")
            
            return pd.read_csv(cache_file)
            
        except requests.RequestException as e:
            self._report_error(f"GitHub download failed: {str(e)}")
            return None
    
    def _load_from_local(self, filename: str) -> Optional[pd.DataFrame]:
        """Load data from local file."""
        if not filename:
            self._report_error("Local filename not provided in configuration")
            return None
        
        # Try multiple possible locations
        possible_paths = [
            Path(filename),
            self.data_dir / filename,
            Path.cwd() / filename,
            Path.cwd() / 'data' / filename
        ]
        
        for file_path in possible_paths:
            if file_path.exists():
                try:
                    if self.verbose:
                        size_mb = file_path.stat().st_size / (1024*1024)
                        print(f"Loading local file: {file_path} ({size_mb:.1f} MB)")
                    
                    # Use chunked reading for large files
                    if file_path.stat().st_size > 100 * 1024 * 1024:  # 100MB
                        return self._load_large_csv(file_path)
                    else:
                        return pd.read_csv(file_path)
                        
                except Exception as e:
                    self._report_error(f"Failed to read {file_path}: {str(e)}")
                    continue
        
        self._report_error(f"Local file '{filename}' not found in any expected location")
        return None
    
    def _load_from_drive(self, drive_path: str) -> Optional[pd.DataFrame]:
        """Load data from Google Drive (Colab only)."""
        if not self.is_colab:
            self._report_error("Google Drive access only available in Colab")
            return None
        
        if not drive_path:
            self._report_error("Google Drive path not provided in configuration")
            return None
        
        try:
            # Mount Drive if not already mounted
            if not Path('/content/drive').exists():
                if self.verbose:
                    print("Mounting Google Drive...")
                from google.colab import drive
                drive.mount('/content/drive')
            
            file_path = Path(drive_path)
            if not file_path.exists():
                self._report_error(f"File not found on Google Drive: {drive_path}")
                return None
            
            if self.verbose:
                size_mb = file_path.stat().st_size / (1024*1024)
                print(f"Loading from Google Drive: {size_mb:.1f} MB")
            
            return self._load_large_csv(file_path)
            
        except Exception as e:
            self._report_error(f"Google Drive access failed: {str(e)}")
            return None
    
    def _load_from_gcs(self, config: Dict[str, Any]) -> Optional[pd.DataFrame]:
        """Load data from Google Cloud Storage with encrypted authentication."""
        bucket_name = config.get('bucket')
        blob_name = config.get('blob')
        
        if not bucket_name or not blob_name:
            self._report_error("GCS bucket and blob names required in configuration")
            return None
        
        try:
            from google.cloud import storage
            
            # Try encrypted service account key first
            client = self._try_encrypted_service_account()
            
            if not client:
                # Fallback to other authentication methods
                auth_methods = [
                    self._try_default_credentials,
                    self._try_service_account,
                    self._try_anonymous_access
                ]
                
                for auth_method in auth_methods:
                    client = auth_method()
                    if client:
                        break
            
            if not client:
                self._report_error("Failed to authenticate with Google Cloud Storage")
                return None
            
            if self.verbose:
                print(f"Accessing gs://{bucket_name}/{blob_name}")
            
            bucket = client.bucket(bucket_name)
            blob = bucket.blob(blob_name)
            
            if not blob.exists():
                self._report_error(f"File not found: gs://{bucket_name}/{blob_name}")
                return None
            
            # Get file info
            blob.reload()
            size_gb = blob.size / (1024**3)
            
            if self.verbose:
                print(f"File size: {size_gb:.2f} GB")
            
            # Download with progress tracking
            cache_file = self.data_dir / f"nass_gcs_{bucket_name}_{blob_name.replace('/', '_')}"
            
            if self.verbose:
                print("Downloading from GCS...")
            
            # Use resumable download for large files
            if blob.size > 100 * 1024 * 1024:  # 100MB
                self._download_large_blob(blob, cache_file)
            else:
                blob.download_to_filename(cache_file)
            
            if self.verbose:
                print("✅ Download complete, loading into memory...")
            
            return self._load_large_csv(cache_file)
            
        except ImportError:
            self._report_error("google-cloud-storage package not installed")
            return None
        except Exception as e:
            self._report_error(f"GCS access failed: {str(e)}")
            return None
    
    def _try_encrypted_service_account(self):
        """Try encrypted service account key."""
        if (self._gcs_encrypted_url and self._gcs_password):
            try:
                from google.cloud import storage
                import json
                import tempfile
                
                # Decrypt the service account key
                decrypted_key = self._decrypt_key(self._gcs_encrypted_url, self._gcs_password)
                
                if decrypted_key:
                    # Parse as JSON and create temporary file
                    key_data = json.loads(decrypted_key)
                    
                    with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
                        json.dump(key_data, f)
                        temp_key_file = f.name
                    
                    try:
                        client = storage.Client.from_service_account_json(temp_key_file)
                        if self.verbose:
                            print("✅ Authenticated using encrypted service account key")
                        return client
                    finally:
                        # Clean up temporary file
                        try:
                            os.unlink(temp_key_file)
                        except:
                            pass
                            
            except Exception as e:
                if self.verbose:
                    print(f"Encrypted service account auth failed: {str(e)}")
        
        return None
    
    def _try_default_credentials(self):
        """Try default application credentials."""
        try:
            from google.cloud import storage
            return storage.Client()
        except:
            return None
    
    def _try_service_account(self):
        """Try service account key file."""
        try:
            from google.cloud import storage
            key_file = self.data_dir / 'gcs_service_account.json'
            if key_file.exists():
                return storage.Client.from_service_account_json(str(key_file))
        except:
            pass
        return None
    
    def _try_anonymous_access(self):
        """Try anonymous access for public buckets."""
        try:
            from google.cloud import storage
            return storage.Client.create_anonymous_client()
        except:
            return None
    
    def _download_large_blob(self, blob, cache_file):
        """Download large blob with progress tracking."""
        chunk_size = 1024 * 1024  # 1MB chunks
        
        with open(cache_file, 'wb') as f:
            downloaded = 0
            
            # Download in chunks
            for chunk_start in range(0, blob.size, chunk_size):
                chunk_end = min(chunk_start + chunk_size - 1, blob.size - 1)
                
                chunk_data = blob.download_as_bytes(
                    start=chunk_start,
                    end=chunk_end
                )
                
                f.write(chunk_data)
                downloaded += len(chunk_data)
                
                if self.verbose:
                    progress = (downloaded / blob.size) * 100
                    print(f"\rProgress: {progress:.1f}%", end='', flush=True)
        
        if self.verbose:
            print()  # New line after progress
   
    def _load_large_csv(self, file_path: Path) -> pd.DataFrame:
        """Load large CSV files efficiently with progress tracking."""
        file_size = file_path.stat().st_size
        
        if file_size < 500 * 1024 * 1024:  # Less than 500MB, load normally
            return pd.read_csv(file_path)
        
        if self.verbose:
            print("Loading large file in chunks...")
        
        try:
            # Memory-efficient chunk processing - NO STORING ALL CHUNKS
            print("📊 Loading data with memory optimization...")
            total_rows = 0
            first_chunk = True

            # Process chunks one at a time without storing them all
            for i, chunk in enumerate(pd.read_csv(file_path,
                                                chunksize=50000,  # Smaller chunks
                                                low_memory=False,
                                                dtype=str)):  # Force string type for mixed columns
        
                # Basic processing on each chunk
                total_rows += len(chunk)
                print(f"   Processed chunk {i+1}: {len(chunk):,} rows (Total: {total_rows:,})")
        
                # For the first chunk, establish the combined dataframe
                if first_chunk:
                    # Take a sample for analysis instead of full dataset
                    sample_size = min(100000, len(chunk))  # Max 100k rows for analysis
                    df = chunk.head(sample_size).copy()
                    first_chunk = False
                    print(f"   ✅ Using sample of {len(df):,} rows for analysis")
                    break  # Exit after first chunk for memory safety
        
                # Memory cleanup every few chunks
                if i % 5 == 0:
                    import gc
                    gc.collect()
                    
                # Safety limit - stop if too many chunks
                if i >= 20:
                    print("   ⚠️ Stopping at 20 chunks for memory safety")
                    break
        
            print(f"✅ Data processing complete!")
            print(f"   Full dataset size: {total_rows:,} rows")
            print(f"   Analysis sample: {len(df):,} rows")
            
            return df
        
        except MemoryError:
            print("❌ Memory limit reached - using smaller sample")
            # Fallback: load just a small sample
            df = pd.read_csv(file_path, nrows=50000, low_memory=False)
            print(f"   Fallback sample: {len(df):,} rows")
            return df
            
        except Exception as e:
            print(f"❌ Error during data loading: {e}")
            # Last resort: very small sample
            df = pd.read_csv(file_path, nrows=10000, low_memory=False)
            print(f"   Emergency sample: {len(df):,} rows")
            return df

# Initialize the data loader
data_loader = NASSDataLoader(verbose=VERBOSE_PRINTS)

# Pass encrypted URLs and passwords to the data loader if available
if 'GCS_ENCRYPTED_KEY_URL' in globals() and 'GCS_PASSWORD' in globals():
    data_loader.set_encryption_config(
        gcs_url=GCS_ENCRYPTED_KEY_URL,
        gcs_password=GCS_PASSWORD
    )

if 'CENSUS_ENCRYPTED_KEY_URL' in globals() and 'CENSUS_PASSWORD' in globals():
    data_loader.set_encryption_config(
        census_url=CENSUS_ENCRYPTED_KEY_URL,
        census_password=CENSUS_PASSWORD
    )

# Load data based on configuration
config_map = {
    "github": {"url": GITHUB_URL},
    "local": {"filename": LOCAL_FILENAME},
    "drive": {"path": DRIVE_PATH},
    "gcs": {
        "bucket": GCS_BUCKET,
        "blob": GCS_BLOB
    }
}

if DATA_SOURCE in config_map:
    print(f"Loading NASS data from {DATA_SOURCE}...")
    df = data_loader.load_data(DATA_SOURCE, **config_map[DATA_SOURCE])
    
    if df is not None:
        print(f"✅ Successfully loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
        print(f"Memory usage: {df.memory_usage(deep=True).sum() / (1024*1024):.1f} MB")
    else:
        print("❌ Failed to load data")
        print("\nTroubleshooting:")
        if DATA_SOURCE == "local":
            print(f"  - Ensure file '{LOCAL_FILENAME}' exists in one of these locations:")
            print(f"    • {Path.cwd()}")
            print(f"    • {data_loader.data_dir}")
        elif DATA_SOURCE == "gcs":
            print(f"  - Verify bucket '{GCS_BUCKET}' and file '{GCS_BLOB}' exist")
            print(f"  - Check decryption password or authentication")
        elif DATA_SOURCE == "drive":
            print(f"  - Ensure file exists at: {DRIVE_PATH}")
            print(f"  - Google Drive must be mounted in Colab")
        elif DATA_SOURCE == "github":
            print(f"  - Check URL is accessible: {GITHUB_URL}")
            print(f"  - Verify internet connection")
else:
    print(f"❌ Invalid data source: {DATA_SOURCE}")
    print(f"Valid sources: {list(config_map.keys())}")

Check if data has been loaded.

In [ ]:
# === DATA LOAD VERIFICATION ===

# Verify data is available before R processing
try:
    if 'df' not in globals():
        print("❌ Data not loaded!")
        print("💡 Please run the 'Data Loading' section first (cell 12)")
        print("   This will create the 'df' variable needed for R analysis")
        raise NameError("df variable not found - run data loading first")
    
    # Check if df exists but is None or empty
    if df is None:
        print("❌ Data loading failed!")
        print("💡 The data loader returned None - check the previous cell for errors")
        print("   Common issues:")
        print("   • Network connection problems")
        print("   • Invalid file path or URL")
        print("   • Authentication issues (for cloud storage)")
        raise ValueError("df is None - data loading failed")
    
    # Check if df is empty
    if hasattr(df, 'shape') and df.shape[0] == 0:
        print("❌ Data is empty!")
        print("💡 The dataset loaded but contains no rows")
        raise ValueError("df is empty - no data to analyze")
    
    # All checks passed
    print(f"✅ Data verified: {df.shape[0]:,} rows x {df.shape[1]} columns")
    print("✅ Ready for R analysis")

except (NameError, ValueError, AttributeError) as e:
    print(f"❌ {e}")
    print("\n🔄 Quick fix: Run these cells in order:")
    print("   1. Configuration (cell 6)")
    print("   2. Environment Setup (cell 8)")  
    print("   3. Data Loading (cell 13)")
    print("   4. Then continue with R analysis")
    
    print("\n🔍 Troubleshooting:")
    print("   • Check your DATA_SOURCE setting in configuration")
    print("   • Verify internet connection for GitHub/cloud sources")
    print("   • Ensure file exists for local sources")
    print("   • Check authentication for cloud storage")
    
    # Don't raise the error - just warn and continue
    print("\n⚠️  Continuing without data - subsequent cells may fail")

## 5. Complete Data Preprocessing

Streamlined preprocessing: remove variables, clean data types, and create new variables - in Python prior to passing to R for efficiency.

In [ ]:
# === DATA PREPROCESSING ===

# Data Preprocessing in Python (before R transfer)
print("Complete preprocessing: removing variables + cleaning data types...")
print(f"Original shape: {df.shape}")

# ===== 1. REMOVE UNNECESSARY VARIABLES =====
print("\n1️⃣ Removing unnecessary variables...")

# Smart pattern-based removal in pandas (much faster than R)
drop_patterns = [
    r'^CPTCCS[2-9]$',      # CPTCCS2-CPTCCS9
    r'^CPTCCS[1-3][0-9]$', # CPTCCS10-30
    r'^CPT[2-9]$',         # CPT2-CPT9
    r'^CPT[1-3][0-9]$',    # CPT10-30
    r'^DXCCSR_',           # All DXCCSR columns (500+)
]

# Find columns to drop using vectorized operations
drop_cols = []
for pattern in drop_patterns:
    matches = df.columns[df.columns.str.match(pattern)].tolist()
    drop_cols.extend(matches)

# Remove duplicates
drop_cols = list(set(drop_cols))

print(f"   Found {len(drop_cols)} columns to drop")
print(f"   Patterns: CPTCCS2-30, CPT2-30, all DXCCSR_*")

# Drop the columns
df = df.drop(columns=drop_cols)
print(f"   ✅ Reduced from {df.shape[1] + len(drop_cols)} to {df.shape[1]} columns")

# ===== 2. CLEAN DATA TYPES FOR rpy2 =====
print("\n2️⃣ Cleaning data types for rpy2 compatibility...")

# Convert all object columns to strings (prevents mixed-type issues)
object_columns = df.select_dtypes(include=['object']).columns
if len(object_columns) > 0:
    for col in object_columns:
        df[col] = df[col].astype(str)
    print(f"   ✅ Converted {len(object_columns)} object columns to strings")

# Handle NaN/inf values consistently - FIXED FOR CATEGORICAL COLUMNS
# First handle categorical columns separately
categorical_columns = df.select_dtypes(include=['category']).columns
if len(categorical_columns) > 0:
    for col in categorical_columns:
        # Convert categorical to string first, then handle NaN
        df[col] = df[col].astype(str)
    print(f"   ✅ Converted {len(categorical_columns)} categorical columns to strings")

# Now handle all non-categorical columns
non_categorical_columns = df.select_dtypes(exclude=['category']).columns
if len(non_categorical_columns) > 0:
    # Replace NaN with empty strings for non-categorical columns
    df[non_categorical_columns] = df[non_categorical_columns].fillna('')

# Handle inf values in float columns
float_cols = df.select_dtypes(include=['float64', 'float32']).columns
if len(float_cols) > 0:
    for col in float_cols:
        df[col] = df[col].replace([float('inf'), float('-inf')], '')
    print(f"   ✅ Cleaned inf values in {len(float_cols)} float columns")

print(f"   ✅ Cleaned NaN values in all columns")

# ===== 3. CREATE KEY ANALYTICAL VARIABLES IN PANDAS =====
print("\n3️⃣ Creating analytical variables...")

# Create WHITE indicator (1=White, 0=Non-White)
if 'RACE' in df.columns:
    df['WHITE'] = (df['RACE'].astype(str) == '1').astype(int)
    print("   ✅ Created race indicator boolean")

# Create age groups
if 'AGE' in df.columns:
    df['AGE'] = pd.to_numeric(df['AGE'], errors='coerce')  # Ensure numeric
    df['AGE_GROUP'] = pd.cut(df['AGE'],
                            bins=[0, 18, 30, 45, 65, float('inf')],
                            labels=['0-17', '18-29', '30-44', '45-64', '65+'],
                            right=False)
    df['AGE_GROUP'] = df['AGE_GROUP'].astype(str)  # Convert to string for R
    print("   ✅ Created AGE_GROUP categories")

# Create income level labels
if 'ZIPINC_QRTL' in df.columns:
    income_map = {1: 'Q1-Lowest', 2: 'Q2', 3: 'Q3', 4: 'Q4-Highest'}
    df['INCOME_LEVEL'] = df['ZIPINC_QRTL'].astype(str).map(lambda x: income_map.get(int(x) if x.isdigit() else 0, 'Unknown'))
    print("   ✅ Created INCOME_LEVEL labels")

# Ensure key numeric variables are properly typed
numeric_vars = ['AGE', 'DISCWT', 'TOTCHG']
for var in numeric_vars:
    if var in df.columns:
        df[var] = pd.to_numeric(df[var], errors='coerce')

# save copy of cleaned data for R transfer
cleaned_path = data_loader.data_dir / "nass_data_cleaned.csv"
df.to_csv(cleaned_path, index=False)

print(f"\n✅ PREPROCESSING COMPLETE!")
print(f"Final shape: {df.shape}")
print(f"Cleaned data saved to: {cleaned_path}")
print(f"Ready for R transfer!")

## 6. Final R Transfer & Processing

Transfer the clean data to R and apply any final R-specific formatting.

In [ ]:
%%R -i df -i VERBOSE_PRINTS # === R Transfer ===

# Convert to data.table and apply R types
NASS <- as.data.table(df)

# Factor variables
factor_vars <- c("ZIPINC_QRTL", "PAY1", "CPTCCS1", "HOSP_LOCATION",
                 "HOSP_TEACH", "HOSP_NASS", "RACE", "AGE_GROUP", "INCOME_LEVEL")
existing_factors <- factor_vars[factor_vars %in% names(NASS)]
NASS[, (existing_factors) := lapply(.SD, as.factor), .SDcols = existing_factors]

# Boolean variables
if("FEMALE" %in% names(NASS)) NASS[, FEMALE := as.logical(as.numeric(FEMALE))]
if("WHITE" %in% names(NASS)) NASS[, WHITE := as.logical(as.numeric(WHITE))]

# Compact output
cat("> R Complete:", nrow(NASS), "rows,", ncol(NASS), "cols,",
    round(object.size(NASS)/1024^2, 1), "MB\n")
cat("Converted", length(existing_factors), "factors + 2 booleans\n")

if(VERBOSE_PRINTS) {
  cat("\nColumns:\n")
  print(colnames(NASS))
}

cat("Data in R!\n")

## 7. Output Saving system

In [ ]:
%%R # === OUTPUT SAVING SYSTEM ===

# ========================================
# OUTPUT SAVING SYSTEM FOR POSTER GENERATION
# ========================================

# Install and load required packages first
required_packages <- c("ggplot2", "data.table", "scales", "RColorBrewer")

for(pkg in required_packages) {
  if(!require(pkg, character.only = TRUE, quietly = TRUE)) {
    cat("Installing", pkg, "...\n")
    install.packages(pkg, quiet = TRUE)
    library(pkg, character.only = TRUE, quietly = TRUE)
  }
}

# Define theme and color schemes
theme_nass <- theme_minimal() +
  theme(
    plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
    plot.subtitle = element_text(size = 12, hjust = 0.5, color = "gray40"),
    axis.title = element_text(size = 11, face = "bold"),
    axis.text = element_text(size = 10),
    legend.title = element_text(size = 11, face = "bold"),
    legend.text = element_text(size = 10),
    strip.text = element_text(size = 10, face = "bold"),
    panel.grid.minor = element_blank()
  )

# Define consistent color palettes
race_colors <- RColorBrewer::brewer.pal(6, "Set2")
pay_colors <- RColorBrewer::brewer.pal(6, "Dark2")
region_colors <- RColorBrewer::brewer.pal(4, "Set1")

# Create plots directory if it doesn't exist
plot_dir <- "saved_plots"
if (!dir.exists(plot_dir)) dir.create(plot_dir, recursive = TRUE)

# Global plot counter and metadata (initialize if not exists)
if(!exists("plot_counter")) plot_counter <- 1
if(!exists("plot_metadata")) plot_metadata <- list()

# Enhanced save function with metadata
save_plot_for_poster <- function(plot_obj = NULL, 
                                name = NULL, 
                                width = 12, 
                                height = 8, 
                                dpi = 300,
                                description = "",
                                category = "analysis") {
  
  # Auto-generate name if not provided
  if (is.null(name)) {
    name <- paste0("plot_", sprintf("%02d", plot_counter))
  }
  
  # File paths
  png_file <- file.path(plot_dir, paste0(name, ".png"))
  pdf_file <- file.path(plot_dir, paste0(name, ".pdf"))
  
  # Save current plot if no plot object provided
  if (is.null(plot_obj)) {
    ggsave(png_file, width = width, height = height, dpi = dpi, units = "in")
    ggsave(pdf_file, width = width, height = height, dpi = dpi, units = "in")
  } else {
    # Save specific plot object
    ggsave(png_file, plot = plot_obj, width = width, height = height, dpi = dpi, units = "in")
    ggsave(pdf_file, plot = plot_obj, width = width, height = height, dpi = dpi, units = "in")
  }
  
  # Store metadata
  plot_metadata[[name]] <<- list(
    file_png = png_file,
    file_pdf = pdf_file,
    width = width,
    height = height,
    dpi = dpi,
    description = description,
    category = category,
    order = plot_counter
  )
  
  plot_counter <<- plot_counter + 1
  
  cat("> Saved plot:", name, "->", basename(png_file), "\n")
  cat("   Description:", description, "\n")
  
  return(invisible(png_file))
}

# Text output saving function
save_text_output <- function(text_content, name, description = "", category = "analysis") {
  
  # Create text outputs directory
  text_dir <- "saved_outputs"
  if (!dir.exists(text_dir)) dir.create(text_dir, recursive = TRUE)
  
  # File path
  text_file <- file.path(text_dir, paste0(name, ".txt"))
  
  # Add metadata header
  header <- paste0(
    "# ", description, "\n",
    "# Generated: ", Sys.time(), "\n",
    "# Category: ", category, "\n",
    "# ========================================\n\n"
  )
  
  # Write content
  writeLines(c(header, text_content), text_file)
  
  cat("> Saved text output:", name, "->", basename(text_file), "\n")
  
  return(invisible(text_file))
}

cat("> Output saving system initialized\n")
cat("   Directory:", plot_dir, "\n")
cat("   Packages loaded and themes defined\n")


## 8. Color and Theme Definitions

In [ ]:
%%R # === COLOR SYSTEM ===

# ========================================
# ENHANCED GLOBAL THEME AND COLOR SYSTEM
# ========================================

cat("> UPDATING GLOBAL THEME AND COLOR SYSTEM\n")
cat(paste(rep("=", 50), collapse = ""), "\n")  # FIXED: Use paste(rep()) instead of "=" * 50

# Install required packages if needed
required_packages <- c("ggplot2", "scales", "RColorBrewer")
for(pkg in required_packages) {
  if(!require(pkg, character.only = TRUE, quietly = TRUE)) {
    install.packages(pkg, quiet = TRUE)
    library(pkg, character.only = TRUE, quietly = TRUE)
  }
}

# ========================================
# ENHANCED POSTER-READY THEME
# ========================================

theme_poster <- theme_minimal() +
  theme(
    # TITLES - Left-aligned, larger fonts
    plot.title = element_text(
      size = 18,           # Increased from 14
      face = "bold", 
      hjust = 0,           # LEFT-ALIGNED (was 0.5 center)
      margin = ggplot2::margin(b = 8, t = 5)
    ),
    plot.subtitle = element_text(
      size = 14,           # Increased from 12  
      hjust = 0,           # LEFT-ALIGNED (was 0.5 center)
      color = "gray30",    # Darker for better readability
      margin = ggplot2::margin(b = 15)
    ),
    
    # AXIS - Larger fonts
    axis.title = element_text(size = 14, face = "bold"),  # Increased from 11
    axis.text = element_text(size = 12),                  # Increased from 10
    axis.title.x = element_text(margin = ggplot2::margin(t = 8)),
    axis.title.y = element_text(margin = ggplot2::margin(r = 8)),
    
    # LEGEND - Larger fonts
    legend.title = element_text(size = 14, face = "bold"), # Increased from 11
    legend.text = element_text(size = 12),                 # Increased from 10
    legend.position = "bottom",
    legend.margin = ggplot2::margin(t = 10),
    
    # FACET STRIPS - Larger fonts
    strip.text = element_text(size = 12, face = "bold"),   # Increased from 10
    
    # GRID
    panel.grid.minor = element_blank(),
    panel.grid.major = element_line(color = "gray90", linewidth = 0.5),
    
    # PLOT MARGINS
    plot.margin = ggplot2::margin(15, 15, 15, 15)
  )

# ========================================
# UNIFIED COLOR PALETTES
# ========================================

# PRIMARY PALETTE: Professional blue/teal scheme for most plots
primary_colors <- c(
  "#1f77b4",  # Professional blue
  "#ff7f0e",  # Orange
  "#2ca02c",  # Green  
  "#d62728",  # Red
  "#9467bd",  # Purple
  "#8c564b",  # Brown
  "#e377c2",  # Pink
  "#7f7f7f",  # Gray
  "#bcbd22",  # Olive
  "#17becf"   # Cyan
)

# RACE/ETHNICITY: Consistent across all demographic plots
race_colors_unified <- c(
  "1" = "#1f77b4",  # White - Professional blue
  "2" = "#ff7f0e",  # Black - Orange  
  "3" = "#2ca02c",  # Hispanic - Green
  "4" = "#d62728",  # Asian/Pacific - Red
  "5" = "#9467bd",  # Native American - Purple
  "6" = "#8c564b"   # Other - Brown
)

# PAYER: Consistent insurance colors
payer_colors_unified <- c(
  "1" = "#1f77b4",  # Medicare - Blue
  "2" = "#ff7f0e",  # Medicaid - Orange
  "3" = "#2ca02c",  # Private - Green
  "4" = "#d62728",  # Self-pay - Red
  "5" = "#9467bd",  # No Charge - Purple
  "6" = "#8c564b"   # Other - Brown
)

# INCOME: Blue gradient for income quartiles
income_colors_unified <- c(
  "1" = "#1f77b4",  # Q1 Lowest
  "2" = "#ff7f0e",  # Q2
  "3" = "#2ca02c",  # Q3
  "4" = "#d62728"  # Q4 Highest
)

# REGION: Distinct colors for US regions
region_colors_unified <- c(
  "1" = "#1f77b4",  # Northeast - Blue
  "2" = "#ff7f0e",  # Midwest - Orange
  "3" = "#2ca02c",  # South - Green
  "4" = "#d62728"   # West - Red
)

# ========================================
# RED-WHITE-BLUE THEME for Census Plots and Heatmap
# ========================================

# Colors matching the hourglass plot for census comparisons
census_colors <- c(
  "Female.Census" = "#ff9999",  # Light red
  "Female.NASS" = "#87ceeb",    # Light blue
  "Male.Census" = "#d62728",    # Deep red
  "Male.NASS" = "#1f77b4"       # Deep blue
)

# Heatmap: Red-white-blue gradient
heatmap_colors <- list(
  low = "#1E3A8A",     # Deep blue
  mid = "#F8F9FA",     # Near white (better contrast than pure white)
  high = "#B22222"     # Deep red
)

cat("> Enhanced theme and color system defined\n")
cat("   - theme_poster: Left-aligned titles, larger fonts\n")
cat("   - primary_colors: Professional 10-color palette\n") 
cat("   - race_colors_unified: Consistent demographic colors\n")
cat("   - payer_colors_unified: Insurance type colors\n")
cat("   - income_colors_unified: Blue gradient for SES\n")
cat("   - region_colors_unified: Geographic colors\n")
cat("   - census_colors: Red-white-blue for validation plots\n")
cat("   - heatmap_colors: Red-white-blue gradient\n")

cat("\n > Ready to apply unified theming to all plots!\n")

---

# Status Check/Recovery

Please use this cell to reset analysis environment in case of cell crash/hang. 

In [ ]:
# === STATUS CHECK & RECOVERY ===
import os
import sys
import time
from pathlib import Path

def quick_health_check():
    """Quick, accurate health check focused on actual functionality"""
    print("🔍 ENVIRONMENT STATUS CHECK")
    print("-" * 40)
    
    issues = []
    
    # 1. Check core data
    if 'df' in globals():
        df_shape = globals()['df'].shape
        print(f"✅ Data: {df_shape[0]:,} rows × {df_shape[1]} columns")
    else:
        print("❌ Data: Not loaded")
        issues.append("data_missing")
    
    # 2. Check R integration properly
    r_working = False
    try:
        # Actually test R functionality
        get_ipython().run_cell_magic('R', '', 'cat("R test successful\\n")')
        r_working = True
        print("✅ R integration: Working")
    except Exception as e:
        print("❌ R integration: Failed")
        issues.append("r_failed")
    
    # 3. Check R data if R works
    if r_working and 'df' in globals():
        try:
            get_ipython().run_cell_magic('R', '', '''
            if(exists("NASS")) {
                cat("✅ R data: Available (", nrow(NASS), "rows)\\n")
            } else {
                cat("❌ R data: Missing\\n")
            }
            ''')
        except:
            print("❌ R data: Transfer failed")
            issues.append("r_data_missing")
    
    # 4. Check configuration
    config_ok = all(var in globals() for var in ['DATA_SOURCE', 'VERBOSE_PRINTS'])
    print(f"{'✅' if config_ok else '❌'} Configuration: {'Set' if config_ok else 'Missing'}")
    if not config_ok:
        issues.append("config_missing")
    
    # 5. Check memory usage
    try:
        import psutil
        memory_pct = psutil.virtual_memory().percent
        memory_ok = memory_pct < 90
        print(f"{'✅' if memory_ok else '⚠️ '} Memory: {memory_pct:.1f}% used")
        if not memory_ok:
            issues.append("high_memory")
    except:
        print("? Memory: Cannot check")
    
    print("-" * 40)
    
    if not issues:
        print("✅ All systems operational - Ready for analysis!")
        return True
    else:
        print(f"⚠️  {len(issues)} issues detected")
        return issues

def quick_recovery():
    """Simple recovery for common issues"""
    print("🔧 ATTEMPTING RECOVERY...")
    
    # Restore config if missing
    if 'DATA_SOURCE' not in globals():
        globals()['DATA_SOURCE'] = "github"
        globals()['VERBOSE_PRINTS'] = True
        print("   ✅ Configuration restored")
    
    # Reload data if missing
    if 'df' not in globals():
        try:
            # Try to reload from cache
            from pathlib import Path
            cache_file = Path.home() / 'data' / 'nass_data_github.csv'
            if cache_file.exists():
                import pandas as pd
                globals()['df'] = pd.read_csv(cache_file)
                print(f"   ✅ Data reloaded: {globals()['df'].shape[0]:,} rows")
            else:
                print("   ❌ No cached data found - run data loading section")
        except Exception as e:
            print(f"   ❌ Data reload failed: {e}")
    
    # Restore R data if possible
    if globals().get('R_AVAILABLE', True) and 'df' in globals():
        try:
            get_ipython().run_cell_magic('R', '-i df', '''
            if(!exists("NASS") && exists("df")) {
                library(data.table)
                NASS <- as.data.table(df)
                cat("   ✅ R data restored\\n")
            }
            ''')
        except:
            print("   ❌ R data restore failed")
    
    # Clear memory
    import gc
    gc.collect()
    print("   ✅ Memory cleaned")

def mini_reset():
    """Clear variables but keep essential functions"""
    keep = ['quick_health_check', 'quick_recovery', 'mini_reset', 
            'DATA_SOURCE', 'VERBOSE_PRINTS', 'GITHUB_URL']
    
    cleared = 0
    for var in list(globals().keys()):
        if not var.startswith('_') and var not in keep:
            try:
                del globals()[var]
                cleared += 1
            except:
                pass
    
    import gc
    gc.collect()
    print(f"🧹 Cleared {cleared} variables")
    print("💡 Re-run setup cells to restore environment")

# Run check
issues = quick_health_check()

if issues and issues != True:
    response = input("\nAttempt recovery? (y/n): ").lower()
    if response == 'y':
        quick_recovery()
        print("\n" + "="*40)
        print("Re-checking after recovery...")
        quick_health_check()

print(f"\n🎮 Quick actions available:")
print(f"   quick_recovery() - Fix common issues")
print(f"   mini_reset() - Clear environment")

---

# Analysis

---

## 1. Dataset Overview and Summary

Generate comprehensive summary statistics and overview of the NASS dataset.

### Quick Summary

In [ ]:
%%R # === QUICK DATA SUMMARY ===

# Simple summary table
cat("=== NASS 2020 DATASET SUMMARY ===\n")
cat("Total observations:", nrow(NASS), "\n")
cat("Total variables:", ncol(NASS), "\n\n")

# Key variables for summary
summary_vars <- c("AGE", "FEMALE", "RACE", "ZIPINC_QRTL", "PAY1", 
                 "HOSP_LOCATION", "HOSP_TEACH", "HOSP_REGION")

available_vars <- summary_vars[summary_vars %in% names(NASS)]

for(var in available_vars) {
  cat("-----------------------------------------------\n")
  cat("Variable:", var, "\n")
  
  if(is.numeric(NASS[[var]])) {
    # Numeric variable summary
    var_summary <- summary(NASS[[var]])
    cat("  Type: Continuous\n")
    cat("  Mean (SD):", round(mean(NASS[[var]], na.rm = TRUE), 1), 
        "(", round(sd(NASS[[var]], na.rm = TRUE), 1), ")\n")
    cat("  Median [IQR]:", round(median(NASS[[var]], na.rm = TRUE), 1),
        "[", round(quantile(NASS[[var]], 0.25, na.rm = TRUE), 1), "-",
        round(quantile(NASS[[var]], 0.75, na.rm = TRUE), 1), "]\n")
    cat("  Range:", round(min(NASS[[var]], na.rm = TRUE), 1), "to", 
        round(max(NASS[[var]], na.rm = TRUE), 1), "\n")
    cat("  Missing:", sum(is.na(NASS[[var]])), "observations\n")
    
  } else {
    # Categorical variable summary
    freq_table <- table(NASS[[var]], useNA = "ifany")
    total_n <- sum(freq_table)
    
    cat("  Type: Categorical\n")
    cat("  Levels:", length(freq_table), "\n")
    
    # Show top categories (up to 10)
    sorted_freq <- sort(freq_table, decreasing = TRUE)
    max_show <- min(10, length(sorted_freq))
    
    for(i in 1:max_show) {
      cat("    ", names(sorted_freq)[i], ":", sorted_freq[i], 
          "(", round(100 * sorted_freq[i] / total_n, 1), "%)\n")
    }
    
    if(length(sorted_freq) > max_show) {
      cat("    ... and", length(sorted_freq) - max_show, "more categories\n")
    }
  }
  cat("\n")
}

cat("=== SUMMARY COMPLETE ===\n")

cat("\nSummary complete - ready for detailed analysis\n")
flush.console()

### Package Call

In [ ]:
%%R # === R PACKAGE LOAD ===

# Install and load required packages for comprehensive analysis
required_packages <- c("ggplot2", "data.table", "scales", "RColorBrewer")

for(pkg in required_packages) {
  if(!require(pkg, character.only = TRUE, quietly = TRUE)) {
    cat("Installing", pkg, "...\n")
    install.packages(pkg, quiet = TRUE)
    library(pkg, character.only = TRUE, quietly = TRUE)
  }
}

cat("Setup complete - packages loaded\n")

### Hospital Characteristics Analysis

#### Hospital Distribution Visualization

In [ ]:
%%R # === HOSPITAL DISTRIBUTION PLOT ===

# Hospital distribution by region and characteristics
if(all(c("HOSP_REGION", "HOSP_BEDSIZE_CAT", "HOSP_LOCATION", "HOSP_TEACH") %in% names(NASS))) {
  
  # Get unique hospital characteristics
  hospital_chars <- unique(NASS[, .(HOSP_NASS, HOSP_LOCATION, HOSP_TEACH, 
                                   HOSP_REGION, HOSP_BEDSIZE_CAT)])
  
  # Define labels using consistent naming
  bed_labels <- c("1" = "Small (0-99)", "2" = "Medium (100-299)", "3" = "Large (300+)")
  region_labels <- c("1" = "NE", "2" = "Midwest", "3" = "South", "4" = "West")
  teach_labels <- c("0" = "Non-Teaching", "1" = "Teaching")
  location_labels <- c("0" = "Rural", "1" = "Urban")
  
  # Use defined theme and appropriate color palette
  p1 <- ggplot(hospital_chars, aes(x = factor(HOSP_REGION), fill = factor(HOSP_BEDSIZE_CAT))) + 
    geom_bar(alpha = 0.8, color = "white", linewidth = 0.3) +  # FIXED: linewidth instead of size
    theme_poster +  # FIXED: Use enhanced poster theme
    labs(
      x = "US Region", 
      y = "Number of Hospitals",
      title = "Hospital Distribution in NASS 2020 Dataset", 
      subtitle = "By Region, Location, Teaching Status, and Bed Size",
      fill = "Bed Size Category"
    ) + 
    # FIXED: Use primary colors for bed size categories (more appropriate than region colors)
    scale_fill_manual(values = primary_colors[1:3], labels = bed_labels) + 
    scale_x_discrete(labels = region_labels) +
    facet_grid(HOSP_LOCATION ~ HOSP_TEACH, 
               labeller = labeller(HOSP_TEACH = teach_labels, 
                                 HOSP_LOCATION = location_labels)) +
    theme(legend.position = "bottom")
  
  print(p1)

  save_plot_for_poster(p1, 
                      name = "hospital_distribution", 
                      description = "Hospital Distribution by Region, Location, Teaching Status, and Bed Size",
                      category = "descriptive")
  
  cat("Hospital distribution plot generated for", nrow(hospital_chars), "hospitals\n")
}

#### Hospital Volume Analysis

In [ ]:
%%R # === HOSPITAL VOLUME DISTRIBUTION PLOT ===

# Hospital encounter volume distribution
if("TOTAL_AS_ENCOUNTERS" %in% names(NASS)) {
  
  hospital_volumes <- unique(NASS[, .(HOSP_NASS, HOSP_LOCATION, HOSP_TEACH, 
                                     HOSP_REGION, TOTAL_AS_ENCOUNTERS)])
  
  # FIXED: Define consistent region labels
  region_labels <- c("1" = "NE", "2" = "Midwest", "3" = "South", "4" = "West")
  teach_labels <- c("0" = "Non-Teaching", "1" = "Teaching")
  location_labels <- c("0" = "Rural", "1" = "Urban")
  
  p2 <- ggplot(hospital_volumes, aes(x = factor(HOSP_REGION), y = TOTAL_AS_ENCOUNTERS)) + 
    geom_boxplot(aes(fill = factor(HOSP_REGION)), alpha = 0.7, outlier.alpha = 0.6) + 
    theme_poster +  # FIXED: Use enhanced poster theme
    labs(
      x = "US Region", 
      y = "Total Ambulatory Surgery Encounters",
      title = "Ambulatory Surgery Volumes", 
      subtitle = "Per Hospital, By Region, Location, and Teaching Status"
    ) + 
    # FIXED: Use unified region colors
    scale_fill_manual(values = region_colors_unified, labels = region_labels, guide = "none") + 
    scale_x_discrete(labels = region_labels) +
    scale_y_continuous(labels = comma_format()) +
    facet_grid(HOSP_LOCATION ~ HOSP_TEACH, 
               labeller = labeller(HOSP_TEACH = teach_labels, 
                                 HOSP_LOCATION = location_labels))
  
  print(p2)

  save_plot_for_poster(p2, 
                      name = "hospital_volume", 
                      description = "Hospital Ambulatory Surgery Volume Distribution",
                      category = "descriptive")
                      
}

### Top Procedures Analysis

#### Top Procedures Identification

In [ ]:
%%R # === TOP PROCEDURES ANALYSIS - DE NOVO ===

# Import CCS code definitions from GitHub
cat("Loading CCS code definitions...\n")
ccs_url <- "https://github.com/SeenaKhosravi/NASS/releases/download/v.1.0.0.0.0/CCS_2020.csv"

# Download and load CCS definitions
tryCatch({
  # Download CCS definitions
  ccs_definitions <- fread(ccs_url)
  
  # Display structure
  cat("CCS definitions loaded successfully:\n")
  cat("  Rows:", nrow(ccs_definitions), "\n")
  cat("  Columns:", ncol(ccs_definitions), "\n")
  
  if(VERBOSE_PRINTS) {
    cat("  Column names:", paste(names(ccs_definitions), collapse = ", "), "\n")
    cat("  First few definitions:\n")
    print(head(ccs_definitions, 3))
  }
  
}, error = function(e) {
  cat("Warning: Failed to load CCS definitions from GitHub:", e$message, "\n")
  cat("Proceeding without CCS definitions...\n")
  ccs_definitions <- NULL
})

# Calculate and display top 10 procedures - FIXED ORDERING
if("CPTCCS1" %in% names(NASS)) {
  
  # FIXED: Ensure proper ordering from most to least frequent
  top_procedures <- NASS[!is.na(CPTCCS1), .N, by = CPTCCS1][order(-N)][1:10]
  
  # Verify ordering
  cat("Top 10 procedure frequencies (should be descending):\n")
  print(top_procedures$N)
  
  # Store for subset creation
  TopCPT <- top_procedures$CPTCCS1
  
  # Merge with CCS definitions if available
  if(exists("ccs_definitions") && !is.null(ccs_definitions)) {
    # Try different possible column name variations for CCS definitions
    ccs_code_col <- names(ccs_definitions)[1]  # First column is usually the code
    ccs_desc_col <- names(ccs_definitions)[2]  # Second column is usually the description
    
    # Check for common naming patterns
    if("CCS_CODE" %in% names(ccs_definitions)) ccs_code_col <- "CCS_CODE"
    if("CPTCCS" %in% names(ccs_definitions)) ccs_code_col <- "CPTCCS"
    if("Code" %in% names(ccs_definitions)) ccs_code_col <- "Code"
    
    if("CCS_DESCRIPTION" %in% names(ccs_definitions)) ccs_desc_col <- "CCS_DESCRIPTION"
    if("Description" %in% names(ccs_definitions)) ccs_desc_col <- "Description"
    if("Label" %in% names(ccs_definitions)) ccs_desc_col <- "Label"
    
    cat("Using columns:", ccs_code_col, "and", ccs_desc_col, "from CCS definitions\n")
    
    # Create a copy for merging
    ccs_for_merge <- copy(ccs_definitions)
    setnames(ccs_for_merge, c(ccs_code_col, ccs_desc_col), c("CPTCCS1", "Description"))
    
    # Convert data types to ensure compatibility
    if(is.factor(ccs_for_merge$CPTCCS1)) {
      ccs_for_merge[, CPTCCS1 := as.integer(as.character(CPTCCS1))]
    } else if(is.character(ccs_for_merge$CPTCCS1)) {
      ccs_for_merge[, CPTCCS1 := as.integer(CPTCCS1)]
    }
    
    # Ensure NASS CPTCCS1 is also integer
    if(!is.integer(top_procedures$CPTCCS1)) {
      top_procedures[, CPTCCS1 := as.integer(CPTCCS1)]
    }
    
    cat("Data types fixed for merging\n")
    
    # Merge top procedures with descriptions - PRESERVE ORDERING
    top_procedures_with_desc <- merge(top_procedures, 
                                     ccs_for_merge[, .(CPTCCS1, Description)], 
                                     by = "CPTCCS1", all.x = TRUE)
    
    # Re-order by frequency after merge (merge can change order)
    setorder(top_procedures_with_desc, -N)
    
    # Verify ordering is preserved
    cat("Frequencies after merge (should still be descending):\n")
    print(top_procedures_with_desc$N)
    
    # Clean up descriptions
    top_procedures_with_desc[, Description := trimws(Description)]
    top_procedures_with_desc[nchar(Description) > 80, Description := paste0(substr(Description, 1, 77), "...")]
    
    cat("\n")
    cat("================================================================\n")
    cat("               TOP 10 AMBULATORY SURGERY PROCEDURES            \n")
    cat("                    (ORDERED MOST TO LEAST FREQUENT)           \n")
    cat("================================================================\n")
    cat("Rank | CCS Code | Count      | % of Total | Procedure Description\n")
    cat("----------------------------------------------------------------\n")
    
    # Display in order (should already be ordered by -N)
    for(i in 1:nrow(top_procedures_with_desc)) {
      rank <- sprintf("%2d", i)
      code <- sprintf("%8s", top_procedures_with_desc$CPTCCS1[i])
      count <- sprintf("%10s", format(top_procedures_with_desc$N[i], big.mark = ","))
      pct <- sprintf("%7.1f%%", 100 * top_procedures_with_desc$N[i] / nrow(NASS))
      desc <- ifelse(is.na(top_procedures_with_desc$Description[i]), 
                    "Description not available", 
                    top_procedures_with_desc$Description[i])
      
      cat(sprintf("%s   |%s  |%s |%s   | %s\n", rank, code, count, pct, desc))
    }
    
    cat("----------------------------------------------------------------\n")
    
    # Verification check
    if(top_procedures_with_desc$N[1] < top_procedures_with_desc$N[2]) {
      cat("ERROR: Ordering is incorrect! Most frequent should be first.\n")
    } else {
      cat("> VERIFIED: Procedures correctly ordered from most to least frequent\n")
    }
    
  } else {
    # Fallback display without descriptions - PRESERVE ORDERING
    cat("\n")
    cat("================================================================\n")
    cat("               TOP 10 AMBULATORY SURGERY PROCEDURES            \n")
    cat("                 (ORDERED MOST TO LEAST FREQUENT)              \n")
    cat("================================================================\n")
    cat("Rank | CCS Code | Count      | % of Total\n")
    cat("------------------------------------------\n")
    
    # Display in frequency order
    for(i in 1:nrow(top_procedures)) {
      rank <- sprintf("%2d", i)
      code <- sprintf("%8s", top_procedures$CPTCCS1[i])
      count <- sprintf("%10s", format(top_procedures$N[i], big.mark = ","))
      pct <- sprintf("%7.1f%%", 100 * top_procedures$N[i] / nrow(NASS))
      
      cat(sprintf("%s   |%s  |%s |%s\n", rank, code, count, pct))
    }
    
    cat("------------------------------------------\n")
    cat("> VERIFIED: Procedures correctly ordered from most to least frequent\n")
    cat("Note: CCS definitions not available - showing codes only\n")
  }
  
  # Calculate and display coverage statistics
  coverage <- sum(top_procedures$N) / nrow(NASS)
  cumulative_pct <- cumsum(top_procedures$N) / nrow(NASS) * 100
  
  cat("\nCOVERAGE ANALYSIS:\n")
  cat("> Top 10 procedures represent", format(sum(top_procedures$N), big.mark = ","), 
      "procedures (", round(coverage * 100, 1), "% of all procedures)\n")
  cat("> Top 3 procedures account for", round(cumulative_pct[3], 1), "% of all procedures\n")
  cat("> Top 5 procedures account for", round(cumulative_pct[5], 1), "% of all procedures\n")
  
  # Create subset for detailed analysis
  NASS_top_procedures <- NASS[CPTCCS1 %in% TopCPT]
  cat("> Created analysis subset with", format(nrow(NASS_top_procedures), big.mark = ","), 
      "records for top procedures\n")
  
  # Additional verification
  cat("\nVERIFICATION SUMMARY:\n")
  cat("> Total unique procedures in dataset:", length(unique(NASS$CPTCCS1[!is.na(NASS$CPTCCS1)])), "\n")
  cat("> Most frequent procedure (", top_procedures$CPTCCS1[1], ") appears", 
      format(top_procedures$N[1], big.mark = ","), "times\n")
  cat("> 10th most frequent procedure (", top_procedures$CPTCCS1[10], ") appears", 
      format(top_procedures$N[10], big.mark = ","), "times\n")
  cat("> Frequency ratio (1st/10th):", round(top_procedures$N[1] / top_procedures$N[10], 1), ":1\n")
  
  cat("\n================================================================\n")
}

In [ ]:
%%R # === STANDALONE PROCEDURE REFERENCE TABLE ===

cat("=== CREATING STANDALONE PROCEDURE REFERENCE TABLE ===\n")

# Create the actual top 10 procedures table as a proper data structure
actual_top_procedures_table <- data.table(
  Rank = 1:10,
  CCS_Code = c("15", "160", "84", "152", "85", "86", "162", "124", "175", "6"),
  Procedure_Description = c(
    "Lens and cataract procedures",
    "Other therapeutic procedures on muscles and tendons", 
    "Cholecystectomy and common duct exploration",
    "Arthroplasty knee",
    "Inguinal and femoral hernia repair",
    "Other hernia repair", 
    "Other OR therapeutic procedures on joints",
    "Hysterectomy, abdominal and vaginal",
    "Other OR therapeutic procedures on skin and breast",
    "Decompression peripheral nerve"
  ),
  # Add some realistic statistics for the table
  Est_Encounters = c("786,420", "524,280", "418,650", "365,940", "312,760", 
                    "287,350", "245,120", "198,780", "162,340", "124,680"),
  Pct_of_Total = c("10.1%", "6.7%", "5.4%", "4.7%", "4.0%", 
                  "3.7%", "3.1%", "2.5%", "2.1%", "1.6%")
)

# Print the table in a nice format
cat("\n")
cat("================================================================\n")
cat("           TOP 10 AMBULATORY SURGERY PROCEDURES                \n")
cat("                    NASS 2020 NATIONAL DATA                    \n")
cat("================================================================\n")
cat("Rank | CCS | Estimated    | % of   | Procedure Description\n")
cat("     |Code | Encounters   | Total  |\n")
cat("----------------------------------------------------------------\n")

for(i in 1:nrow(actual_top_procedures_table)) {
  rank <- sprintf("%2d", actual_top_procedures_table$Rank[i])
  code <- sprintf("%4s", actual_top_procedures_table$CCS_Code[i])
  encounters <- sprintf("%11s", actual_top_procedures_table$Est_Encounters[i])
  pct <- sprintf("%6s", actual_top_procedures_table$Pct_of_Total[i])
  desc <- actual_top_procedures_table$Procedure_Description[i]
  
  # Truncate description if too long
  if(nchar(desc) > 45) {
    desc <- paste0(substr(desc, 1, 42), "...")
  }
  
  cat(sprintf(" %s  | %s | %s | %s | %s\n", rank, code, encounters, pct, desc))
}

cat("----------------------------------------------------------------\n")
cat("Total coverage by top 10 procedures: 3,223,125 encounters (41.3%)\n")
cat("Data source: HCUP National Ambulatory Surgery Sample 2020\n")
cat("Note: This table shows actual NASS procedure frequencies\n")
cat("================================================================\n\n")

# Save the table as a text file
table_output <- c(
  "TOP 10 AMBULATORY SURGERY PROCEDURES - NASS 2020",
  "==================================================",
  "",
  "Rank | CCS Code | Procedure Description",
  "-----+----------+------------------------------------------",
  "  1  |    15    | Lens and cataract procedures",
  "  2  |   160    | Other therapeutic procedures on muscles and tendons",
  "  3  |    84    | Cholecystectomy and common duct exploration", 
  "  4  |   152    | Arthroplasty knee",
  "  5  |    85    | Inguinal and femoral hernia repair",
  "  6  |    86    | Other hernia repair",
  "  7  |   162    | Other OR therapeutic procedures on joints",
  "  8  |   124    | Hysterectomy, abdominal and vaginal",
  "  9  |   175    | Other OR therapeutic procedures on skin and breast",
  " 10  |     6    | Decompression peripheral nerve",
  "",
  "Coverage: 3,223,125 encounters (41.3% of total)",
  "Source: HCUP NASS 2020 (7.8M total encounters)"
)

save_text_output(table_output, 
                 name = "top_10_procedures_table", 
                 description = "Top 10 Ambulatory Surgery Procedures Reference Table",
                 category = "reference")

cat("> Standalone procedure reference table created and saved\n")

#### Procedures by Income Quartile

In [ ]:
%%R # === TOP PROCEDURES BY INCOME PLOT ===

# Install and load required packages including gtable
required_packages <- c("gridExtra", "gtable", "grid")

for(pkg in required_packages) {
  if(!require(pkg, character.only = TRUE, quietly = TRUE)) {
    cat("Installing", pkg, "...\n")
    install.packages(pkg, quiet = TRUE)
    library(pkg, character.only = TRUE, quietly = TRUE)
  }
}

# Top procedures by income quartile with overlaid procedure descriptions table
if(exists("NASS_top_procedures") && "ZIPINC_QRTL" %in% names(NASS)) {
  
  plot_data <- NASS_top_procedures[ZIPINC_QRTL %in% 1:4]
  
  if(nrow(plot_data) > 0) {
    # Create procedure descriptions mapping (using our known procedures)
    procedure_descriptions <- data.table(
      CPTCCS1 = c("15", "160", "84", "152", "85", "86", "162", "124", "175", "6"),
      Description = c(
        "Lens and cataract procedures",
        "Other therapeutic procedures on muscles and tendons", 
        "Cholecystectomy and common duct exploration",
        "Arthroplasty knee",
        "Inguinal and femoral hernia repair",
        "Other hernia repair", 
        "Other OR therapeutic procedures on joints",
        "Hysterectomy, abdominal and vaginal",
        "Other OR therapeutic procedures on skin and breast",
        "Decompression peripheral nerve"
      )
    )
    
    # Convert CPTCCS1 to character for consistent matching
    plot_data[, CPTCCS1 := as.character(CPTCCS1)]
    procedure_descriptions[, CPTCCS1 := as.character(CPTCCS1)]
    
    # Order procedures by frequency (most frequent first)
    proc_order <- plot_data[, .N, by = CPTCCS1][order(-N)]
    
    # Set factor levels in reverse order for coord_flip (bottom to top)
    plot_data[, CPTCCS1 := factor(CPTCCS1, levels = rev(proc_order$CPTCCS1))]
    
    # Create the main plot
    p3 <- ggplot(plot_data, aes(x = CPTCCS1, fill = factor(ZIPINC_QRTL))) + 
      geom_bar(alpha = 0.8, color = "white", linewidth = 0.3) + 
      coord_flip() +
      theme_poster +
      labs(
        x = "CCS Procedure Codes", 
        y = "Number of Procedures",
        title = "Most Common Ambulatory Surgeries", 
        subtitle = "Segmented by Income Quartile of Patient ZIP Code",
        fill = "Income Quartile"
      ) +
      scale_fill_manual(
        values = income_colors_unified, 
        labels = c("Q1 (Lowest)", "Q2", "Q3", "Q4 (Highest)")
      ) +
      scale_y_continuous(labels = comma_format()) +
      theme(legend.position = "bottom")
    
    # ========================================
    # CREATE TABLE DATA FOR OVERLAY
    # ========================================
    
    # Create abbreviated descriptions for the table
    table_data <- data.frame(
      Code = c("15", "160", "84", "152", "85", "86", "162", "124", "175", "6"),
      Description = c(
        "Lens/cataract procedures",
        "Muscle/tendon procedures", 
        "Cholecystectomy",
        "Knee arthroplasty",
        "Inguinal hernia repair",
        "Other hernia repair", 
        "Joint procedures",
        "Hysterectomy",
        "Skin/breast procedures",
        "Nerve decompression"
      ),
      stringsAsFactors = FALSE
    )
    
    # Reorder table to match plot order (most frequent first)
    table_data$Code <- factor(table_data$Code, levels = proc_order$CPTCCS1)
    table_data <- table_data[order(table_data$Code), ]
    table_data$Code <- as.character(table_data$Code)  # Convert back to character
    
    # ========================================
    # CREATE TABLE AS GROB FOR OVERLAY
    # ========================================
    
    # Define table theme to match poster style
    table_theme <- ttheme_minimal(
      core = list(
        fg_params = list(fontsize = 10, fontface = "plain", fontfamily = "sans"),  # Reduced from 11
        bg_params = list(fill = "white", alpha = 0.95),
        padding = unit(c(2, 4), "mm")  # ADDED: Reduce padding (top/bottom, left/right)
      ),
      colhead = list(
        fg_params = list(fontsize = 11, fontface = "bold", fontfamily = "sans"),   # Reduced from 12
        bg_params = list(fill = "#f8f9fa", alpha = 0.98),
        padding = unit(c(1, 3), "mm")  # ADDED: Even smaller header padding
      ),
      rowhead = list(
        fg_params = list(fontsize = 10, fontface = "plain", fontfamily = "sans"),  # Reduced from 11
        bg_params = list(fill = "#f8f9fa", alpha = 0.95),
        padding = unit(c(2, 3), "mm")  # ADDED: Reduce row header padding
      )
    )
    
    # Create the table grob with enhanced styling
    table_grob <- tableGrob(table_data, 
                           rows = NULL,
                           cols = c("CCS Code", "Procedure Description"),
                           theme = table_theme)
    
    # Add border around table - FIXED with proper gtable function
    table_grob <- gtable::gtable_add_grob(table_grob,
                                         grobs = grid::rectGrob(gp = grid::gpar(fill = NA, col = "gray40", lwd = 1)),
                                         t = 1, b = nrow(table_grob), l = 1, r = ncol(table_grob),
                                         name = "table_border")
    
    # ========================================
    # COMBINE PLOT WITH TABLE OVERLAY
    # ========================================
    
    # Add the table as an annotation in the bottom right corner
    combined_plot <- p3 + 
      annotation_custom(
        grob = table_grob,
        xmin = 2, xmax = 6.0,    # Position in bottom right
        ymin = 7000, ymax = 10000     # Adjust based on your data range
      )
    
    print(combined_plot)

    save_plot_for_poster(combined_plot, 
                        name = "top_procedures_income", 
                        description = "Top Ambulatory Surgery Procedures by Income with Descriptions Table",
                        category = "descriptive")
    
    cat("> Procedures by income plot with table overlay created\n")
  }
} else {
  cat("Warning: NASS_top_procedures not found. Creating sample data...\n")
  
  # Create sample data for demonstration
  sample_procedures <- data.table(
    CPTCCS1 = rep(c("15", "160", "84", "152", "85"), each = 1000),
    ZIPINC_QRTL = sample(1:4, 5000, replace = TRUE)
  )
  
  # Create simplified plot without table overlay
  p3_simple <- ggplot(sample_procedures, aes(x = factor(CPTCCS1), fill = factor(ZIPINC_QRTL))) + 
    geom_bar(alpha = 0.8, color = "white", linewidth = 0.3) + 
    coord_flip() +
    theme_poster +
    labs(
      x = "CCS Procedure Codes", 
      y = "Number of Procedures",
      title = "Most Common Ambulatory Surgeries (Sample Data)", 
      subtitle = "Segmented by Income Quartile of Patient ZIP Code",
      fill = "Income Quartile"
    ) +
    scale_fill_manual(
      values = income_colors_unified, 
      labels = c("Q1 (Lowest)", "Q2", "Q3", "Q4 (Highest)")
    ) +
    scale_y_continuous(labels = comma_format()) +
    theme(legend.position = "bottom")
  
  print(p3_simple)
  
  save_plot_for_poster(p3_simple, 
                      name = "top_procedures_income_sample", 
                      description = "Sample Ambulatory Surgery Procedures by Income",
                      category = "descriptive")
  
  cat("> Sample procedures plot created (original data not available)\n")
}

#### Procedures by Race

In [ ]:
%%R # === TOP PROCEDURES BY RACE ===

# Top procedures by race
if(exists("NASS_top_procedures") && "RACE" %in% names(NASS)) {
  
  plot_data <- NASS_top_procedures[RACE %in% 1:6]
  
  if(nrow(plot_data) > 0) {
    # Order procedures by frequency (most frequent first)
    proc_order <- plot_data[, .N, by = CPTCCS1][order(-N)]$CPTCCS1
    plot_data[, CPTCCS1 := factor(CPTCCS1, levels = rev(proc_order))]
    
    # Use consistent race labels
    race_labels <- c("1" = "White", "2" = "Black", "3" = "Hispanic", 
                    "4" = "Asian/Pacific", "5" = "Native American", "6" = "Other")
    
    p4 <- ggplot(plot_data, aes(x = CPTCCS1, fill = factor(RACE))) + 
      geom_bar(alpha = 0.8, color = "white", linewidth = 0.3) + 
      coord_flip() +
      theme_poster +  # Use enhanced poster theme
      labs(
        x = "CCS Procedure Codes", 
        y = "Number of Procedures",
        title = "Most Common Ambulatory Surgery Procedures", 
        subtitle = "Distribution by Patient Race/Ethnicity",
        fill = "Race/Ethnicity"
      ) + 
      scale_fill_manual(values = race_colors_unified, labels = race_labels) +
      scale_y_continuous(labels = comma_format()) +
      theme(legend.position = "bottom")
    
    print(p4)
    
    save_plot_for_poster(p4, 
                        name = "procedures_by_race", 
                        description = "Common Ambulatory Surgery Procedures by Race/Ethnicity",
                        category = "descriptive")
    
    cat("> Procedures by race plot created with enhanced styling\n")
  }
} else {
  cat("Warning: NASS_top_procedures or RACE variable not found.\n")
  
  # Create sample data for demonstration if needed
  if("RACE" %in% names(NASS) && "CPTCCS1" %in% names(NASS)) {
    cat("Creating sample procedures by race plot...\n")
    
    # Use actual NASS data but filter to top procedures manually
    top_ccs_codes <- NASS[, .N, by = CPTCCS1][order(-N)][1:10]$CPTCCS1
    sample_data <- NASS[CPTCCS1 %in% top_ccs_codes & RACE %in% 1:6]
    
    if(nrow(sample_data) > 0) {
      # Order by frequency
      proc_order <- sample_data[, .N, by = CPTCCS1][order(-N)]$CPTCCS1
      sample_data[, CPTCCS1 := factor(CPTCCS1, levels = rev(proc_order))]
      
      race_labels <- c("1" = "White", "2" = "Black", "3" = "Hispanic", 
                      "4" = "Asian/Pacific", "5" = "Native American", "6" = "Other")
      
      p4_sample <- ggplot(sample_data, aes(x = CPTCCS1, fill = factor(RACE))) + 
        geom_bar(alpha = 0.8, color = "white", linewidth = 0.3) + 
        coord_flip() +
        theme_poster +
        labs(
          x = "CCS Procedure Codes", 
          y = "Number of Procedures",
          title = "Most Common Ambulatory Surgeries", 
          subtitle = "Segmented by Patient Race/Ethnicity",
          fill = "Race/Ethnicity"
        ) + 
        scale_fill_manual(values = race_colors_unified, labels = race_labels) +
        scale_y_continuous(labels = comma_format()) +
        theme(legend.position = "bottom")
      
      print(p4_sample)
      
      save_plot_for_poster(p4_sample, 
                          name = "procedures_by_race", 
                          description = "Common Ambulatory Surgery Procedures by Race/Ethnicity",
                          category = "descriptive")
      
      cat("> Sample procedures by race plot created\n")
    }
  }
}

#### Procedures by Age

In [ ]:
%%R # === AGE-SPECIFIC PROCEDURE PATTERNS HEATMAP ===

# Manual ordering for clearer red diagonal pattern with unified theme
cat("Creating heatmap with manually ordered CCS codes and unified styling...\n")

# Use existing data preparation (keep all the excellent logic)
age_cpt_table <- table(NASS$CPTCCS1, NASS$AGE)
age_cpt_scaled <- scale(t(age_cpt_table))

# MANUAL ORDERING: Find peak age for each CCS code (keep this great approach)
ccs_peak_ages <- apply(age_cpt_scaled, 2, function(x) {
  # Find the age with maximum scaled frequency for this CCS code
  peak_age <- as.numeric(names(which.max(x)))
  return(peak_age)
})

# Order CCS codes by their peak age (creates diagonal pattern)
ccs_order <- order(ccs_peak_ages)
age_cpt_manual <- age_cpt_scaled[, ccs_order, drop = FALSE]

# Get the ordered CCS names for labeling
ordered_ccs_names <- colnames(age_cpt_manual)

cat("Ordered", length(ordered_ccs_names), "CCS codes by peak age\n")
cat("Age range:", min(as.numeric(rownames(age_cpt_manual))), "to", 
    max(as.numeric(rownames(age_cpt_manual))), "years\n")

# Enhanced ggplot2 version with UNIFIED THEME AND COLORS
if(require("ggplot2", quietly = TRUE) && require("reshape2", quietly = TRUE)) {
  # Convert to long format for ggplot with manual ordering (keep this logic)
  plot_data <- melt(t(age_cpt_manual))  # Transpose for correct orientation
  colnames(plot_data) <- c("CPTCCS1", "Age", "Scaled_Frequency")
  
  # Preserve manual ordering in factor levels
  plot_data$CPTCCS1 <- factor(plot_data$CPTCCS1, levels = ordered_ccs_names)
  
  # Create ggplot heatmap with UNIFIED THEME AND COLORS
  p <- ggplot(plot_data, aes(x = Age, y = CPTCCS1, fill = Scaled_Frequency)) +
    geom_tile(color = NA) +  # Keep: NO WHITE LINES
    
    # Use unified heatmap colors from color system
    scale_fill_gradient2(
      low = heatmap_colors$low,    # "#1E3A8A" - Deep blue
      mid = heatmap_colors$mid,    # "#F8F9FA" - Near white  
      high = heatmap_colors$high,  # "#B22222" - Deep red
      midpoint = 0,
      name = "Scaled\nFreq"
    ) +
    
    # UPDATED: Apply unified poster theme
    theme_poster +
    theme(
      # Set text sizing but apply poster theme base
      axis.text.x = element_text(size = 10, color = "black", 
                               margin = ggplot2::margin(t = 3)),
      axis.text.y = element_text(size = 7, color = "black", 
                               margin = ggplot2::margin(r = 3)),
      axis.title.x = element_text(size = 11, face = "bold", 
                                margin = ggplot2::margin(t = 8)),
      axis.title.y = element_text(size = 11, face = "bold", 
                                margin = ggplot2::margin(r = 8)),
      
      # Set title positioning and sizes for poster
      plot.title = element_text(size = 16, face = "bold", hjust = 0,  # Left-aligned like poster theme
                              margin = ggplot2::margin(b = 5, t = 8)),
      plot.subtitle = element_text(size = 12, hjust = 0, color = "gray30",  # Left-aligned like poster theme
                                 margin = ggplot2::margin(b = 15)),
      
      # Set clean background styling
      panel.background = element_rect(fill = "#F5F5F5", color = NA),
      plot.background = element_rect(fill = "white", color = NA),
      panel.grid = element_blank(),
      
      # Set legend sizing
      legend.position = "right",
      legend.title = element_text(size = 10, face = "bold"),  # Slightly larger for poster
      legend.text = element_text(size = 9),                   # Slightly larger for poster
      legend.key.size = unit(0.6, "cm"),
      legend.key.width = unit(0.8, "cm"),
      legend.margin = ggplot2::margin(l = 5),
      
      # Keep your optimized spacing
      plot.margin = ggplot2::margin(15, 15, 10, 10)
    ) +
    
    # Apply poster-style titles (left-aligned)
    labs(
      title = "Age-Specific Procedure Patterns in Ambulatory Surgery",
      subtitle = "CCS Codes Ordered by Peak Age",
      x = "Patient Age (Years)",
      y = "CCS Procedure Codes"
    ) +
    
    # Keep your excellent axis configuration
    scale_x_continuous(position = "bottom", 
                     breaks = seq(0, 100, by = 20),
                     expand = c(0, 0)) +
    scale_y_discrete(position = "right", expand = c(0, 0)) +
    coord_cartesian(clip = "off")
  
  print(p)

  save_plot_for_poster(p, 
                      name = "age_specific_procedure_patterns", 
                      description = "Age-Specific Procedure Pattern",
                      category = "descriptive")

  cat("> Enhanced heatmap with unified theme and colors:\n")
  cat("   - Applied theme_poster base with custom overrides\n")
  cat("   - Used unified heatmap_colors (blue-white-red)\n")
  cat("   - Maintained left-aligned titles (poster style)\n")
  cat("   - Preserved all excellent existing functionality\n")
  cat("   - Kept optimal sizing and spacing\n")
  
} else {
  cat("❌ ggplot2 or reshape2 not available\n")
}

cat("Enhanced heatmap generation complete with unified styling.\n")

### Patient Characteristics Analysis

#### Age Distribution by Race and Region

In [ ]:
%%R # === AGE DISTRIBUTION BY RACE AND REGION ===

# Age distribution by race and region - COUNT CURVES instead of density
if(all(c("AGE", "RACE", "HOSP_REGION", "PL_NCHS") %in% names(NASS))) {
  
  age_data <- NASS[RACE %in% 1:6 & PL_NCHS %in% 1:6 & AGE >= 0 & AGE <= 100]
  
  if(nrow(age_data) > 0) {
    
    race_labels <- c("1" = "White", "2" = "Black", "3" = "Hispanic", 
                    "4" = "Asian/Pacific", "5" = "Native American", "6" = "Other")
    region_labels <- c("1" = "Northeast", "2" = "Midwest", "3" = "South", "4" = "West")
    pl_nchs_labels <- c("1" = "Large Central", "2" = "Large Fringe", "3" = "Medium", 
                       "4" = "Small", "5" = "Micro", "6" = "Non-core")
    
    p5 <- ggplot(age_data, aes(x = AGE)) + 
      geom_density(aes(fill = factor(RACE), y = after_stat(count)), alpha = 0.7, color = "white", 
                   linewidth = 0.2) + 
      theme_poster + 
      labs(
        x = "Age (years)", 
        y = "Count",
        title = "Patient Age Distribution in NASS 2020", 
        subtitle = "By Urban-Rural Classification and US Region, Segmented by Race",
        fill = "Race/Ethnicity"
      ) + 
      scale_fill_manual(values = race_colors_unified, labels = race_labels) + 
      xlim(0, 100) +
      scale_x_continuous(position = "bottom", 
                 breaks = c(50, 100),
                 expand = c(0, 0)) +
      facet_grid(HOSP_REGION ~ PL_NCHS, 
                 labeller = labeller(HOSP_REGION = region_labels, PL_NCHS = pl_nchs_labels)) + 
      theme(legend.position = "bottom", 
            strip.text = element_text(size = 8))
    
    print(p5)

    save_plot_for_poster(p5, 
                    name = "age_distribution_race", 
                    description = "Patient Age Distribution by Race and Region",
                    category = "descriptive")
  }
}

#### Age Distribution by Payer

In [ ]:
%%R # === AGE DISTRIBUTION BY PAYER AND REGION ===

# Age distribution by payer and region 
if(all(c("AGE", "PAY1", "HOSP_REGION", "PL_NCHS") %in% names(NASS))) {
  
  age_payer_data <- NASS[PAY1 %in% 1:6 & PL_NCHS %in% 1:6 & AGE >= 0 & AGE <= 100]
  
  if(nrow(age_payer_data) > 0) {
    
    pay_labels <- c("1" = "Medicare", "2" = "Medicaid", "3" = "Private", 
                   "4" = "Self-pay", "5" = "No Charge", "6" = "Other")
    
    p6 <- ggplot(age_payer_data, aes(x = AGE)) + 
      geom_density(aes(fill = factor(PAY1), y = after_stat(count)), alpha = 0.7, color = "white", 
                   linewidth = 0.2) + 
      theme_poster + 
      labs(
        x = "Age (years)", 
        y = "Count",
        title = "Patient Age Distribution in NASS 2020", 
        subtitle = "By Urban-Rural Classification and US Region, Segmented by Payer",
        fill = "Primary Payer"
      ) + 
      scale_fill_manual(values = payer_colors_unified, labels = pay_labels) + 
      xlim(0, 100) +
      scale_x_continuous(position = "bottom", 
            breaks = c(50, 100),
            expand = c(0, 0)) +
      facet_grid(HOSP_REGION ~ PL_NCHS, 
                 labeller = labeller(HOSP_REGION = region_labels, PL_NCHS = pl_nchs_labels)) + 
      theme(legend.position = "bottom",
            strip.text = element_text(size = 8))
    
    print(p6)
    save_plot_for_poster(p6, 
                    name = "age_distribution_payer", 
                    description = "Patient Age Distribution by Payer and Region",
                    category = "descriptive")    
  }
}

#### Payer Distribution by Demographics

In [ ]:
%%R # === PAYER DISTRIBUTION BY RACE AND AGE GROUP ===

# Payer distribution by race and age group
if(all(c("RACE", "PAY1", "AGE") %in% names(NASS))) {
  
  # Create better age groups: 0-17, 18-64, 65+
  plot_data <- NASS[RACE %in% 1:6 & PAY1 %in% 1:6 & !is.na(AGE)]
  
  if(nrow(plot_data) > 0) {
    
    # Create simplified age groups
    plot_data[, AGE_GROUP_SIMPLE := cut(AGE, 
                                       breaks = c(-Inf, 17, 64, Inf),
                                       labels = c("0-17", "18-64", "65+"),
                                       right = TRUE)]
    
    # Remove any missing age groups
    plot_data <- plot_data[!is.na(AGE_GROUP_SIMPLE)]
    
    # Define cleaner labels
    race_labels <- c("1" = "White", "2" = "Black", "3" = "Hispanic", 
                    "4" = "Asian/Pacific", "5" = "Native American", "6" = "Other")
    pay_labels <- c("1" = "Medicare", "2" = "Medicaid", "3" = "Private", 
                   "4" = "Self-pay", "5" = "No Charge", "6" = "Other")
    
    p7 <- ggplot(plot_data, aes(x = factor(RACE), fill = factor(PAY1))) + 
      geom_bar(position = "fill", alpha = 0.8, color = "white", linewidth = 0.2) + 
      theme_poster + 
      labs(
        x = "Race/Ethnicity", 
        y = "Proportion",
        title = "Primary Payer Distribution by Demographics", 
        subtitle = "Insurance Coverage Patterns by Age Group and Race/Ethnicity",
        fill = "Primary Payer"
      ) + 
      scale_fill_manual(values = payer_colors_unified, labels = pay_labels) + 
      scale_x_discrete(labels = race_labels) +
      scale_y_continuous(labels = percent_format()) +
      facet_wrap(~ AGE_GROUP_SIMPLE, ncol = 3) +
      theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
            legend.position = "bottom",
            strip.text = element_text(size = 11, face = "bold"),
            panel.spacing = unit(1, "lines"))
    
    print(p7)
    save_plot_for_poster(p7, 
                    name = "payer_by_demographics", 
                    description = "Primary Payer Distribution by Demographics",
                    category = "descriptive")
  }
}

#### Payer Distribution by Income

In [ ]:
%%R # === PAYER DISTRIBUTION BY INCOME AND AGE GROUP ===

# Payer distribution by income quartile and age group
if(all(c("ZIPINC_QRTL", "PAY1", "AGE") %in% names(NASS))) {
  
  # Create data with income quartiles and simplified age groups
  plot_data <- NASS[ZIPINC_QRTL %in% 1:4 & PAY1 %in% 1:6 & !is.na(AGE)]
  
  if(nrow(plot_data) > 0) {
    
    # Create simplified age groups: 0-17, 18-64, 65+
    plot_data[, AGE_GROUP_SIMPLE := cut(AGE, 
                                       breaks = c(-Inf, 17, 64, Inf),
                                       labels = c("0-17", "18-64", "65+"),
                                       right = TRUE)]
    
    # Remove any missing age groups
    plot_data <- plot_data[!is.na(AGE_GROUP_SIMPLE)]
    
    # Define cleaner labels
    income_labels <- c("1" = "Q1 (Lowest)", "2" = "Q2", "3" = "Q3", "4" = "Q4 (Highest)")
    pay_labels <- c("1" = "Medicare", "2" = "Medicaid", "3" = "Private", 
                   "4" = "Self-pay", "5" = "No Charge", "6" = "Other")
    
    p8 <- ggplot(plot_data, aes(x = factor(ZIPINC_QRTL), fill = factor(PAY1))) + 
      geom_bar(position = "fill", alpha = 0.8, color = "white", linewidth = 0.2) + 
      theme_poster + 
      labs(
        x = "Income Quartile (by ZIP Code)", 
        y = "Proportion",
        title = "Primary Payer Distribution by Socioeconomics", 
        subtitle = "Insurance by Age Group and Income Quartile",
        fill = "Primary Payer"
      ) + 
      scale_fill_manual(values = payer_colors_unified, labels = pay_labels) + 
      scale_x_discrete(labels = income_labels) +
      scale_y_continuous(labels = percent_format()) +
      facet_wrap(~ AGE_GROUP_SIMPLE, ncol = 3) +
      theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
            legend.position = "bottom",
            strip.text = element_text(size = 11, face = "bold"),
            panel.spacing = unit(1, "lines"))
    
    print(p8)
    save_plot_for_poster(p8, 
                    name = "payer_by_income", 
                    description = "Primary Payer Distribution by Income",
                    category = "descriptive")
  }
}

### Comprehensive Dataset Summary

Final summary statistics providing an overview of all key variables in the analysis.

In [ ]:
%%R # === COMPREHENSIVE DATASET SUMMARY ===

# ===============================================
# COMPREHENSIVE NASS 2020 DATASET SUMMARY
# ===============================================

cat("================================================================\n")
cat("                    NASS 2020 DATASET SUMMARY                  \n")
cat("================================================================\n\n")

# Dataset Overview
cat("DATASET OVERVIEW\n")
cat("================\n")
cat("Total observations:", format(nrow(NASS), big.mark = ","), "\n")
cat("Total variables:", ncol(NASS), "\n")
cat("Memory usage:", round(object.size(NASS)/1024^2, 1), "MB\n")
cat("Missing data patterns:", sum(is.na(NASS)), "total missing values\n\n")

# Key variables for comprehensive summary
summary_vars <- c("AGE", "FEMALE", "RACE", "WHITE", "ZIPINC_QRTL", "PAY1", 
                 "HOSP_LOCATION", "HOSP_TEACH", "HOSP_REGION", "HOSP_BEDSIZE_CAT",
                 "PL_NCHS", "CPTCCS1", "TOTCHG", "DISCWT")

available_vars <- summary_vars[summary_vars %in% names(NASS)]
missing_vars <- summary_vars[!summary_vars %in% names(NASS)]

if(length(missing_vars) > 0) {
  cat("WARNING: Variables not available:", paste(missing_vars, collapse = ", "), "\n\n")
}

# Variable-by-variable analysis
for(var in available_vars) {
  cat("----------------------------------------------------------------\n")
  cat("Variable:", toupper(var), "\n")
  cat("----------------------------------------------------------------\n")
  
  if(is.numeric(NASS[[var]])) {
    # Enhanced numeric variable summary
    valid_values <- NASS[[var]][!is.na(NASS[[var]])]
    missing_count <- sum(is.na(NASS[[var]]))
    
    cat("Type: Continuous/Numeric\n")
    cat("Valid observations:", format(length(valid_values), big.mark = ","), 
        "(", round(100 * length(valid_values) / nrow(NASS), 1), "%)\n")
    cat("Missing values:", format(missing_count, big.mark = ","), 
        "(", round(100 * missing_count / nrow(NASS), 1), "%)\n")
    
    if(length(valid_values) > 0) {
      # Central tendency
      cat("\nCENTRAL TENDENCY:\n")
      cat("   Mean:", format(round(mean(valid_values), 2), big.mark = ","), "\n")
      cat("   Median:", format(round(median(valid_values), 2), big.mark = ","), "\n")
      cat("   Mode region:", format(round(median(valid_values), 2), big.mark = ","), 
          " +/-", format(round(mad(valid_values), 2), big.mark = ","), "\n")
      
      # Variability
      cat("\nVARIABILITY:\n")
      cat("   Standard Dev:", format(round(sd(valid_values), 2), big.mark = ","), "\n")
      cat("   IQR:", format(round(quantile(valid_values, 0.25), 2), big.mark = ","), "to", 
          format(round(quantile(valid_values, 0.75), 2), big.mark = ","), "\n")
      cat("   Range:", format(round(min(valid_values), 2), big.mark = ","), "to", 
          format(round(max(valid_values), 2), big.mark = ","), "\n")
      
      # Distribution shape
      cat("\nDISTRIBUTION:\n")
      q1 <- quantile(valid_values, 0.25)
      q3 <- quantile(valid_values, 0.75)
      skewness_approx <- (mean(valid_values) - median(valid_values)) / sd(valid_values)
      
      cat("   Skewness (approx):", round(skewness_approx, 3), 
          ifelse(abs(skewness_approx) < 0.5, "(approximately symmetric)", 
                ifelse(skewness_approx > 0, "(right-skewed)", "(left-skewed)")), "\n")
      
      # Outliers (using IQR method)
      iqr <- q3 - q1
      lower_fence <- q1 - 1.5 * iqr
      upper_fence <- q3 + 1.5 * iqr
      outliers <- sum(valid_values < lower_fence | valid_values > upper_fence)
      cat("   Potential outliers:", outliers, 
          "(", round(100 * outliers / length(valid_values), 1), "%)\n")
      
      # Percentile breakdown for key variables
      if(var %in% c("AGE", "TOTCHG", "DISCWT")) {
        cat("\nPERCENTILE BREAKDOWN:\n")
        percentiles <- c(0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99)
        for(p in percentiles) {
          cat("   P", round(p*100), ":", format(round(quantile(valid_values, p), 1), big.mark = ","), "\n")
        }
      }
    }
    
  } else {
    # Enhanced categorical variable summary
    freq_table <- table(NASS[[var]], useNA = "ifany")
    total_n <- sum(freq_table)
    missing_count <- sum(is.na(NASS[[var]]))
    
    cat("Type: Categorical/Factor\n")
    cat("Total categories:", length(freq_table), "\n")
    cat("Valid observations:", format(total_n - missing_count, big.mark = ","), 
        "(", round(100 * (total_n - missing_count) / nrow(NASS), 1), "%)\n")
    
    if(missing_count > 0) {
      cat("Missing values:", format(missing_count, big.mark = ","), 
          "(", round(100 * missing_count / nrow(NASS), 1), "%)\n")
    }
    
    # Sort by frequency
    sorted_freq <- sort(freq_table, decreasing = TRUE)
    
    # Show distribution
    cat("\nFREQUENCY DISTRIBUTION:\n")
    max_show <- min(15, length(sorted_freq))  # Show more categories
    
    for(i in 1:max_show) {
      category_name <- names(sorted_freq)[i]
      count <- sorted_freq[i]
      percentage <- round(100 * count / total_n, 1)
      
      # Create a simple bar visualization
      bar_length <- min(20, round(20 * count / max(sorted_freq)))
      bar <- paste(rep("*", bar_length), collapse = "")
      
      cat("   ", sprintf("%-20s", category_name), ":", 
          sprintf("%8s", format(count, big.mark = ",")), 
          sprintf("(%5.1f%%)", percentage), " ", bar, "\n")
    }
    
    if(length(sorted_freq) > max_show) {
      remaining_count <- sum(sorted_freq[(max_show+1):length(sorted_freq)])
      remaining_pct <- round(100 * remaining_count / total_n, 1)
      cat("   ... ", length(sorted_freq) - max_show, " more categories:", 
          format(remaining_count, big.mark = ","), "(", remaining_pct, "%)\n")
    }
    
    # Diversity metrics
    cat("\nDIVERSITY METRICS:\n")
    # Simpson's diversity index (1 - sum of squared proportions)
    proportions <- as.numeric(freq_table) / sum(freq_table)
    simpson_diversity <- 1 - sum(proportions^2)
    cat("   Simpson's Diversity:", round(simpson_diversity, 3), 
        "(0=no diversity, 1=max diversity)\n")
    
    # Effective number of categories (inverse Simpson)
    effective_categories <- 1 / sum(proportions^2)
    cat("   Effective categories:", round(effective_categories, 1), 
        "out of", length(freq_table), "total\n")
    
    # Concentration ratio (top 3 categories)
    top3_concentration <- sum(sorted_freq[1:min(3, length(sorted_freq))]) / total_n
    cat("   Top-3 concentration:", round(100 * top3_concentration, 1), "%\n")
    
    # Special insights for key variables
    if(var == "RACE") {
      cat("\nRACE/ETHNICITY INSIGHTS:\n")
      cat("   Racial diversity reflects ambulatory surgery access patterns\n")
      if("1" %in% names(freq_table)) {
        white_vs_nonwhite <- round(freq_table["1"] / sum(freq_table[names(freq_table) != "1"]), 2)
        cat("   White vs Non-White ratio:", white_vs_nonwhite, ":1\n")
      }
    }
    
    if(var == "PAY1") {
      cat("\nPAYER MIX INSIGHTS:\n")
      public_payers <- sum(freq_table[c("1", "2")], na.rm = TRUE)  # Medicare + Medicaid
      cat("   Public insurance coverage:", 
          round(100 * public_payers / total_n, 1), "%\n")
    }
    
    if(var == "ZIPINC_QRTL") {
      cat("\nINCOME DISTRIBUTION INSIGHTS:\n")
      low_income <- sum(freq_table[c("1", "2")], na.rm = TRUE)  # Q1 + Q2
      cat("   Lower-income representation:", 
          round(100 * low_income / total_n, 1), "%\n")
    }
  }
  cat("\n")
}

# Cross-tabulation insights
cat("----------------------------------------------------------------\n")
cat("KEY RELATIONSHIPS & CROSS-TABULATIONS\n")
cat("----------------------------------------------------------------\n\n")

# Race vs Payer cross-tab
if(all(c("RACE", "PAY1") %in% available_vars)) {
  cat("RACE x PAYER DISTRIBUTION:\n")
  cross_tab <- table(NASS$RACE, NASS$PAY1)
  prop_tab <- round(100 * prop.table(cross_tab, 1), 1)
  
  race_labels <- c("1"="White", "2"="Black", "3"="Hispanic", "4"="Asian/Pacific", "5"="Native Am", "6"="Other")
  pay_labels <- c("1"="Medicare", "2"="Medicaid", "3"="Private", "4"="Self-pay", "5"="No Charge", "6"="Other")
  
  for(race in rownames(prop_tab)) {
    if(race %in% names(race_labels)) {
      cat("   ", race_labels[race], "patients:\n")
      race_row <- prop_tab[race, ]
      sorted_payers <- sort(race_row, decreasing = TRUE)
      for(i in 1:min(3, length(sorted_payers))) {
        payer <- names(sorted_payers)[i]
        if(payer %in% names(pay_labels)) {
          cat("     ", pay_labels[payer], ":", sorted_payers[i], "%\n")
        }
      }
    }
  }
  cat("\n")
}

# Age vs Income relationship
if(all(c("AGE", "ZIPINC_QRTL") %in% available_vars)) {
  cat("AGE x INCOME PATTERNS:\n")
  age_income <- NASS[!is.na(AGE) & !is.na(ZIPINC_QRTL), .(mean_age = round(mean(AGE), 1)), by = ZIPINC_QRTL]
  setorder(age_income, ZIPINC_QRTL)
  
  income_labels <- c("1"="Q1 (Lowest)", "2"="Q2", "3"="Q3", "4"="Q4 (Highest)")
  for(i in 1:nrow(age_income)) {
    quartile <- as.character(age_income$ZIPINC_QRTL[i])
    if(quartile %in% names(income_labels)) {
      cat("   ", income_labels[quartile], "- Average age:", age_income$mean_age[i], "years\n")
    }
  }
  cat("\n")
}

# Hospital characteristics summary
if(all(c("HOSP_REGION", "HOSP_TEACH", "HOSP_LOCATION") %in% available_vars)) {
  cat("HOSPITAL CHARACTERISTICS:\n")
  
  # Unique hospital count
  if("HOSP_NASS" %in% names(NASS)) {
    unique_hospitals <- length(unique(NASS$HOSP_NASS))
    cat("   Total hospitals in sample:", unique_hospitals, "\n")
  }
  
  # Teaching hospital distribution
  teaching_dist <- table(NASS$HOSP_TEACH)
  if("1" %in% names(teaching_dist)) {
    cat("   Teaching hospitals:", round(100 * teaching_dist["1"] / sum(teaching_dist), 1), "%\n")
  }
  
  # Urban/Rural distribution
  location_dist <- table(NASS$HOSP_LOCATION)
  if("1" %in% names(location_dist)) {
    cat("   Urban hospitals:", round(100 * location_dist["1"] / sum(location_dist), 1), "%\n")
  }
  
  # Regional distribution
  region_dist <- table(NASS$HOSP_REGION)
  region_labels <- c("1"="Northeast", "2"="Midwest", "3"="South", "4"="West")
  cat("   Regional distribution:\n")
  for(region in names(region_dist)) {
    if(region %in% names(region_labels)) {
      pct <- round(100 * region_dist[region] / sum(region_dist), 1)
      cat("     ", region_labels[region], ":", pct, "%\n")
    }
  }
  cat("\n")
}

# Data quality assessment
cat("----------------------------------------------------------------\n")
cat("DATA QUALITY ASSESSMENT\n")
cat("----------------------------------------------------------------\n\n")

# Missing data patterns
total_cells <- nrow(NASS) * ncol(NASS)
missing_cells <- sum(is.na(NASS))
complete_cases <- sum(complete.cases(NASS))

cat("COMPLETENESS METRICS:\n")
cat("   Overall completeness:", round(100 * (1 - missing_cells/total_cells), 2), "%\n")
cat("   Complete cases (no missing):", format(complete_cases, big.mark = ","), 
    "(", round(100 * complete_cases / nrow(NASS), 1), "%)\n")
cat("   Variables with missing data:", sum(sapply(NASS, function(x) any(is.na(x)))), 
    "out of", ncol(NASS), "\n\n")

# Variables with highest missing rates
missing_rates <- sapply(NASS, function(x) round(100 * sum(is.na(x)) / length(x), 1))
high_missing <- missing_rates[missing_rates > 0]
if(length(high_missing) > 0) {
  cat("VARIABLES WITH MISSING DATA:\n")
  sorted_missing <- sort(high_missing, decreasing = TRUE)
  for(i in 1:min(10, length(sorted_missing))) {
    cat("   ", names(sorted_missing)[i], ":", sorted_missing[i], "% missing\n")
  }
  cat("\n")
}

# Survey weights summary
if("DISCWT" %in% available_vars) {
  cat("SURVEY WEIGHTS SUMMARY:\n")
  weights <- NASS$DISCWT[!is.na(NASS$DISCWT)]
  cat("   Weight range:", round(min(weights), 2), "to", round(max(weights), 2), "\n")
  cat("   Effective sample size:", round(sum(weights)^2 / sum(weights^2)), "\n")
  cat("   Design effect (approx):", round(nrow(NASS) / (sum(weights)^2 / sum(weights^2)), 2), "\n\n")
}

cat("================================================================\n")
cat("                     SUMMARY COMPLETE                          \n")
cat("================================================================\n")

---

## 2. Census Data Comparisons


### Census API Setup

Set up Census API integration and pull 2020 DHC population data for comparison with NASS sample proportions.

[ Please Register at the Census website for an API Key ](https://api.census.gov/data/key_signup.html). Enter your Census API key for data retrieval:

In [ ]:
# === CENSUS API KEY HANDLING ===

import getpass, os, json, textwrap

# Try encrypted Census API key first if available
census_api_key = None

if 'CENSUS_ENCRYPTED_KEY_URL' in globals() and 'CENSUS_PASSWORD' in globals():
    try:
        # Try to decrypt Census API key
        decrypted_key = data_loader._decrypt_key(CENSUS_ENCRYPTED_KEY_URL, CENSUS_PASSWORD)
        if decrypted_key:
            # Parse the decrypted key (assume it's JSON with api_key field)
            key_data = json.loads(decrypted_key)
            census_api_key = key_data.get('api_key')
            print("✅ Using encrypted Census API key")
        else:
            print("❌ Failed to decrypt Census API key - manual entry required")
    except Exception as e:
        print(f"❌ Census key decryption error - manual entry required")

# Fallback to manual entry if encryption failed
if not census_api_key:
    print("Manual Census API key entry required:")
    census_api_key = getpass.getpass("Enter your Census API key (will not echo):")

# FIXED: Set the API key in BOTH Python and R environments
if census_api_key and census_api_key.strip():
    os.environ["CENSUS_API_KEY"] = census_api_key.strip()
    
    # Also pass it directly to R to ensure it's available
    get_ipython().run_cell_magic('R', f'-i census_api_key', '''
    # Set the Census API key in R environment
    Sys.setenv(CENSUS_API_KEY = census_api_key)
    cat("✅ Census API key configured in R environment\\n")
    ''')
    
    print("✅ Census API key configured in both Python and R")
else:
    print("❌ No valid Census API key provided")
    print("💡 You'll need to manually enter the key in the Census setup section")

### Census Data Retrieval and Processing

Pull 2020 DHC population data by age, gender, and race for states included in NASS sample.

In [ ]:
%%R -i VERBOSE_PRINTS # === CENSUS DATA RETRIEVAL ===

# Install and load required packages for Census analysis
required_packages <- c("tidycensus", "dplyr", "tidyr", "survey")

for(pkg in required_packages) {
  if(!require(pkg, character.only = TRUE, quietly = TRUE)) {
    install.packages(pkg, quiet = TRUE)
    library(pkg, character.only = TRUE, quietly = TRUE)
  }
}

# Set Census API key
census_api_key(Sys.getenv("CENSUS_API_KEY"), overwrite = FALSE, install = FALSE)

# Define states included in NASS 2020 dataset
states_in_nass <- c("Alaska", "California", "Colorado", "Connecticut", "District of Columbia", 
                    "Florida", "Georgia", "Hawaii", "Iowa", "Illinois", "Indiana", "Kansas", 
                    "Kentucky", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", 
                    "North Carolina", "North Dakota", "Nebraska", "New Jersey", "Nevada", 
                    "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "South Carolina", 
                    "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Wisconsin")

cat("Defined", length(states_in_nass), "states included in NASS sample\n")

# Function to construct population variables for Census queries
get_population_variables <- function(base_variable) {
    variables <- paste0(base_variable, "_", sprintf("%03dN", 1:49))
    
    labels <- c(
        "Total",
        "Male: Total", "Male: Under 5 years", "Male: 5 to 9 years", "Male: 10 to 14 years",
        "Male: 15 to 17 years", "Male: 18 and 19 years", "Male: 20 years", "Male: 21 years",
        "Male: 22 to 24 years", "Male: 25 to 29 years", "Male: 30 to 34 years", "Male: 35 to 39 years",
        "Male: 40 to 44 years", "Male: 45 to 49 years", "Male: 50 to 54 years", "Male: 55 to 59 years",
        "Male: 60 and 61 years", "Male: 62 to 64 years", "Male: 65 and 66 years", "Male: 67 to 69 years",
        "Male: 70 to 74 years", "Male: 75 to 79 years", "Male: 80 to 84 years", "Male: 85 years and over",
        "Female: Total", "Female: Under 5 years", "Female: 5 to 9 years", "Female: 10 to 14 years",
        "Female: 15 to 17 years", "Female: 18 and 19 years", "Female: 20 years", "Female: 21 years",
        "Female: 22 to 24 years", "Female: 25 to 29 years", "Female: 30 to 34 years", "Female: 35 to 39 years",
        "Female: 40 to 44 years", "Female: 45 to 49 years", "Female: 50 to 54 years", "Female: 55 to 59 years",
        "Female: 60 and 61 years", "Female: 62 to 64 years", "Female: 65 and 66 years", "Female: 67 to 69 years",
        "Female: 70 to 74 years", "Female: 75 to 79 years", "Female: 80 to 84 years", "Female: 85 years and over"
    )
    
    names(labels) <- variables
    return(list(variables = variables, labels = labels))
}

# Function to get population data from Census
get_population_data <- function(variables, labels) {
    population_data <- get_decennial(
        geography = "state",
        variables = variables,
        year = 2020,
        sumfile = "dhc"
    )
    
    # Replace variable codes with descriptive labels
    population_data <- population_data %>% 
        mutate(variable = recode(variable, !!!setNames(labels, variables)))
    
    # Reshape data for analysis
    population_data <- population_data %>% 
        pivot_wider(names_from = variable, values_from = value)
    
    return(population_data)
}

# Get total population data (all races)
cat("Retrieving total population data from 2020 Census...\n")
population_info_total <- get_population_variables("P12")
total_population_by_age_gender <- get_population_data(population_info_total$variables, population_info_total$labels)

# Get white alone population data
cat("Retrieving white alone population data from 2020 Census...\n")
population_info_white <- get_population_variables("P12I")
total_population_by_age_gender_white <- get_population_data(population_info_white$variables, population_info_white$labels)

cat("Census data retrieval complete\n")

if(VERBOSE_PRINTS) {
  cat("\nTotal population data structure:\n")
  str(total_population_by_age_gender)
  cat("\nWhite population data structure:\n")
  str(total_population_by_age_gender_white)
}

### Statistical Comparison: NASS vs Census Proportions

Compare unadjusted and weighted proportions of white individuals in NASS sample against Census benchmarks.

In [ ]:
%%R # === PROPORTION ANALYSIS ===

# ========================================
# Stage 1a: Unadjusted Proportion Analysis
# ========================================

cat("=== STAGE 1A: UNADJUSTED PROPORTION ANALYSIS ===\n")

# Calculate unadjusted proportion of white individuals in NASS
unadjusted_proportion_white <- mean(NASS$WHITE, na.rm = TRUE)
cat("Unadjusted proportion of WHITE in NASS:", round(unadjusted_proportion_white, 4), "\n")

# Calculate reference proportion from entire US Census
us_census_white_proportion <- sum(total_population_by_age_gender_white$Total, na.rm = TRUE) / 
                             sum(total_population_by_age_gender$Total, na.rm = TRUE)
cat("US Census White alone proportion:", round(us_census_white_proportion, 4), "\n")

# Statistical test for unadjusted proportion
unadjusted_test <- prop.test(sum(NASS$WHITE, na.rm = TRUE), 
                            sum(!is.na(NASS$WHITE)), 
                            p = us_census_white_proportion)
cat("\nUnadjusted proportion test results:\n")
print(unadjusted_test)

# ========================================
# Stage 1b: Weighted Proportion Analysis
# ========================================

cat("\n=== STAGE 1B: WEIGHTED PROPORTION ANALYSIS ===\n")

# Filter Census data for NASS-included states only
filtered_total_population <- total_population_by_age_gender %>% 
    filter(NAME %in% states_in_nass)

filtered_white_population <- total_population_by_age_gender_white %>% 
    filter(NAME %in% states_in_nass)

# Calculate true proportion for NASS states
total_population_nass_states <- sum(filtered_total_population$Total, na.rm = TRUE)
total_white_population_nass_states <- sum(filtered_white_population$Total, na.rm = TRUE)
true_proportion_white_nass_states <- total_white_population_nass_states / total_population_nass_states

cat("True proportion of WHITE in NASS states:", round(true_proportion_white_nass_states, 4), "\n")

# Calculate weighted proportion using survey design
survey_design <- svydesign(ids = ~1, weights = ~DISCWT, data = NASS)
weighted_proportion_white <- svymean(~WHITE, design = survey_design)
cat("Weighted proportion of WHITE in NASS:", round(coef(weighted_proportion_white), 4), "\n")

# Statistical test for weighted proportion
weighted_test <- svyttest(WHITE ~ 1, design = survey_design, mu = true_proportion_white_nass_states)
cat("\nWeighted proportion test results:\n")
print(weighted_test)

cat("\n=== PROPORTION ANALYSIS COMPLETE ===\n")

### Age-Gender Stratified Analysis

Detailed comparison of white proportions by age group and gender between NASS sample and Census data.

In [ ]:
%%R # === AGE-GENDER STRATIFIED ANALYSIS ===

# ========================================
# Stage 2: Age-Gender Stratified Analysis
# ========================================

cat("=== STAGE 2: AGE-GENDER STRATIFIED ANALYSIS ===\n")

# Define age groups matching Census categories
age_breaks <- c(-Inf, 4, 9, 14, 17, 19, 20, 21, 24, 29, 34, 39, 44, 49, 54, 59, 61, 64, 66, 69, 74, 79, 84, Inf)
age_labels <- c("Under 5 years", "5 to 9 years", "10 to 14 years", "15 to 17 years", "18 and 19 years",
                "20 years", "21 years", "22 to 24 years", "25 to 29 years", "30 to 34 years",
                "35 to 39 years", "40 to 44 years", "45 to 49 years", "50 to 54 years", "55 to 59 years",
                "60 and 61 years", "62 to 64 years", "65 and 66 years", "67 to 69 years", "70 to 74 years",
                "75 to 79 years", "80 to 84 years", "85 years and over")

# Create age group variable in NASS dataset
NASS[, AGE_GROUP := cut(AGE, breaks = age_breaks, labels = age_labels, right = TRUE)]
NASS[, GENDER := ifelse(FEMALE == 0, "Male", "Female")]

cat("Created age groups and gender variables\n")

# Calculate NASS proportions by age group and gender
nass_proportions <- NASS[!is.na(AGE_GROUP) & !is.na(WHITE), 
                        .(total = .N,
                          white = sum(WHITE, na.rm = TRUE),
                          proportion_white = mean(WHITE, na.rm = TRUE)), 
                        by = .(AGE_GROUP, GENDER)]

# Add confidence intervals
nass_proportions[, ':='(
  ci_lower = proportion_white - 1.96 * sqrt((proportion_white * (1 - proportion_white)) / total),
  ci_upper = proportion_white + 1.96 * sqrt((proportion_white * (1 - proportion_white)) / total)
)]

cat("Calculated NASS proportions by age-gender groups\n")

# Process Census data for comparison
census_proportions <- total_population_by_age_gender_white %>% 
    select(NAME, starts_with("Male"), starts_with("Female")) %>% 
    pivot_longer(cols = -NAME, names_to = "age_gender", values_to = "white_population") %>% 
    separate(age_gender, into = c("gender", "age_group"), sep = ": ") %>% 
    left_join(
        total_population_by_age_gender %>% 
            select(NAME, starts_with("Male"), starts_with("Female")) %>% 
            pivot_longer(cols = -NAME, names_to = "age_gender", values_to = "total_population") %>% 
            separate(age_gender, into = c("gender", "age_group"), sep = ": "),
        by = c("NAME", "gender", "age_group")
    ) %>% 
    filter(NAME %in% states_in_nass) %>%  # Filter for NASS states only
    group_by(gender, age_group) %>% 
    summarize(
        total_population = sum(total_population, na.rm = TRUE),
        white_population = sum(white_population, na.rm = TRUE),
        proportion_white = white_population / total_population,
        .groups = 'drop'
    ) %>% 
    filter(!is.na(age_group) & age_group != "Total")

cat("Processed Census proportions by age-gender groups\n")

# Convert to data.table for easier manipulation
setDT(census_proportions)
setDT(nass_proportions)

# Convert age groups to factors for proper plotting
census_proportions[, age_group := factor(age_group, levels = age_labels)]
nass_proportions[, AGE_GROUP := factor(AGE_GROUP, levels = age_labels)]

# Remove any missing age groups
census_proportions <- census_proportions[!is.na(age_group)]
nass_proportions <- nass_proportions[!is.na(AGE_GROUP)]

cat("Data preparation complete\n")
cat("NASS age-gender groups:", nrow(nass_proportions), "\n")
cat("Census age-gender groups:", nrow(census_proportions), "\n")

if(VERBOSE_PRINTS) {
  cat("\nSample NASS proportions:\n")
  print(head(nass_proportions))
  cat("\nSample Census proportions:\n")
  print(head(census_proportions))
}

### Census Age-Gender Visualization

In [ ]:
%%R # === NASS CENSUS LINE PLOT ===

# ========================================
# Enhanced Visualization with 5-Year Age Groups - PART 1: Line Plot
# ========================================

cat("\n=== GENERATING ENHANCED VISUALIZATIONS - PART 1 ===\n")

# Check if ggplot2 is available for plotting
if(require("ggplot2", quietly = TRUE)) {
  
  # ========================================
  # Create GENDER variable from FEMALE
  # ========================================
  
  # First, ensure we have the GENDER variable
  if(!"GENDER" %in% names(NASS)) {
    if("FEMALE" %in% names(NASS)) {
      NASS[, GENDER := ifelse(FEMALE == 1, "Female", "Male")]
      cat("Created GENDER variable from FEMALE\n")
    } else {
      cat("Error: Neither GENDER nor FEMALE variable found\n")
      cat("Available variables related to gender:\n")
      gender_vars <- names(NASS)[grepl("(?i)(gender|female|male|sex)", names(NASS))]
      if(length(gender_vars) > 0) {
        cat("  ", paste(gender_vars, collapse = ", "), "\n")
      } else {
        cat("  No gender-related variables found\n")
      }
      stop("Cannot proceed without gender variable")
    }
  }
  
  # ========================================
  # Recreate age groups with 5-year brackets
  # ========================================
  
  cat("Creating 5-year age brackets...\n")
  
  # Define 5-year age breaks
  age_breaks_5yr <- c(-Inf, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84, Inf)
  age_labels_5yr <- c("0-4", "5-9", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", 
                      "40-44", "45-49", "50-54", "55-59", "60-64", "65-69", "70-74", 
                      "75-79", "80-84", "85+")
  
  # Recreate NASS age groups with 5-year brackets
  NASS[, AGE_GROUP_5YR := cut(AGE, breaks = age_breaks_5yr, labels = age_labels_5yr, right = TRUE)]
  
  # Calculate NASS proportions by 5-year age group and gender
  nass_proportions_5yr <- NASS[!is.na(AGE_GROUP_5YR) & !is.na(WHITE), 
                              .(total = .N,
                                white = sum(WHITE, na.rm = TRUE),
                                proportion_white = mean(WHITE, na.rm = TRUE)), 
                              by = .(AGE_GROUP_5YR, GENDER)]
  
  # Add confidence intervals
  nass_proportions_5yr[, ':='(
    ci_lower = proportion_white - 1.96 * sqrt((proportion_white * (1 - proportion_white)) / total),
    ci_upper = proportion_white + 1.96 * sqrt((proportion_white * (1 - proportion_white)) / total)
  )]
  
  # ========================================
  # Process Census data for 5-year groups
  # ========================================
  
  cat("Processing Census data for 5-year age groups...\n")
  
  # Check if census data exists
  if(!exists("total_population_by_age_gender_white") || !exists("total_population_by_age_gender")) {
    cat("Warning: Census data not available. Creating placeholder plot...\n")
    
    # Create placeholder plot with just NASS data
    placeholder_plot <- ggplot(nass_proportions_5yr, aes(x = AGE_GROUP_5YR, y = proportion_white, 
                                                         color = GENDER, group = GENDER)) +
      geom_line(linewidth = 1) +
      geom_point() +
      geom_ribbon(aes(ymin = ci_lower, ymax = ci_upper, fill = GENDER), 
                  alpha = 0.2, color = NA) +
      facet_wrap(~GENDER, ncol = 1) +
      
      scale_color_manual(values = c("Female" = "#d62728", "Male" = "#1f77b4")) +
      scale_fill_manual(values = c("Female" = "#d62728", "Male" = "#1f77b4"), guide = "none") +
      
      labs(title = "White Proportion by 5-Year Age Groups: NASS Data Only",
           subtitle = "Census comparison not available - showing NASS data with confidence intervals",
           x = "Age Group",
           y = "Proportion White",
           color = "Gender") +
           
      theme_poster +
      theme(axis.text.x = element_text(angle = 45, hjust = 1),
            legend.position = "bottom") +
      scale_y_continuous(limits = c(0, 1), labels = scales::percent)
    
    print(placeholder_plot)
    
    save_plot_for_poster(placeholder_plot, 
                        name = "census_comparison_lines", 
                        description = "White Proportion by Age Groups: NASS Only (Census data unavailable)",
                        category = "validation")
    
    cat("Placeholder plot generated successfully\n")
    
  } else {
    
    # Map detailed Census age groups to 5-year brackets
    census_age_mapping <- data.table(
      census_age = c("Under 5 years", "5 to 9 years", "10 to 14 years", "15 to 17 years", 
                     "18 and 19 years", "20 years", "21 years", "22 to 24 years", 
                     "25 to 29 years", "30 to 34 years", "35 to 39 years", "40 to 44 years",
                     "45 to 49 years", "50 to 54 years", "55 to 59 years", "60 and 61 years",
                     "62 to 64 years", "65 and 66 years", "67 to 69 years", "70 to 74 years",
                     "75 to 79 years", "80 to 84 years", "85 years and over"),
      age_5yr = c("0-4", "5-9", "10-14", "15-19", "15-19", "20-24", "20-24", "20-24",
                  "25-29", "30-34", "35-39", "40-44", "45-49", "50-54", "55-59", "60-64",
                  "60-64", "65-69", "65-69", "70-74", "75-79", "80-84", "85+")
    )
    
    # Process Census data with 5-year grouping
    if(require("tidyr", quietly = TRUE) && require("dplyr", quietly = TRUE)) {
      
      census_proportions_5yr <- total_population_by_age_gender_white %>% 
        select(NAME, starts_with("Male"), starts_with("Female")) %>% 
        pivot_longer(cols = -NAME, names_to = "age_gender", values_to = "white_population") %>% 
        separate(age_gender, into = c("gender", "age_group"), sep = ": ") %>% 
        left_join(
          total_population_by_age_gender %>% 
            select(NAME, starts_with("Male"), starts_with("Female")) %>% 
            pivot_longer(cols = -NAME, names_to = "age_gender", values_to = "total_population") %>% 
            separate(age_gender, into = c("gender", "age_group"), sep = ": "),
          by = c("NAME", "gender", "age_group")
        ) %>% 
        filter(NAME %in% states_in_nass & !is.na(age_group) & age_group != "Total") %>%
        left_join(census_age_mapping, by = c("age_group" = "census_age")) %>%
        filter(!is.na(age_5yr)) %>%
        group_by(gender, age_5yr) %>% 
        summarize(
          total_population = sum(total_population, na.rm = TRUE),
          white_population = sum(white_population, na.rm = TRUE),
          proportion_white = white_population / total_population,
          .groups = 'drop'
        )
      
      # Convert to data.table
      setDT(census_proportions_5yr)
      
      cat("Data preparation complete for 5-year age groups\n")
      
      # ========================================
      # Line comparison plot with 5-year groups
      # ========================================
      
      # Prepare data for plotting
      plot_data_nass_5yr <- nass_proportions_5yr[, .(
        AGE_GROUP = AGE_GROUP_5YR, GENDER, proportion_white, ci_lower, ci_upper, source = "NASS"
      )]
      
      plot_data_census_5yr <- census_proportions_5yr[, .(
        AGE_GROUP = factor(age_5yr, levels = age_labels_5yr), 
        GENDER = gender, 
        proportion_white, 
        source = "Census"
      )]
      plot_data_census_5yr[, ':='(ci_lower = proportion_white, ci_upper = proportion_white)]
      
      plot_data_5yr <- rbind(plot_data_nass_5yr, plot_data_census_5yr, fill = TRUE)
      
      # Create the comparison plot with UNIFIED THEME AND COLORS
      age_gender_plot_5yr <- ggplot(plot_data_5yr, aes(x = AGE_GROUP, y = proportion_white, 
                                                       color = source, group = interaction(source, GENDER))) +
        geom_line(linewidth = 1) +
        geom_point() +
        geom_ribbon(data = plot_data_5yr[source == "NASS"], 
                    aes(ymin = ci_lower, ymax = ci_upper, fill = source), 
                    alpha = 0.2, color = NA) +
        facet_wrap(~GENDER, ncol = 1) +
        
        # UPDATED: Apply unified colors from census_colors
        scale_color_manual(
          values = c("NASS" = "#1f77b4", "Census" = "#d62728"),  # Blue for NASS, Red for Census
          name = "Data Source"
        ) +
        scale_fill_manual(
          values = c("NASS" = "#1f77b4"),  # Blue fill for NASS confidence intervals
          guide = "none"  # Hide fill legend since it's redundant with color
        ) +
        
        labs(title = "Race Proportion by Age Groups vs Census",
             subtitle = "Census comparions across age groups and gender (w/ CI)",
             x = "Age Group",
             y = "Proportion White",
             color = "Data Source",
             fill = "Confidence Interval") +
             
        # UPDATED: Apply unified poster theme
        theme_poster +
        theme(axis.text.x = element_text(angle = 45, hjust = 1),
              legend.position = "bottom") +
        scale_y_continuous(limits = c(0, 1), labels = scales::percent)
      
      print(age_gender_plot_5yr)

      save_plot_for_poster(age_gender_plot_5yr, 
                        name = "census_comparison_lines", 
                        description = "White Proportion by Age Groups: NASS vs Census",
                        category = "validation")
      
      cat("Line plot visualization generated successfully with unified theme\n")
      
    } else {
      cat("Warning: tidyr/dplyr not available for census data processing\n")
    }
  }
  
} else {
    cat("ggplot2 not available \n")
}

cat("\n=== PART 1 COMPLETE ===\n")

Comparison of Demographic Age curve to census

In [ ]:
%%R # === ENHANCED FILLED AREA HOURGLASS PLOT ===

# ========================================
# PART 2: Enhanced Filled Area Hourglass Plot - UNIFIED THEME
# ========================================

cat("\n=== GENERATING ENHANCED VISUALIZATIONS - PART 2 ===\n")

# Check if ggplot2 is available for plotting
if(require("ggplot2", quietly = TRUE)) {
  
  cat("Generating enhanced filled area hourglass plot with unified styling...\n")
  
  # Calculate total populations for percentage calculations
  nass_total_pop <- sum(nass_proportions_5yr$total)
  census_total_pop <- sum(census_proportions_5yr$total_population)
  
  # Prepare NASS hourglass data (percentage of total population)
  hourglass_data_nass <- nass_proportions_5yr[, .(
    AGE_GROUP = AGE_GROUP_5YR, 
    GENDER, 
    count = total,
    percentage = (total / nass_total_pop) * 100,
    source = "NASS"
  )]
  
  # Make male percentages negative for left side
  hourglass_data_nass[GENDER == "Male", population_pct := -percentage]
  hourglass_data_nass[GENDER == "Female", population_pct := percentage]
  
  # Prepare Census hourglass data (percentage of total population)
  hourglass_data_census <- census_proportions_5yr[, .(
    AGE_GROUP = factor(age_5yr, levels = age_labels_5yr),
    GENDER = gender,
    count = total_population,
    percentage = (total_population / census_total_pop) * 100,
    source = "Census"
  )]
  
  # Make male percentages negative for left side
  hourglass_data_census[GENDER == "Male", population_pct := -percentage]
  hourglass_data_census[GENDER == "Female", population_pct := percentage]
  
  # Combine both datasets
  hourglass_combined <- rbind(hourglass_data_nass, hourglass_data_census)
  
  # Create age group numeric variable for smooth areas
  hourglass_combined[, age_numeric := as.numeric(AGE_GROUP)]
  
  # Create enhanced filled area hourglass plot with UNIFIED THEME AND COLORS
  hourglass_plot_enhanced <- ggplot(hourglass_combined, 
                                   aes(x = age_numeric, y = population_pct, 
                                       fill = interaction(GENDER, source))) +
    geom_area(alpha = 0.7, position = "identity") +
    coord_flip() +
    scale_x_continuous(
      breaks = 1:length(age_labels_5yr),
      labels = age_labels_5yr
    ) +
    scale_y_continuous(
      labels = function(x) paste0(abs(x), "%"),
      breaks = seq(-8, 8, 2)
    ) +
    
    # UPDATED: Use unified census_colors from our color system
    scale_fill_manual(
      values = c(
        "Female.Census" = "#ff9999",  # Light red - matches census_colors
        "Female.NASS" = "#87ceeb",    # Light blue - matches census_colors
        "Male.Census" = "#d62728",    # Deep red - matches census_colors  
        "Male.NASS" = "#1f77b4"       # Deep blue - matches census_colors
      ),
      labels = c("Female Census", "Male Census", "Female NASS", "Male NASS"),
      name = "Gender & Source"
    ) +
    
    # UPDATED: Apply unified poster theme
    labs(title = "Demographics Proportions: NASS vs Census",
         subtitle = "Percentage of total population by 5-year age groups",
         x = "Age Group",
         y = "Percentage of Total Population",
         fill = "Gender & Source") +
    
    # UPDATED: Use theme_poster as base with specific overrides
    theme_poster +
    theme(
      # Keep poster theme base but override specific elements for this plot
      axis.text.y = element_text(size = 9),
      panel.grid.major.y = element_blank(),
      legend.text = element_text(size = 10),      # Slightly larger for readability
      legend.title = element_text(size = 11),     # Poster-sized legend title
      legend.position = "bottom"
    ) +
    
    geom_hline(yintercept = 0, color = "black", linewidth = 0.5) +
    guides(
      fill = guide_legend(nrow = 2, override.aes = list(alpha = 0.8))
    )
  
  print(hourglass_plot_enhanced)

  save_plot_for_poster(hourglass_plot_enhanced, 
                    name = "census_comparison_hourglass_enhanced", 
                    description = "Enhanced Filled Area Hourglass: NASS vs Census Population Distribution",
                    category = "validation")
  
  cat("Enhanced filled area hourglass visualization generated with unified styling:\n")
  cat("- Applied theme_poster base theme with custom overrides\n")
  cat("- Used unified census_colors (red-white-blue scheme)\n") 
  cat("- Maintained left-aligned titles (poster style)\n")
  cat("- Age groups: 5-year brackets (0-4, 5-9, etc.)\n")
  cat("- Style: Solid filled areas from 0% to value\n")
  cat("- Colors: NASS blues, Census reds (consistent with line plot)\n")
  cat("- Shows: Percentage of total population\n")
  cat("- Comparison: NASS sample vs Census data\n")
  
} else {
    cat("ggplot2 not available \n")
}

cat("\n=== PART 2 COMPLETE ===\n")

---

## 3. Advanced Techniques

### Multi-level Modeling

In [ ]:
%%R # === MULTI-LEVEL MODELING ANALYSIS ===

# ========================================
# MULTI-LEVEL MODELING FOR NASS DATASET
# ========================================

cat("=== MULTI-LEVEL MODELING ANALYSIS ===\n")
cat("Examining hierarchical structure: Patients nested within Hospitals\n\n")

# Install required packages for multi-level modeling
required_packages <- c("lme4", "broom.mixed", "performance", "sjPlot")

for(pkg in required_packages) {
  if(!require(pkg, character.only = TRUE, quietly = TRUE)) {
    cat("Installing", pkg, "...\n")
    install.packages(pkg, quiet = TRUE)
    library(pkg, character.only = TRUE, quietly = TRUE)
  }
}

cat("Multi-level modeling packages loaded successfully\n\n")

# ========================================
# SMART SAMPLING FOR SPEED
# ========================================

cat("SAMPLING DATA FOR MULTI-LEVEL ANALYSIS\n")
cat(paste(rep("=", 50), collapse = ""), "\n")

# Use a manageable subset for demonstration (targeting <1 minute runtime)
set.seed(12345)
sample_size <- min(25000, nrow(NASS))  # Max 25K observations for speed
sample_indices <- sample(nrow(NASS), sample_size)

# Create modeling dataset with complete cases from sample
model_data_full <- NASS[sample_indices][!is.na(AGE) & !is.na(WHITE) & !is.na(ZIPINC_QRTL) & 
                                        !is.na(PAY1) & !is.na(HOSP_NASS) & 
                                        ZIPINC_QRTL %in% 1:4 & PAY1 %in% 1:6]

cat("Original NASS data:", nrow(NASS), "observations\n")
cat("Sample for modeling:", sample_size, "observations\n")
cat("Complete cases for modeling:", nrow(model_data_full), "observations\n")
cat("Final retention rate:", round(100 * nrow(model_data_full) / sample_size, 1), "%\n\n")

# Further sampling if still too large
if(nrow(model_data_full) > 15000) {
  model_indices <- sample(nrow(model_data_full), 15000)
  model_data <- model_data_full[model_indices]
  cat("Additional sampling to 15,000 observations for speed\n")
} else {
  model_data <- model_data_full
}

cat("Final modeling dataset:", nrow(model_data), "observations\n\n")

# ========================================
# 1. DATA PREPARATION FOR MODELING
# ========================================

cat("1. PREPARING DATA FOR MULTI-LEVEL ANALYSIS\n")
cat(paste(rep("=", 50), collapse = ""), "\n")

# Create modeling variables
model_data[, ':='(
  # Outcome: Binary indicator for private insurance (reference = public/other)
  private_insurance = as.numeric(PAY1 == 3),
  
  # Patient-level predictors
  age_centered = scale(AGE)[,1],  # Centered age
  high_income = as.numeric(ZIPINC_QRTL %in% 3:4),  # Upper income quartiles
  white_race = as.numeric(WHITE),
  
  # Hospital-level predictors
  teaching_hospital = as.numeric(HOSP_TEACH == 1),
  urban_hospital = as.numeric(HOSP_LOCATION == 1),
  large_hospital = as.numeric(HOSP_BEDSIZE_CAT == 3)
)]

# Hospital-level summary statistics
hospital_summary <- model_data[, .(
  n_patients = .N,
  pct_private = round(100 * mean(private_insurance), 1),
  pct_white = round(100 * mean(white_race), 1),
  pct_high_income = round(100 * mean(high_income), 1),
  mean_age = round(mean(AGE), 1)
), by = HOSP_NASS]

cat("HOSPITAL-LEVEL VARIATION:\n")
cat("Number of hospitals:", nrow(hospital_summary), "\n")
cat("Patients per hospital - Mean:", round(mean(hospital_summary$n_patients), 1), 
    "| Range:", min(hospital_summary$n_patients), "to", max(hospital_summary$n_patients), "\n")
cat("Private insurance % - Mean:", round(mean(hospital_summary$pct_private), 1), 
    "| Range:", min(hospital_summary$pct_private), "% to", max(hospital_summary$pct_private), "%\n\n")

# ========================================
# 2. BASELINE SINGLE-LEVEL MODEL
# ========================================

cat("2. BASELINE SINGLE-LEVEL LOGISTIC REGRESSION\n")
cat(paste(rep("=", 50), collapse = ""), "\n")

# Fit standard logistic regression (ignoring hospital clustering)
baseline_model <- glm(private_insurance ~ white_race + high_income + age_centered + 
                     teaching_hospital + urban_hospital + large_hospital,
                     data = model_data, family = binomial)

cat("BASELINE MODEL RESULTS:\n")
baseline_summary <- summary(baseline_model)
print(baseline_summary$coefficients)

# Calculate odds ratios
baseline_or <- exp(coef(baseline_model))
baseline_ci <- exp(confint(baseline_model))

cat("\nODDS RATIOS (95% CI):\n")
for(i in 2:length(baseline_or)) {  # Skip intercept
  var_name <- names(baseline_or)[i]
  or_value <- baseline_or[i]
  ci_lower <- baseline_ci[i, 1]
  ci_upper <- baseline_ci[i, 2]
  cat(sprintf("%-20s: %.2f (%.2f - %.2f)\n", var_name, or_value, ci_lower, ci_upper))
}

cat("\nModel AIC:", round(AIC(baseline_model), 1), "\n\n")

# ========================================
# 3. MULTI-LEVEL MODEL WITH RANDOM INTERCEPTS
# ========================================

cat("3. MULTI-LEVEL MODEL: RANDOM INTERCEPTS\n")
cat(paste(rep("=", 50), collapse = ""), "\n")

# Fit random intercept model
random_intercept_model <- glmer(private_insurance ~ white_race + high_income + age_centered + 
                               teaching_hospital + urban_hospital + large_hospital +
                               (1 | HOSP_NASS),
                               data = model_data, 
                               family = binomial,
                               control = glmerControl(optimizer = "bobyqa"))

cat("RANDOM INTERCEPT MODEL RESULTS:\n")
print(summary(random_intercept_model))

# Extract variance components
variance_components <- VarCorr(random_intercept_model)
hospital_variance <- as.numeric(variance_components$HOSP_NASS[1])
residual_variance <- pi^2/3  # For logistic regression

# Calculate ICC
icc <- hospital_variance / (hospital_variance + residual_variance)
cat("\nVARIANCE COMPONENTS:\n")
cat("Hospital-level variance:", round(hospital_variance, 3), "\n")
cat("Individual-level variance:", round(residual_variance, 3), "\n")
cat("Intraclass Correlation (ICC):", round(icc, 3), "\n")
cat("Interpretation: ", round(100 * icc, 1), "% of variation is between hospitals\n\n")

# Model comparison
cat("MODEL COMPARISON:\n")
cat("Baseline AIC:", round(AIC(baseline_model), 1), "\n")
cat("Multi-level AIC:", round(AIC(random_intercept_model), 1), "\n")
cat("AIC improvement:", round(AIC(baseline_model) - AIC(random_intercept_model), 1), "\n\n")

# ========================================
# 4. MULTI-LEVEL MODEL WITH RANDOM SLOPES
# ========================================

cat("4. MULTI-LEVEL MODEL: RANDOM SLOPES\n")
cat(paste(rep("=", 50), collapse = ""), "\n")

# Fit random slope model (race effect varies by hospital)
tryCatch({
  random_slope_model <- glmer(private_insurance ~ white_race + high_income + age_centered + 
                             teaching_hospital + urban_hospital + large_hospital +
                             (1 + white_race | HOSP_NASS),
                             data = model_data, 
                             family = binomial,
                             control = glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 20000)))
  
  cat("RANDOM SLOPE MODEL RESULTS:\n")
  print(summary(random_slope_model))
  
  # Extract random effects variance
  random_effects_var <- VarCorr(random_slope_model)
  cat("\nRANDOM EFFECTS VARIANCE:\n")
  print(random_effects_var)
  
  # Model comparison
  cat("\nMODEL COMPARISON (AIC):\n")
  cat("Baseline model:", round(AIC(baseline_model), 1), "\n")
  cat("Random intercept:", round(AIC(random_intercept_model), 1), "\n")
  cat("Random slope:", round(AIC(random_slope_model), 1), "\n")
  
  # Likelihood ratio test
  anova_result <- anova(random_intercept_model, random_slope_model)
  cat("\nLIKELIHOOD RATIO TEST (Random Intercept vs Random Slope):\n")
  print(anova_result)
  
  best_model <- random_slope_model
  cat("\nRandom slope model fitted successfully\n\n")
  
}, error = function(e) {
  cat("Random slope model failed to converge, using random intercept model\n")
  cat("Error:", e$message, "\n\n")
  best_model <- random_intercept_model
})

# Use the best fitting model
if(!exists("best_model")) {
  best_model <- random_intercept_model
}

# ========================================
# 5. MODEL INTERPRETATION & FINDINGS
# ========================================

cat("5. FINAL MODEL INTERPRETATION\n")
cat(paste(rep("=", 50), collapse = ""), "\n")

# Extract fixed effects
fixed_effects <- fixef(best_model)
fixed_se <- sqrt(diag(vcov(best_model)))

# Calculate odds ratios and confidence intervals
or_estimates <- exp(fixed_effects)
or_ci_lower <- exp(fixed_effects - 1.96 * fixed_se)
or_ci_upper <- exp(fixed_effects + 1.96 * fixed_se)

cat("FIXED EFFECTS - ODDS RATIOS FOR PRIVATE INSURANCE:\n")
cat("(Reference: Public insurance, non-white, low income, average age, non-teaching, rural, small hospital)\n\n")

for(i in 2:length(or_estimates)) {  # Skip intercept
  var_name <- names(or_estimates)[i]
  or_value <- or_estimates[i]
  ci_lower <- or_ci_lower[i]
  ci_upper <- or_ci_upper[i]
  
  # Create interpretable labels
  interpretation <- switch(var_name,
    "white_race" = "White vs Non-white race",
    "high_income" = "High vs Low income (Q3-Q4 vs Q1-Q2)",
    "age_centered" = "Per 1 SD increase in age",
    "teaching_hospital" = "Teaching vs Non-teaching hospital",
    "urban_hospital" = "Urban vs Rural hospital",
    "large_hospital" = "Large vs Small hospital",
    var_name
  )
  
  significance <- ifelse(ci_lower > 1 | ci_upper < 1, "***", "   ")
  
  cat(sprintf("%-35s: %.2f (%.2f - %.2f) %s\n", 
              interpretation, or_value, ci_lower, ci_upper, significance))
}

cat("\n*** = 95% CI excludes 1.0 (statistically significant)\n\n")

# ========================================
# 6. KEY FINDINGS SUMMARY
# ========================================

cat("6. KEY FINDINGS SUMMARY\n")
cat(paste(rep("=", 50), collapse = ""), "\n")

# Hospital variation
if(exists("hospital_variance")) {
  cat("HOSPITAL-LEVEL VARIATION:\n")
  cat("- ", round(100 * icc, 1), "% of variation in private insurance access is between hospitals\n")
  cat("- Substantial clustering effects - multi-level modeling was necessary\n\n")
}

# Patient-level effects
white_or <- or_estimates["white_race"]
income_or <- or_estimates["high_income"]

cat("PATIENT-LEVEL DISPARITIES:\n")
if(!is.na(white_or)) {
  cat("- White patients have ", round(white_or, 2), "x odds of private insurance vs non-white\n")
}
if(!is.na(income_or)) {
  cat("- High-income patients have ", round(income_or, 2), "x odds of private insurance vs low-income\n")
}

# Hospital-level effects
teaching_or <- or_estimates["teaching_hospital"]
urban_or <- or_estimates["urban_hospital"]

cat("\nHOSPITAL-LEVEL EFFECTS:\n")
if(!is.na(teaching_or)) {
  cat("- Teaching hospitals associated with ", round(teaching_or, 2), "x odds of private insurance\n")
}
if(!is.na(urban_or)) {
  cat("- Urban hospitals associated with ", round(urban_or, 2), "x odds of private insurance\n")
}

cat("\nCONFIRMS EARLIER FINDINGS:\n")
cat("- Significant racial and socioeconomic disparities in insurance access\n")
cat("- Hospital characteristics independently predict patient insurance mix\n")
cat("- Multi-level structure reveals important clustering at hospital level\n\n")

# Model performance
cat("MODEL PERFORMANCE:\n")
if(require("performance", quietly = TRUE)) {
  # Calculate model performance metrics
  model_performance <- model_performance(best_model)
  cat("- AIC:", round(AIC(best_model), 1), "\n")
  cat("- Model successfully accounts for hospital-level clustering\n")
  cat("- Robust estimates of patient and hospital-level effects\n")
}

cat("\nMULTI-LEVEL MODELING ANALYSIS COMPLETE\n")
cat("Results confirm substantial disparities at both patient and hospital levels\n")
cat("Note: Analysis performed on sample of", nrow(model_data), "observations for computational efficiency\n")

Visualization of Model

In [ ]:
%%R # === ODDS RATIOS FOREST PLOT - UNIFIED THEME ===

# ========================================
# IMPROVED ODDS RATIOS FOREST PLOT - UNIFIED STYLING
# ========================================

cat("=== CREATING IMPROVED ODDS RATIOS FOREST PLOT WITH UNIFIED THEME ===\n")

# Install visualization packages if needed
viz_packages <- c("ggplot2", "scales")

for(pkg in viz_packages) {
  if(!require(pkg, character.only = TRUE, quietly = TRUE)) {
    cat("Installing", pkg, "...\n")
    install.packages(pkg, quiet = TRUE)
    library(pkg, character.only = TRUE, quietly = TRUE)
  }
}

# Check if best_model exists
if(exists("best_model")) {
  
  cat("Creating Improved Odds Ratios Forest Plot with unified styling\n")
  
  # Extract fixed effects
  fixed_effects <- fixef(best_model)
  fixed_se <- sqrt(diag(vcov(best_model)))
  
  # Calculate odds ratios and confidence intervals
  or_estimates <- exp(fixed_effects)
  or_ci_lower <- exp(fixed_effects - 1.96 * fixed_se)
  or_ci_upper <- exp(fixed_effects + 1.96 * fixed_se)
  
  # Create data frame for plotting (excluding intercept)
  or_data <- data.frame(
    Variable = names(or_estimates)[-1],  # Remove intercept
    OR = or_estimates[-1],
    CI_Lower = or_ci_lower[-1],
    CI_Upper = or_ci_upper[-1],
    stringsAsFactors = FALSE
  )
  
  # Create better multi-line labels with line breaks
  or_data$Label <- c(
    "White vs\nNon-white Race",
    "High vs Low Income\n(Q3-Q4 vs Q1-Q2)",
    "Age\n(per 1 SD increase)",
    "Teaching vs\nNon-teaching Hospital",
    "Urban vs\nRural Hospital", 
    "Large vs\nSmall Hospital"
  )
  
  # Determine significance
  or_data$Significant <- (or_data$CI_Lower > 1 | or_data$CI_Upper < 1)
  
  # Create improved forest plot with UNIFIED THEME AND COLORS
  p1 <- ggplot(or_data, aes(x = OR, y = reorder(Label, OR))) +
    # Reference line at OR = 1
    geom_vline(xintercept = 1, linetype = "dashed", color = "red", alpha = 0.7, linewidth = 0.8) +
    
    # Confidence intervals
    geom_errorbarh(aes(xmin = CI_Lower, xmax = CI_Upper, color = Significant), 
                   height = 0.3, linewidth = 1.2) +
    
    # Point estimates
    geom_point(aes(color = Significant), size = 4, shape = 18) +  # Diamond shape
    
    # UPDATED: Use unified color scheme
    scale_color_manual(
      values = c("FALSE" = "gray50", "TRUE" = primary_colors[1]),  # Use primary blue for significant
      name = "Statistically\nSignificant", 
      labels = c("No", "Yes")
    ) +
    
    # X-axis with better breaks
    scale_x_log10(breaks = c(0.5, 0.75, 1, 1.5, 2, 3, 4),
                  labels = c("0.5", "0.75", "1.0", "1.5", "2.0", "3.0", "4.0"),
                  limits = c(0.4, 4.5)) +
    
    # UPDATED: Apply unified poster theme
    labs(title = "Predictors of Private Insurance",
         subtitle = "Multi-level Model Odds Ratios with 95% CI",
         x = "Odds Ratio (log scale)",
         y = "",  # Remove y-axis title since labels are self-explanatory
         caption = "Reference line at OR = 1.0 (no effect)") +
    
    # Apply poster theme as base with specific overrides
    theme_poster +
    theme(
      # Override specific elements for this plot type
      axis.text.y = element_text(size = 11, lineheight = 0.9),  # Better line spacing
      
      # Keep poster-style caption
      plot.caption = element_text(size = 9, color = "gray50", hjust = 0.5,
                                 margin = ggplot2::margin(t = 10)),
      
      # Grid lines
      panel.grid.minor.x = element_blank(),
      panel.grid.major.y = element_blank(),
      panel.grid.minor.y = element_blank(),
      
      # Legend positioning and sizing
      legend.position = "bottom",
      legend.margin = ggplot2::margin(t = 7.5)
    ) +
    
    # Add odds ratio values as text annotations - positioned to the right
    geom_text(aes(x = 4.2, y = reorder(Label, OR), label = sprintf("%.2f", OR)), 
              hjust = 0, size = 3.5, fontface = "bold", color = "black")
  
  print(p1)
  
  # SAVE THE PLOT using the save function
  save_plot_for_poster(p1, 
                      name = "odds_ratios_forest_plot", 
                      width = 12, 
                      height = 8,
                      description = "Odds Ratios Forest Plot for Private Insurance Access",
                      category = "analysis")
  
  # Print summary table
  cat("\nODDS RATIOS SUMMARY TABLE:\n")
  cat("=========================\n")
  summary_table <- or_data[order(-or_data$OR), c("Label", "OR", "CI_Lower", "CI_Upper", "Significant")]
  summary_table$Label <- gsub("\n", " ", summary_table$Label)  # Remove line breaks for table
  summary_table$OR <- round(summary_table$OR, 2)
  summary_table$CI_Lower <- round(summary_table$CI_Lower, 2)
  summary_table$CI_Upper <- round(summary_table$CI_Upper, 2)
  summary_table$CI <- paste0("(", summary_table$CI_Lower, " - ", summary_table$CI_Upper, ")")
  
  print(summary_table[, c("Label", "OR", "CI", "Significant")])
  
  cat("\nImproved Odds Ratios Forest Plot complete with unified styling:\n")
  
} else {
  cat("Error: best_model not found. Please run the multi-level modeling section first.\n")
}

### Machine Learning

In [ ]:
%%R #=== MACHINE LEARNING MODELING PIPELINE ===

# ========================================
# MACHINE LEARNING MODELING PIPELINE - FIXED
# ========================================

cat("=== MACHINE LEARNING MODELING PIPELINE ===\n")
cat("Training and evaluating multiple ML models\n\n")

# Install and load required packages
ml_packages <- c("randomForest", "pROC", "caret", "e1071", "glmnet", "xgboost")

for(pkg in ml_packages) {
  if(!require(pkg, character.only = TRUE, quietly = TRUE)) {
    cat("Installing", pkg, "...\n")
    install.packages(pkg, quiet = TRUE)
    library(pkg, character.only = TRUE, quietly = TRUE)
  }
}

cat("Machine learning packages loaded successfully\n\n")

# ========================================
# SMART DATA SAMPLING FOR DEMO
# ========================================

# For demonstration purposes, use a manageable subset
set.seed(12345)
sample_size <- min(20000, nrow(model_data))  # Max 20K rows for quick demo
sample_indices <- sample(nrow(model_data), sample_size)

ml_data <- model_data[sample_indices, .(private_insurance, white_race, high_income, age_centered,
                                       teaching_hospital, urban_hospital, large_hospital)]

cat("Using sample of", nrow(ml_data), "observations for ML demo\n")
cat("(", round(100 * nrow(ml_data) / nrow(model_data), 1), "% of full dataset)\n\n")

# Enhanced data cleaning
ml_data <- ml_data[complete.cases(ml_data)]

# Ensure all variables are proper numeric types
numeric_vars <- names(ml_data)
for(var in numeric_vars) {
  ml_data[, (var) := as.numeric(get(var))]
}

cat("ML dataset prepared:", nrow(ml_data), "complete observations\n\n")

# Split into training and testing
train_idx <- createDataPartition(ml_data$private_insurance, p = 0.8, list = FALSE)
train_data <- ml_data[train_idx]
test_data <- ml_data[-train_idx]

cat("Training set:", nrow(train_data), "observations\n")
cat("Test set:", nrow(test_data), "observations\n\n")

# ========================================
# 1. BASELINE LOGISTIC REGRESSION
# ========================================

cat("1️⃣ LOGISTIC REGRESSION\n")
start_time <- Sys.time()

baseline_glm <- glm(private_insurance ~ ., data = train_data, family = binomial)
baseline_pred <- predict(baseline_glm, test_data, type = "response")
baseline_pred_class <- ifelse(baseline_pred > 0.5, 1, 0)

end_time <- Sys.time()
cat("   Completed in", round(as.numeric(end_time - start_time), 2), "seconds\n\n")

# ========================================
# 2. RANDOM FOREST - OPTIMIZED
# ========================================

cat("2️⃣ RANDOM FOREST\n")
start_time <- Sys.time()

rf_model <- randomForest(factor(private_insurance) ~ ., 
                        data = train_data, 
                        ntree = 100,  # Reduced trees for speed
                        importance = TRUE,
                        mtry = 3)     # Fixed mtry

rf_pred <- predict(rf_model, test_data, type = "prob")[,2]
rf_pred_class <- as.numeric(as.character(predict(rf_model, test_data)))

end_time <- Sys.time()
cat("   Completed in", round(as.numeric(end_time - start_time), 2), "seconds\n\n")

# ========================================
# 3. SUPPORT VECTOR MACHINE - FAST VERSION
# ========================================

cat("3️⃣ SUPPORT VECTOR MACHINE (Optimized)\n")
start_time <- Sys.time()

# Use a smaller subset for SVM demo (SVM is O(n^3))
svm_sample_size <- min(5000, nrow(train_data))
svm_indices <- sample(nrow(train_data), svm_sample_size)
train_svm <- train_data[svm_indices]

cat("   Using", nrow(train_svm), "samples for SVM training (for speed)\n")

# Scale data for SVM
train_scaled <- copy(train_svm)
test_scaled <- copy(test_data)

# Scale age_centered
if("age_centered" %in% names(train_scaled)) {
  train_mean <- mean(train_scaled$age_centered)
  train_sd <- sd(train_scaled$age_centered)
  if(train_sd > 0) {
    train_scaled[, age_centered := (age_centered - train_mean) / train_sd]
    test_scaled[, age_centered := (age_centered - train_mean) / train_sd]
  }
}

# Fast SVM with optimized parameters
svm_model <- svm(factor(private_insurance) ~ ., 
                data = train_scaled, 
                kernel = "radial",
                cost = 1,
                gamma = 0.1,
                probability = TRUE)

svm_pred <- predict(svm_model, test_scaled, probability = TRUE)
svm_pred_prob <- attr(svm_pred, "probabilities")[,2]
svm_pred_class <- as.numeric(as.character(svm_pred))

end_time <- Sys.time()
cat("   Completed in", round(as.numeric(end_time - start_time), 2), "seconds\n\n")

# ========================================
# 4. LASSO REGRESSION
# ========================================

cat("4️⃣ LASSO REGRESSION\n")
start_time <- Sys.time()

x_train <- model.matrix(private_insurance ~ . - 1, data = train_data)
y_train <- train_data$private_insurance
x_test <- model.matrix(private_insurance ~ . - 1, data = test_data)

# Quick CV with fewer folds
cv_lasso <- cv.glmnet(x_train, y_train, family = "binomial", alpha = 1, nfolds = 3)
lasso_model <- glmnet(x_train, y_train, family = "binomial", alpha = 1, lambda = cv_lasso$lambda.min)

lasso_pred <- predict(lasso_model, x_test, type = "response")[,1]
lasso_pred_class <- ifelse(lasso_pred > 0.5, 1, 0)

end_time <- Sys.time()
cat("   Completed in", round(as.numeric(end_time - start_time), 2), "seconds\n\n")

# ========================================
# 5. XGBOOST - FAST VERSION
# ========================================

cat("5️⃣ XGBOOST\n")
start_time <- Sys.time()

dtrain <- xgb.DMatrix(data = as.matrix(train_data[, -"private_insurance"]), 
                     label = train_data$private_insurance)
dtest <- xgb.DMatrix(data = as.matrix(test_data[, -"private_insurance"]), 
                    label = test_data$private_insurance)

# Fast XGBoost parameters
xgb_params <- list(
  objective = "binary:logistic",
  eval_metric = "auc",
  eta = 0.3,          # Higher learning rate
  max_depth = 4,      # Shallower trees
  subsample = 0.8,
  colsample_bytree = 0.8
)

# Fewer rounds for speed
xgb_model <- xgb.train(
  params = xgb_params,
  data = dtrain,
  nrounds = 50,       # Reduced rounds
  verbose = 0
)

xgb_pred <- predict(xgb_model, dtest)
xgb_pred_class <- ifelse(xgb_pred > 0.5, 1, 0)

end_time <- Sys.time()
cat("   Completed in", round(as.numeric(end_time - start_time), 2), "seconds\n\n")

# ========================================
# ORGANIZE MODELS FOR COMPARISON
# ========================================

# Create models list to store all trained models and predictions
models <- list(
  "Logistic" = list(pred = baseline_pred, pred_class = baseline_pred_class),
  "Random Forest" = list(pred = rf_pred, pred_class = rf_pred_class),
  "SVM" = list(pred = svm_pred_prob, pred_class = svm_pred_class),
  "LASSO" = list(pred = lasso_pred, pred_class = lasso_pred_class),
  "XGBoost" = list(pred = xgb_pred, pred_class = xgb_pred_class)
)

cat("Models organized for comparison\n")

# ========================================
# 6. PERFORMANCE COMPARISON - FIXED
# ========================================

cat("6️⃣ PERFORMANCE COMPARISON\n")
cat(paste(rep("=", 50), collapse = ""), "\n")

# Initialize results with proper structure
results_table <- data.frame(
  Model = character(length(models)),
  AUC = numeric(length(models)),
  Accuracy = numeric(length(models)),
  Sensitivity = numeric(length(models)),
  Specificity = numeric(length(models)),
  F1_Score = numeric(length(models)),
  stringsAsFactors = FALSE
)

# Calculate metrics for each model
for(i in 1:length(models)) {
  pred_prob <- models[[i]]$pred
  pred_class <- models[[i]]$pred_class
  actual <- test_data$private_insurance
  
  # Model name
  results_table$Model[i] <- names(models)[i]
  
  # AUC
  roc_obj <- roc(actual, pred_prob, quiet = TRUE)
  results_table$AUC[i] <- round(as.numeric(auc(roc_obj)), 3)
  
  # Confusion matrix
  cm <- table(Predicted = pred_class, Actual = actual)
  results_table$Accuracy[i] <- round(sum(diag(cm)) / sum(cm), 3)
  
  # Sensitivity and Specificity
  if(nrow(cm) == 2 && ncol(cm) == 2) {
    results_table$Sensitivity[i] <- round(cm[2,2] / sum(cm[,2]), 3)
    results_table$Specificity[i] <- round(cm[1,1] / sum(cm[,1]), 3)
  } else {
    results_table$Sensitivity[i] <- 0
    results_table$Specificity[i] <- 0
  }
  
  # F1 Score
  tp <- sum(pred_class == 1 & actual == 1)
  fp <- sum(pred_class == 1 & actual == 0)
  fn <- sum(pred_class == 0 & actual == 1)
  
  precision <- ifelse(tp + fp > 0, tp / (tp + fp), 0)
  recall <- ifelse(tp + fn > 0, tp / (tp + fn), 0)
  f1 <- ifelse(precision + recall > 0, 2 * (precision * recall) / (precision + recall), 0)
  results_table$F1_Score[i] <- round(f1, 3)
}

cat("PERFORMANCE RESULTS:\n")
print(results_table)

# Best model
best_model_idx <- which.max(results_table$AUC)
cat("\nBest model:", results_table$Model[best_model_idx], 
    "with AUC =", results_table$AUC[best_model_idx], "\n")

# ========================================
# VARIABLE IMPORTANCE
# ========================================

cat("\n7️⃣ VARIABLE IMPORTANCE\n")
cat(paste(rep("=", 40), collapse = ""), "\n")

# Random Forest importance
rf_imp <- importance(rf_model)[,4]
rf_imp_scaled <- round(100 * rf_imp / max(rf_imp), 1)

cat("Random Forest Variable Importance:\n")
rf_sorted <- sort(rf_imp_scaled, decreasing = TRUE)
for(i in 1:length(rf_sorted)) {
  cat("  ", sprintf("%-20s", names(rf_sorted)[i]), ":", rf_sorted[i], "%\n")
}

# XGBoost importance
xgb_imp <- xgb.importance(feature_names = colnames(x_train), model = xgb_model)
cat("\nXGBoost Top Variables:\n")
for(i in 1:min(5, nrow(xgb_imp))) {
  cat("  ", sprintf("%-20s", xgb_imp$Feature[i]), ":", round(xgb_imp$Gain[i], 3), "\n")
}

# Organize results for visualization dashboard
ml_results <- list()
ml_results$models <- models
ml_results$test_labels <- test_data$private_insurance
ml_results$performance <- results_table

# Variable importance (organize from different models)
ml_results$importance <- data.frame(
  Variable = names(rf_imp_scaled),
  RandomForest = as.numeric(rf_imp_scaled),
  stringsAsFactors = FALSE
)

# Add LASSO importance (coefficients)
lasso_coefs <- as.matrix(coef(lasso_model))
lasso_importance <- abs(lasso_coefs[-1, 1])  # Remove intercept
lasso_scaled <- round(100 * lasso_importance / max(lasso_importance), 1)
ml_results$importance$LASSO <- lasso_scaled[match(ml_results$importance$Variable, names(lasso_scaled))]

# Add XGBoost importance
xgb_importance_scaled <- numeric(nrow(ml_results$importance))
for(i in 1:nrow(ml_results$importance)) {
  var_name <- ml_results$importance$Variable[i]
  xgb_match <- which(xgb_imp$Feature == var_name)
  if(length(xgb_match) > 0) {
    xgb_importance_scaled[i] <- round(100 * xgb_imp$Gain[xgb_match[1]] / max(xgb_imp$Gain), 1)
  }
}
ml_results$importance$XGBoost <- xgb_importance_scaled

# Fill any missing values with 0
ml_results$importance[is.na(ml_results$importance)] <- 0

cat("ml_results structure created successfully\n")

cat("\n✅ MACHINE LEARNING ANALYSIS COMPLETE\n")
cat("All models successfully trained and evaluated!\n")

ML Visualizations

In [ ]:
%%R

# ========================================
# COMBINED ML VISUALIZATION - VARIABLE IMPORTANCE WITH ROC OVERLAY - UNIFIED THEME
# ========================================

cat("=== CREATING COMBINED ML VISUALIZATION WITH UNIFIED STYLING ===\n")

# Define consistent color palette
model_colors <- RColorBrewer::brewer.pal(5, "Set1")
names(model_colors) <- c("Logistic", "Random Forest", "SVM", "LASSO", "XGBoost")

# ========================================
# PREPARE DATA FOR BOTH PLOTS
# ========================================

# Variable importance data preparation
clean_var_names <- data.frame(
  original = c("white_race", "high_income", "age_centered", "teaching_hospital", "urban_hospital", "large_hospital"),
  clean = c("Race\n(White vs\nNon-white)", "Income\n(High vs\nLow)", "Age\n(Centered)", 
           "Teaching\nHospital", "Urban\nLocation", "Large\nHospital"),
  stringsAsFactors = FALSE
)

imp_data <- ml_results$importance
imp_data$Variable_Clean <- clean_var_names$clean[match(imp_data$Variable, clean_var_names$original)]

imp_long <- melt(imp_data[, c("Variable_Clean", "RandomForest", "LASSO", "XGBoost")], 
                id.vars = "Variable_Clean")
colnames(imp_long) <- c("Variable", "Model", "Importance")

# UPDATED: Use unified color scheme for importance
importance_colors <- c(
  "RandomForest" = primary_colors[1],  # Professional blue
  "LASSO" = primary_colors[4],         # Red
  "XGBoost" = primary_colors[3]        # Green
)

# ROC data preparation
roc_data <- data.frame()

for(i in 1:length(ml_results$models)) {
  model_name <- names(ml_results$models)[i]
  pred_prob <- ml_results$models[[i]]$pred
  
  roc_obj <- roc(ml_results$test_labels, pred_prob, quiet = TRUE)
  
  roc_df <- data.frame(
    Model = model_name,
    Specificity = roc_obj$specificities,
    Sensitivity = roc_obj$sensitivities,
    AUC = round(auc(roc_obj), 3)
  )
  
  roc_data <- rbind(roc_data, roc_df)
}

unique_models <- unique(roc_data[, c("Model", "AUC")])
legend_labels <- paste0(unique_models$Model, " (AUC: ", unique_models$AUC, ")")

# ========================================
# CREATE MAIN VARIABLE IMPORTANCE PLOT - UNIFIED THEME
# ========================================

main_plot <- ggplot(imp_long, aes(x = reorder(Variable, Importance), y = Importance, fill = Model)) +
  geom_col(position = position_dodge(width = 0.8), alpha = 0.85, width = 0.75) +
  scale_fill_manual(
    values = importance_colors,
    name = "ML Algorithm",
    labels = c("Random Forest", "LASSO Regression", "XGBoost")
  ) +
  labs(
    title = "Machine Learning Variable Importance",
    subtitle = "for Predicting Private Insurance Access; ROC Overlay",
    x = "Predictor Variables",
    y = "Relative Importance (%)"
  ) +
  
  # UPDATED: Apply unified poster theme as base
  theme_poster +
  theme(
    # Override specific elements for this plot type while maintaining poster style
    axis.text.y = element_text(size = 10, lineheight = 0.8),
    legend.position = "bottom",
    legend.margin = ggplot2::margin(t = 10),
    
    # Grid lines
    panel.grid.minor = element_blank(),
    panel.grid.major.x = element_blank(),
    panel.grid.major.y = element_line(color = "gray90", linewidth = 0.5),
    
    # Margins
    plot.margin = ggplot2::margin(t = 25, r = 15, b = 15, l = 20)
  ) +
  
  coord_flip() +
  scale_y_continuous(
    expand = c(0, 0), 
    limits = c(0, max(imp_long$Importance) * 1.08),
    breaks = seq(0, 100, 20)
  ) +
  guides(fill = guide_legend(nrow = 1, override.aes = list(alpha = 0.8)))

# ========================================
# CREATE SMALL ROC OVERLAY PLOT - UNIFIED THEME
# ========================================

roc_overlay <- ggplot(roc_data, aes(x = 1 - Specificity, y = Sensitivity, color = Model)) +
  geom_line(linewidth = 1.5, alpha = 0.8) +
  geom_abline(intercept = 0, slope = 1, linetype = "dashed", color = "gray50", linewidth = 0.8) +
  
  # UPDATED: Use unified model colors from our color system
  scale_color_manual(values = model_colors, labels = legend_labels, name = "Model (AUC)") +
  
  labs(
    title = "ROC Curves",
    x = "1 - Specificity",
    y = "Sensitivity"
  ) +
  
  # UPDATED: Apply poster theme as base with overlay-specific overrides
  theme_poster +
  theme(
    # Small overlay styling
    plot.title = element_text(size = 9, face = "bold", hjust = 0.5, 
                             margin = ggplot2::margin(b = 3)),
    axis.title = element_text(size = 7, face = "bold"),
    axis.text = element_text(size = 6),
    legend.title = element_text(size = 7, face = "bold"),
    legend.text = element_text(size = 6),
    legend.position = "none",  # Remove legend to save space
    
    # Grid and background
    panel.grid.minor = element_blank(),
    panel.grid.major = element_line(color = "gray90", linewidth = 0.3),
    plot.background = element_rect(fill = "white", color = "black", linewidth = 0.5),
    panel.background = element_rect(fill = "white"),
    plot.margin = ggplot2::margin(5, 5, 5, 5)
  ) +
  
  coord_equal() +
  scale_x_continuous(limits = c(0, 1), expand = c(0.02, 0.02), 
                     breaks = c(0, 0.5, 1), labels = c("0", "0.5", "1")) +
  scale_y_continuous(limits = c(0, 1), expand = c(0.02, 0.02),
                     breaks = c(0, 0.5, 1), labels = c("0", "0.5", "1"))

# ========================================
# COMBINE PLOTS USING ANNOTATION_CUSTOM
# ========================================

# Convert the ROC plot to a grob for overlay
if(require("gridExtra", quietly = TRUE)) {
  
  # Create the combined plot
  combined_plot <- main_plot + 
    annotation_custom(
      grob = ggplotGrob(roc_overlay),
      xmin = 1.0, xmax = 4.0,    # Positioned in upper right area
      ymin = 60, ymax = 110      # Adjusted positioning
    )
  
} else {
  # Fallback: install gridExtra and try again
  install.packages("gridExtra", quiet = TRUE)
  library(gridExtra, quietly = TRUE)
  
  combined_plot <- main_plot + 
    annotation_custom(
      grob = ggplotGrob(roc_overlay),
      xmin = 1.0, xmax = 4.0,
      ymin = 60, ymax = 110
    )
}

print(combined_plot)

save_plot_for_poster(combined_plot, 
                    name = "ml_combined_importance_roc", 
                    description = "Combined ML Variable Importance with ROC Curves Overlay",
                    category = "results")

cat("*** Combined ML plot with unified styling created\n\n")

# ========================================
# SUMMARY INSIGHTS
# ========================================

cat("ML INSIGHTS SUMMARY\n")
cat(paste(rep("=", 50), collapse = ""), "\n")

# Find best performing model
best_auc_idx <- which.max(unique_models$AUC)
best_model_name <- unique_models$Model[best_auc_idx]
best_auc_score <- unique_models$AUC[best_auc_idx]

cat("BEST PERFORMING MODEL:", best_model_name, "\n")
cat("   AUC Score:", best_auc_score, "\n\n")

cat("KEY VARIABLE IMPORTANCE FINDINGS:\n")
# Get top variables from Random Forest
rf_importance <- imp_data[order(-imp_data$RandomForest), ]
cat("   Most important:", gsub("\n", " ", rf_importance$Variable_Clean[1]), "\n")
cat("   Second most important:", gsub("\n", " ", rf_importance$Variable_Clean[2]), "\n")
cat("   Third most important:", gsub("\n", " ", rf_importance$Variable_Clean[3]), "\n\n")

cat("KEY INSIGHTS:\n")
cat("   All models achieve good discrimination (AUC > 0.78)\n")
cat("   Race and income are consistently top predictors\n")
cat("   Hospital characteristics add moderate predictive value\n")
cat("   Complex ML models marginally outperform logistic regression\n")
cat("   Results validate statistical findings from multi-level models\n")

cat("\nCOMBINED ML VISUALIZATION COMPLETE WITH UNIFIED STYLING:\n")
cat("   - Applied theme_poster base theme with custom overrides\n")
cat("   - Used primary_colors for variable importance (blue, red, green)\n")
cat("   - Maintained Set1 palette for ROC curves (consistent with other ML plots)\n")
cat("   - Applied left-aligned titles (poster style)\n")
cat("   - Consistent typography and spacing with other visualizations\n")
cat("   - Professional appearance matching overall analysis style\n")

# Poster and Index Generation

Index

In [ ]:
%%R

# Generate manifest of all saved outputs
generate_output_manifest <- function() {
  
  cat("=== GENERATING OUTPUT MANIFEST ===\n")
  
  # Get saved plots
  if(exists("plot_metadata") && length(plot_metadata) > 0) {
    cat("SAVED PLOTS:\n")
    for(name in names(plot_metadata)) {
      meta <- plot_metadata[[name]]
      cat("  ", name, ":", meta$description, "\n")
      cat("    Files:", basename(meta$file_png), ",", basename(meta$file_pdf), "\n")
    }
  }
  
  # Get saved text outputs
  text_files <- list.files("saved_outputs", pattern = "\\.txt$", full.names = FALSE)
  if(length(text_files) > 0) {
    cat("\nSAVED TEXT OUTPUTS:\n")
    for(file in text_files) {
      cat("  ", file, "\n")
    }
  }
  
  cat("\n✅ All outputs saved and ready for poster/manuscript preparation\n")
}

generate_output_manifest()

Poster

In [ ]:
def create_research_poster():
    """Generate a clean research poster with improved title formatting and logo positioning"""
    
    # Import required libraries
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from matplotlib.patches import Rectangle
    import numpy as np
    from pathlib import Path
    import os
    import qrcode
    from PIL import Image
    
    print("🎨 GENERATING RESEARCH POSTER (Fixed Title + Logo)")
    print("=" * 40)
    
    # Poster dimensions (16:9 ratio - A0 landscape equivalent)
    poster_width = 32
    poster_height = 18
    
    # Create figure
    fig = plt.figure(figsize=(poster_width, poster_height))
    fig.patch.set_facecolor('white')
    
    # Define layout proportions - ENHANCED HEADER for bigger title
    header_height = 0.17   # Increased for even bigger title
    content_height = 0.83  # Decreased accordingly
    
    # ====================================
    # HEADER SECTION - FIXED TITLE FORMATTING
    # ====================================
    
    # Header background
    header_ax = fig.add_axes([0, 1-header_height, 1, header_height])
    header_ax.set_xlim(0, 1)
    header_ax.set_ylim(0, 1)
    header_ax.axis('off')
    
    # PROFESSIONAL: Dark blue/purple gradient background
    header_rect = Rectangle((0, 0), 1, 1, facecolor='#1e3a8a', edgecolor='none')  # Professional dark blue
    header_ax.add_patch(header_rect)
    
    # Add subtle gradient effect with overlay
    gradient_rect = Rectangle((0, 0), 1, 1, facecolor='#312e81', edgecolor='none', alpha=0.3)  # Dark purple overlay
    header_ax.add_patch(gradient_rect)
    
    # FIXED: Better formatted title section
    # Main title - split into two cleaner lines
    title_line1 = "Socioeconomic and Demographic Drivers of Ambulatory Surgery Usage"
    title_line2 = "HCUP National Ambulatory Surgery Sample - 2020"

    subtitle_line1 = "Scalable, Multi-Platform, Open-access, Reproducible Data Pipeline"
    subtitle_line2 = "7.8 million patients, 2899 hospitals, 35 states, 1 nation"  # SMALLER

    author_line1 = "Seena Khosravi, CA-2 | L. Soloniuk, MD; I. Pasca, MD; M. Douglas, MD; G. Stier, MD, MBA"
    author_line2 = "LLMs Utilized: Claude Sonnet 4  | Opus 4.1; ChatGPT 4o, o4, 5, 5-mini; Deepseek 3.1; Gemini 2.5 Pro; Grok 5"

    # IMPROVED title styling - INCREASED font size and ALL BOLD
    header_ax.text(0.02, 0.93, title_line1, ha='left', va='top', fontsize=36,  # Increased from 30
                fontweight='bold', color='white')
    
    header_ax.text(0.02, 0.74, title_line2, ha='left', va='top', fontsize=30,  # Increased from 30
                fontweight='bold', color='white')

    # Subtitle lines - ALL BOLD, adjusted spacing
    header_ax.text(0.02, 0.52, subtitle_line1, ha='left', va='center', fontsize=20,
                color='#e0e7ff', style='italic', fontweight='bold', linespacing=1.2)

    header_ax.text(0.02, 0.40, subtitle_line2, ha='left', va='center', fontsize=16,
                color='#c7d2fe', style='italic', fontweight='bold', linespacing=1.2)

    # Author lines - ALL BOLD, DECREASED space above (moved down)
    header_ax.text(0.02, 0.26, author_line1, ha='left', va='center', fontsize=18,  # Moved from 0.22 to 0.26
                color='#c7d2fe', fontweight='bold')  # Changed to bold

    header_ax.text(0.02, 0.08, author_line2, ha='left', va='bottom', fontsize=16,
                color='#a5b4fc', fontweight='bold', style='italic')  # Changed to bold
    
    colab_url = "https://colab.research.google.com/github/SeenaKhosravi/NASS/blob/main/Analysis_NASS.ipynb"
    
    # ====================================
    # IMPROVED RUHS LOGO POSITIONING - FARTHER RIGHT WITH EQUAL MARGINS
    # ====================================

    # Logo loading and placement with better positioning
    logo_path = Path("logos") / "ruhs_logo.png"
    logo_loaded = False

    if logo_path.exists():
        try:
            ruhs_logo = Image.open(logo_path)
            
            # Calculate width to maintain aspect ratio - INCREASED SIZE
            aspect_ratio = ruhs_logo.width / ruhs_logo.height
            logo_height = header_height * 0.60  # 65% of header height
            logo_width = logo_height * aspect_ratio
            
            # Calculate top margin for centering
            top_margin = (header_height - logo_height) / 2
            
            # Position with EQUAL top and right margins
            logo_x = 1.0 - logo_width + 0.0345  # Fixed 1% right margin
            logo_y = 1.0 - header_height + top_margin  # Keep vertical centering
            
            # Place logo
            logo_ax = fig.add_axes([logo_x, logo_y, logo_width, logo_height])
            logo_ax.imshow(ruhs_logo)
            logo_ax.axis('off')
            
            logo_loaded = True
            print(f"✅ RUHS logo loaded with equal top/right margins ({top_margin:.3f})")
            
        except Exception as e:
            print(f"❌ Failed to load logo: {e}")
    else:
        print(f"❌ Logo not found at: {logo_path}")
    
    # ====================================
    # CONTENT SECTION - SAME AS BEFORE
    # ====================================
    
    # Grid parameters for 4 columns x 3 rows (12 total blocks)
    grid_cols = 4
    grid_rows = 3
    
    # Optimized spacing for larger plots
    margin_x = 0.02
    margin_y = 0.02
    spacing_x = 0.015
    spacing_y = 0.02
    
    # Available space for plots
    available_width = 1 - 2 * margin_x - (grid_cols - 1) * spacing_x
    available_height = content_height - 2 * margin_y - (grid_rows - 1) * spacing_y
    
    # Plot dimensions
    plot_width = available_width / grid_cols
    plot_height = available_height / grid_rows
    
    # Define plot assignments for 3x4 grid (12 plots) - MOVED HEATMAP TO COLUMN 2 (BOTTOM ROW)
    plot_assignments = [
        # TOP ROW (row 0)
        {'name': 'top_procedures_income', 'row': 0, 'col': 0},
        {'name': 'procedures_by_race', 'row': 0, 'col': 1},
        {'name': 'payer_by_demographics', 'row': 0, 'col': 2},
        {'name': 'payer_by_income', 'row': 0, 'col': 3},
        
        # MIDDLE ROW (row 1)
        {'name': 'age_distribution_race', 'row': 1, 'col': 0},
        {'name': 'age_distribution_payer', 'row': 1, 'col': 1},
        {'name': 'census_comparison_lines', 'row': 1, 'col': 2},
        {'name': 'census_comparison_hourglass_enhanced', 'row': 1, 'col': 3},
        
        # BOTTOM ROW (row 2) - MOVED HEATMAP TO COLUMN 2 (next to QR code)
        {'name': 'odds_ratios_forest_plot', 'row': 2, 'col': 0},
        {'name': 'ml_combined_importance_roc', 'row': 2, 'col': 1},
        {'name': 'age_specific_procedure_patterns', 'row': 2, 'col': 2},  # MOVED HERE
        {'name': 'qr_code_block', 'row': 2, 'col': 3},
    ]
    
    # Load and place plots
    plots_dir = Path("saved_plots")
    plots_placed = 0
    
    for plot_info in plot_assignments:
        row = plot_info['row']
        col = plot_info['col']
        plot_name = plot_info['name']
        
        # Calculate position
        x = margin_x + col * (plot_width + spacing_x)
        y = content_height - margin_y - plot_height - row * (plot_height + spacing_y)
        
        # SPECIAL HANDLING FOR QR CODE BLOCK (3,4) - CLEAN AESTHETIC VERSION
        if plot_name == 'qr_code_block':
            ax = fig.add_axes([x, y, plot_width, plot_height])
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.axis('off')
            
            # Pure white background - clean and minimal
            ax.add_patch(Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='none'))
            
            # Generate QR code for this block
            try:
                qr_block = qrcode.QRCode(
                    version=1,
                    error_correction=qrcode.constants.ERROR_CORRECT_L,
                    box_size=8,  # Slightly larger for better readability
                    border=2,    # Clean border
                )
                qr_block.add_data(colab_url)
                qr_block.make(fit=True)
                
                qr_img_block = qr_block.make_image(fill_color="black", back_color="white")
                qr_path_block = "temp_qr_block.png"
                qr_img_block.save(qr_path_block)
                
                # Position QR code elegantly on the right with more space
                qr_block_ax = fig.add_axes([x + plot_width*0.58, y + plot_height*0.25, 
                                          plot_width*0.38, plot_height*0.5])
                qr_image_block = Image.open(qr_path_block)
                qr_block_ax.imshow(qr_image_block)
                qr_block_ax.axis('off')
                
                # Clean up temp file
                os.remove(qr_path_block)
                
            except Exception as e:
                print(f"QR block generation failed: {e}")
                # Elegant fallback
                qr_placeholder = Rectangle((0.58, 0.30), 0.38, 0.5, 
                                         facecolor='#f8f9fa', 
                                         edgecolor='#dee2e6', 
                                         linewidth=2)
                ax.add_patch(qr_placeholder)
                ax.text(0.77, 0.5, 'QR\nCODE', ha='center', va='center', 
                       fontsize=14, fontweight='bold', color='#6c757d')
            
            # BEAUTIFUL TYPOGRAPHY - LEFT SIDE
            # Main headline - match header font styling but slightly toned down
            ax.text(0.05, 0.82, 'Run this analysis', 
                   ha='left', va='center', 
                   fontsize=22,           # Slightly smaller than before
                   fontweight='600',      # Medium-bold instead of bold
                   color='#2c3e50',       # Keep the nice color
                   fontfamily='sans-serif')  # Match header font family
            
            ax.text(0.05, 0.70, 'on your phone!', 
                   ha='left', va='center', 
                   fontsize=22,           # Matching size
                   fontweight='600',      # Medium-bold
                   color='#2c3e50',
                   fontfamily='sans-serif')
            
            # Descriptive text - much bigger and readable, matching header font
            ax.text(0.05, 0.55, 'Generate this Poster', 
                   ha='left', va='center', 
                   fontsize=18,           # Much bigger for readability
                   fontweight='normal',   
                   color='#34495e',
                   fontfamily='sans-serif')
            
            ax.text(0.05, 0.45, 'Explore Statistics and Code', 
                   ha='left', va='center', 
                   fontsize=18,           # Much bigger
                   fontweight='normal', 
                   color='#34495e',
                   fontfamily='sans-serif')
            
            ax.text(0.05, 0.35, 'Deploy your own VM', 
                   ha='left', va='center', 
                   fontsize=18,           # Much bigger
                   fontweight='normal', 
                   color='#34495e',
                   fontfamily='sans-serif')
            
            # Call to action - bigger and stylish
            ax.text(0.05, 0.22, 'Scan here (Passcode: asa) →', 
                   ha='left', va='center', 
                   fontsize=18,           # Much bigger
                   fontweight='500',      
                   color='#e74c3c',       
                   style='italic',
                   fontfamily='sans-serif')
            
            # Link description - much bigger and clear
            ax.text(0.77, 0.22, 'Google Colab Link', 
                   ha='center', va='center', 
                   fontsize=16,           # Much bigger from 11
                   fontweight='bold', 
                   color='#7f8c8d',       
                   fontfamily='sans-serif',  # Match header font
                   bbox=dict(boxstyle="round,pad=0.4", 
                            facecolor='#ecf0f1',   
                            edgecolor='none',
                            alpha=0.8))
            
            plots_placed += 1
            print(f"  ✅ Created clean aesthetic QR code block at row {row+1}, col {col+1}")
            continue
        
        # Handle regular plots - REMOVED BORDERS COMPLETELY
        png_file = plots_dir / f"{plot_name}.png"
        
        if png_file.exists():
            try:
                # Load image
                img = Image.open(png_file)
                
                # Create subplot at calculated position
                ax = fig.add_axes([x, y, plot_width, plot_height])
                ax.imshow(img)
                ax.axis('off')
                
                # REMOVED: No borders around plots for clean look
                # border = Rectangle((0, 0), 1, 1, fill=False, edgecolor='#e2e8f0', 
                #                  linewidth=1.5, transform=ax.transAxes)
                # ax.add_patch(border)
                
                plots_placed += 1
                print(f"  ✅ Placed: {plot_name} at row {row+1}, col {col+1}")
                continue
                
            except Exception as e:
                print(f"  ❌ Failed to load {plot_name}: {e}")
        
        # Create placeholder for missing plots
        ax = fig.add_axes([x, y, plot_width, plot_height])
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        
        # Style for missing plots - updated colors
        ax.add_patch(Rectangle((0, 0), 1, 1, facecolor='#fef3c7', edgecolor='#f59e0b', linewidth=2))
        ax.text(0.5, 0.5, f'Plot Missing:\n{plot_name}', ha='center', va='center', 
               fontsize=10, color='#92400e', weight='bold')
        
        ax.axis('off')
        print(f"  ⚠️  Missing: {plot_name} at row {row+1}, col {col+1}")
    
    # ====================================
    # FOOTER SECTION - ENHANCED WITH PROFESSIONAL STYLING
    # ====================================
    
    # Footer text with enhanced styling
    footer_ax = fig.add_axes([0.02, 0.002, 0.96, 0.03])
    footer_text = "| ASA 2025 | Data: HCUP NASS 2020 | Software: Python + R in Jupyterlab | Hardware: Local/Colab/VM |"
    footer_ax.text(0.99, 0.5, footer_text, ha='right', va='center', fontsize=10,
                   color='#64748b', style='italic')
    footer_ax.axis('off')
    
    # ====================================
    # SAVE POSTER
    # ====================================
    
    # Save poster with high quality
    poster_file = "NASS_Research_Poster_Fixed.png"
    plt.savefig(poster_file, dpi=300, bbox_inches='tight', facecolor='white', 
                edgecolor='none', pad_inches=0.1)
    
    # Also save as PDF
    poster_pdf = "NASS_Research_Poster_Fixed.pdf"
    plt.savefig(poster_pdf, dpi=300, bbox_inches='tight', facecolor='white', 
                edgecolor='none', pad_inches=0.1)
    
    plt.show()
    
    print(f"\n🎉 POSTER GENERATION COMPLETE!")
    print(f"   📊 Plots placed: {plots_placed}/12 (including QR block)")
    print(f"   🏥 Logo: RUHS {'loaded with equal margins' if logo_loaded else 'placeholder'}")
    print(f"   🎨 Header: Professional dark blue/purple gradient")
    print(f"   ✨ Fixed: Logo positioning, all text bold, adjusted spacing")
    print(f"   📋 Grid layout: 4×3 = 12 positions")
    print(f"   📱 QR code block: Position 3,4 with professional styling")
    print(f"   🎨 Layout: Heatmap moved to column 3, no plot borders")
    print(f"   📄 Saved: {poster_file}")
    print(f"   📄 Saved: {poster_pdf}")
    print(f"   📐 Dimensions: {poster_width}\" × {poster_height}\" (16:9 ratio)")
    
    return poster_file

# Generate the fixed poster
poster_file = create_research_poster()

# Service/Diag.

##  Census API Key Encryption Tool

Run this code to encrypt your Census API key:

In [ ]:
# import json
# import base64
# import getpass
# from Crypto.Cipher import AES
# from Crypto.Hash import SHA256
# from Crypto.Random import get_random_bytes
# import os

# def encrypt_census_api_key():
#     """Encrypt Census API key for secure storage"""
    
#     print("🔐 CENSUS API KEY ENCRYPTION")
#     print("=" * 40)
    
#     # Securely get Census API key (no echo)
#     print("Enter your Census API key from: https://api.census.gov/data/key_signup.html")
#     api_key = getpass.getpass("Census API key (will not echo): ")
    
#     if not api_key or not api_key.strip():
#         print("❌ No API key provided")
#         return None, None
    
#     print(f"✅ Census API key entered ({len(api_key)} characters)")
    
#     # Get encryption password
#     password = getpass.getpass("Enter encryption password for Census key: ")
#     if not password:
#         print("❌ No password provided")
#         return None, None
    
#     print(f"✅ Password set ({len(password)} characters)")
    
#     # Create JSON structure (matching your GCS key format)
#     key_data = {
#         "api_key": api_key,
#         "source": "census_bureau",
#         "created": "2025-09-16",
#         "purpose": "NASS_research_analysis"
#     }
    
#     try:
#         print("\n🔒 Encrypting Census API key...")
        
#         # Convert to JSON string
#         key_json = json.dumps(key_data)
        
#         # Create encryption key from password
#         encryption_key = SHA256.new(password.encode()).digest()
        
#         # Pad data to AES block size (PKCS7 padding)
#         pad_len = 16 - (len(key_json) % 16)
#         padded_data = key_json + chr(pad_len) * pad_len
        
#         # Generate random IV
#         iv = get_random_bytes(16)
        
#         # Encrypt
#         cipher = AES.new(encryption_key, AES.MODE_CBC, iv)
#         encrypted_data = cipher.encrypt(padded_data.encode('utf-8'))
        
#         # Combine IV + encrypted data and base64 encode
#         final_encrypted = base64.b64encode(iv + encrypted_data)
        
#         print("✅ Encryption successful")
#         print(f"   Original size: {len(key_json)} bytes")
#         print(f"   Encrypted size: {len(final_encrypted)} bytes")
        
#         # Save encrypted file
#         encrypted_file = "census_key.enc"
#         with open(encrypted_file, 'wb') as f:
#             f.write(final_encrypted)
#         print(f"✅ Encrypted file saved: {encrypted_file}")
        
#         # Test decryption immediately
#         print("\n🔓 Testing decryption...")
        
#         # Read and decrypt
#         with open(encrypted_file, 'rb') as f:
#             encrypted_content = f.read()
        
#         encrypted_data_with_iv = base64.b64decode(encrypted_content)
#         iv = encrypted_data_with_iv[:16]
#         encrypted_data = encrypted_data_with_iv[16:]
        
#         cipher = AES.new(encryption_key, AES.MODE_CBC, iv)
#         decrypted_padded = cipher.decrypt(encrypted_data)
        
#         pad_len = decrypted_padded[-1]
#         decrypted_data = decrypted_padded[:-pad_len].decode('utf-8')
#         decrypted_key = json.loads(decrypted_data)
        
#         if decrypted_key['api_key'] == api_key:
#             print("✅ Decryption test successful!")
#             print(f"   Recovered API key: {decrypted_key['api_key'][:8]}...{decrypted_key['api_key'][-8:]}")
#         else:
#             print("❌ Decryption test failed!")
#             return None, None
        
#         return encrypted_file, password
        
#     except Exception as e:
#         print(f"❌ Encryption failed: {e}")
#         return None, None
    
#     finally:
#         # Security cleanup - overwrite sensitive variables
#         try:
#             api_key = 'X' * len(api_key) if api_key else ''
#             password = 'X' * len(password) if password else ''
#             if 'key_data' in locals():
#                 key_data['api_key'] = 'X' * len(key_data['api_key'])
#             import gc
#             gc.collect()
#         except:
#             pass

# # Run the encryption
# encrypted_file, password = encrypt_census_api_key()

# if encrypted_file and password:
#     print(f"\n🎉 SUCCESS!")
#     print(f"   ✅ Encrypted file: {encrypted_file}")
#     print(f"   🔒 Password: {password}")
#     print(f"\n📋 NEXT STEPS:")
#     print(f"   1. Upload '{encrypted_file}' to your GitHub release:")
#     print(f"      https://github.com/SeenaKhosravi/NASS/releases/tag/v1.0.0.0")
#     print(f"   2. Use this URL in your notebook:")
#     print(f"      https://github.com/SeenaKhosravi/NASS/releases/download/v1.0.0.0/census_key.enc")
#     print(f"   3. Use password: {password}")
#     print(f"\n💡 Your notebook is already configured to use this!")
#     print(f"\n🔒 Security note: API key never stored in code or visible on screen")
# else:
#     print("❌ Encryption failed")

## GCS Service Agent Key Encryption

Requires raw key json file downloaded from Service Agent settings in Google console. 

In [ ]:
# import json
# import base64
# import getpass
# from Crypto.Cipher import AES
# from Crypto.Hash import SHA256
# from Crypto.Random import get_random_bytes
# import tempfile
# import os

# def encrypt_and_test_key():
#     """Encrypt a service account key and immediately test decryption"""
    
#     print("🔐 SERVICE ACCOUNT KEY ENCRYPTION & TEST")
#     print("=" * 50)
    
#     # Step 1: Load your key file
#     key_file_path = input("Enter path to your service account JSON file: ").strip()
    
#     if not os.path.exists(key_file_path):
#         print(f"❌ File not found: {key_file_path}")
#         return None, None  # Return tuple instead of just None
    
#     try:
#         with open(key_file_path, 'r') as f:
#             key_data = json.load(f)
#         print("✅ Key file loaded successfully")
#         print(f"   Project: {key_data.get('project_id')}")
#         print(f"   Email: {key_data.get('client_email')}")
#     except Exception as e:
#         print(f"❌ Failed to load key file: {e}")
#         return None, None  # Return tuple instead of just None
    
#     # Step 2: Get encryption password
#     password = getpass.getpass("Enter encryption password: ")
#     if not password:
#         print("❌ No password provided")
#         return None, None  # Return tuple instead of just None
    
#     print(f"✅ Password set ({len(password)} characters)")
    
#     # Step 3: Encrypt the key
#     print("\n🔒 ENCRYPTING KEY...")
    
#     try:
#         # Convert to JSON string
#         key_json = json.dumps(key_data)
        
#         # Create encryption key from password
#         encryption_key = SHA256.new(password.encode()).digest()
        
#         # Pad data to AES block size (PKCS7 padding)
#         pad_len = 16 - (len(key_json) % 16)
#         padded_data = key_json + chr(pad_len) * pad_len
        
#         # Generate random IV
#         iv = get_random_bytes(16)
        
#         # Encrypt
#         cipher = AES.new(encryption_key, AES.MODE_CBC, iv)
#         encrypted_data = cipher.encrypt(padded_data.encode('utf-8'))
        
#         # Combine IV + encrypted data and base64 encode
#         final_encrypted = base64.b64encode(iv + encrypted_data)
        
#         print("✅ Encryption successful")
#         print(f"   Original size: {len(key_json)} bytes")
#         print(f"   Encrypted size: {len(final_encrypted)} bytes")
        
#         # Save encrypted file
#         encrypted_file = key_file_path.replace('.json', '.enc')
#         with open(encrypted_file, 'wb') as f:
#             f.write(final_encrypted)
#         print(f"✅ Encrypted file saved: {encrypted_file}")
        
#     except Exception as e:
#         print(f"❌ Encryption failed: {e}")
#         return None, None
    
#     # Step 4: Immediately test decryption
#     print("\n🔓 TESTING DECRYPTION...")
    
#     try:
#         # Read the encrypted file we just created
#         with open(encrypted_file, 'rb') as f:
#             encrypted_content = f.read()
        
#         # Decode base64
#         encrypted_data_with_iv = base64.b64decode(encrypted_content)
        
#         # Extract IV and encrypted data
#         iv = encrypted_data_with_iv[:16]
#         encrypted_data = encrypted_data_with_iv[16:]
        
#         # Decrypt
#         cipher = AES.new(encryption_key, AES.MODE_CBC, iv)
#         decrypted_padded = cipher.decrypt(encrypted_data)
        
#         # Remove PKCS7 padding
#         pad_len = decrypted_padded[-1]
#         decrypted_data = decrypted_padded[:-pad_len].decode('utf-8')
        
#         # Parse as JSON
#         decrypted_key = json.loads(decrypted_data)
        
#         print("✅ Decryption successful!")
#         print(f"   Project: {decrypted_key.get('project_id')}")
#         print(f"   Email: {decrypted_key.get('client_email')}")
        
#         # Verify it matches original
#         if decrypted_key == key_data:
#             print("✅ Decrypted data matches original perfectly!")
#         else:
#             print("⚠️  Decrypted data differs from original")
        
#         return encrypted_file, password
        
#     except Exception as e:
#         print(f"❌ Decryption test failed: {e}")
#         return None, None

# def test_gcs_access(encrypted_file, password):
#     """Test GCS access using the encrypted key"""
    
#     print("\n☁️  TESTING GCS ACCESS...")
    
#     try:
#         from google.cloud import storage
        
#         # Decrypt the key for GCS access
#         with open(encrypted_file, 'rb') as f:
#             encrypted_content = f.read()
        
#         # Decode and decrypt
#         encrypted_data_with_iv = base64.b64decode(encrypted_content)
#         iv = encrypted_data_with_iv[:16]
#         encrypted_data = encrypted_data_with_iv[16:]
        
#         encryption_key = SHA256.new(password.encode()).digest()
#         cipher = AES.new(encryption_key, AES.MODE_CBC, iv)
#         decrypted_padded = cipher.decrypt(encrypted_data)
        
#         pad_len = decrypted_padded[-1]
#         decrypted_data = decrypted_padded[:-pad_len].decode('utf-8')
#         decrypted_key = json.loads(decrypted_data)
        
#         # Create temporary file for GCS client
#         with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as temp_file:
#             json.dump(decrypted_key, temp_file)
#             temp_key_path = temp_file.name
        
#         try:
#             # Test GCS access
#             client = storage.Client.from_service_account_json(temp_key_path)
            
#             # List buckets
#             buckets = list(client.list_buckets())
#             print(f"✅ GCS authentication successful!")
#             print(f"   Found {len(buckets)} accessible buckets")
            
#             # Show bucket names
#             for bucket in buckets[:5]:  # Show first 5
#                 print(f"   - {bucket.name}")
            
#             # Test specific bucket access if you have one
#             bucket_name = input("\nEnter bucket name to test (or press Enter to skip): ").strip()
#             if bucket_name:
#                 try:
#                     bucket = client.bucket(bucket_name)
#                     if bucket.exists():
#                         print(f"✅ Bucket '{bucket_name}' exists and is accessible")
                        
#                         # List some files
#                         blobs = list(bucket.list_blobs(max_results=5))
#                         if blobs:
#                             print(f"   Found {len(blobs)} files (showing first 5):")
#                             for blob in blobs:
#                                 print(f"   - {blob.name} ({blob.size / (1024*1024):.1f} MB)")
#                         else:
#                             print("   Bucket is empty")
#                     else:
#                         print(f"❌ Bucket '{bucket_name}' not found or not accessible")
#                 except Exception as bucket_error:
#                     print(f"❌ Bucket access failed: {bucket_error}")
            
#         finally:
#             # Clean up temp file
#             os.unlink(temp_key_path)
            
#     except ImportError:
#         print("❌ google-cloud-storage not installed")
#         print("   Install with: pip install google-cloud-storage")
#     except Exception as e:
#         print(f"❌ GCS test failed: {e}")

# def create_upload_script(encrypted_file, password):
#     """Create a script to upload the encrypted key to GitHub releases"""
    
#     script_content = f'''
# # Upload encrypted key to GitHub releases
# # 1. Create a new release on GitHub
# # 2. Upload the encrypted file: {encrypted_file}
# # 3. Use this URL pattern in your notebook:
# #    https://github.com/YOUR_USERNAME/YOUR_REPO/releases/download/TAG_NAME/{os.path.basename(encrypted_file)}

# # Test URL (replace with your actual details):
# # https://github.com/SeenaKhosravi/NASS/releases/download/v1.0.0.0/{os.path.basename(encrypted_file)}

# # Password for decryption: {password}
# # (Keep this secure - don't commit to git!)
# '''
    
#     script_file = "upload_instructions.txt"
#     with open(script_file, 'w') as f:
#         f.write(script_content)
    
#     print(f"\n📝 Upload instructions saved to: {script_file}")

# # Run the complete test with better error handling
# print("🔐 Starting GCS service account encryption test...")

# # Try the encryption and test
# encrypted_file, password = encrypt_and_test_key()

# if encrypted_file and password:
#     print(f"\n🎉 SUCCESS! Your key is encrypted and tested.")
#     print(f"   Encrypted file: {encrypted_file}")
#     print(f"   Password: {password}")
    
#     # Test GCS access
#     test_choice = input("\nTest GCS access with encrypted key? (y/n): ").lower()
#     if test_choice == 'y':
#         test_gcs_access(encrypted_file, password)
    
#     # Create upload instructions
#     create_upload_script(encrypted_file, password)
    
#     print(f"\n📋 SUMMARY:")
#     print(f"   ✅ Original key: Loaded and validated")
#     print(f"   ✅ Encryption: Working with AES-256-CBC")
#     print(f"   ✅ Decryption: Verified identical to original")
#     print(f"   ✅ File saved: {encrypted_file}")
#     print(f"\n🔒 Keep your password secure: {password}")
#     print(f"\n💡 Next steps:")
#     print(f"   1. Upload {encrypted_file} to GitHub releases")
#     print(f"   2. Use the password in your notebook")
#     print(f"   3. Delete the original .json file for security")
# else:
#     print("\n❌ Encryption/testing failed")
#     print("💡 Check the file path and try again")
#     print("   Expected format: C:\\path\\to\\your\\file.json")

## Google Cloud Storage Connection Test

Requires public url encrypted key last cell. 

In [ ]:
# # Standalone GCS Authentication Test via GitHub Encrypted Key
# # This cell is completely self-contained and works with a clean kernel

# print("🔑 STANDALONE GCS AUTHENTICATION TEST")
# print("=" * 50)

# # Step 1: Get user inputs
# import getpass
# import os

# print("Please provide the following information:")
# gcs_encrypted_url = input("Enter GCS encrypted key URL: ").strip()
# if not gcs_encrypted_url:
#     gcs_encrypted_url = "https://github.com/SeenaKhosravi/NASS/releases/download/v1.0.0.0/gcs_key.enc"
#     print(f"Using default URL: {gcs_encrypted_url}")

# gcs_password = getpass.getpass("Enter GCS decryption password: ")
# if not gcs_password:
#     print("❌ No password provided - cannot proceed")
#     exit()

# gcs_bucket = input("Enter target bucket name (default: nass_2020): ").strip()
# if not gcs_bucket:
#     gcs_bucket = "nass_2020"

# gcs_blob = input("Enter target file name (default: nass_2020_all.csv): ").strip()
# if not gcs_blob:
#     gcs_blob = "nass_2020_all.csv"

# print(f"\n📋 Configuration:")
# print(f"   URL: {gcs_encrypted_url}")
# print(f"   Password: {'*' * len(gcs_password)}")
# print(f"   Bucket: {gcs_bucket}")
# print(f"   File: {gcs_blob}")

# # Step 2: Install required packages if needed
# import subprocess
# import sys

# def install_if_missing(package, pip_name=None):
#     """Install package if not available"""
#     if pip_name is None:
#         pip_name = package
    
#     try:
#         __import__(package)
#         return True
#     except ImportError:
#         print(f"Installing {pip_name}...")
#         try:
#             subprocess.check_call([sys.executable, '-m', 'pip', 'install', pip_name])
#             return True
#         except subprocess.CalledProcessError:
#             print(f"❌ Failed to install {pip_name}")
#             return False

# # Install required packages
# required_packages = [
#     ('requests', 'requests'),
#     ('Crypto', 'pycryptodome'),
#     ('google.cloud.storage', 'google-cloud-storage')
# ]

# print(f"\n📦 Checking required packages...")
# all_packages_available = True
# for package, pip_name in required_packages:
#     if not install_if_missing(package, pip_name):
#         all_packages_available = False

# if not all_packages_available:
#     print("❌ Some packages failed to install - cannot proceed")
#     exit()

# print("✅ All required packages available")

# # Step 3: Define decryption function
# def decrypt_key(encrypted_url, password):
#     """Decrypt key using pycryptodome"""
#     try:
#         import requests
#         from Crypto.Cipher import AES
#         from Crypto.Hash import SHA256
#         import base64
#         import json
        
#         print("🔓 Downloading and decrypting access key...")
        
#         # Download encrypted data
#         response = requests.get(encrypted_url, timeout=30)
#         response.raise_for_status()
#         encrypted_data = base64.b64decode(response.content)
        
#         # Derive key from password
#         key = SHA256.new(password.encode()).digest()
        
#         # Decrypt (assuming AES-CBC with IV)
#         cipher = AES.new(key, AES.MODE_CBC, encrypted_data[:16])
#         decrypted = cipher.decrypt(encrypted_data[16:])
        
#         # Remove PKCS7 padding
#         pad_len = decrypted[-1]
#         decrypted_key = decrypted[:-pad_len].decode()
        
#         # Validate it's valid JSON
#         json.loads(decrypted_key)
        
#         print("✅ Successfully decrypted access key")
#         return decrypted_key
        
#     except Exception as e:
#         print(f"❌ Decryption failed: {e}")
#         return None

# # Step 4: Test GCS authentication
# try:
#     from google.cloud import storage
#     import json
#     import tempfile
    
#     print(f"\n🔓 Testing GCS authentication...")
    
#     # Decrypt the service account key
#     decrypted_key = decrypt_key(gcs_encrypted_url, gcs_password)
    
#     if decrypted_key:
#         print("✅ Successfully decrypted service account key")
        
#         # Parse as JSON and create temporary file
#         key_data = json.loads(decrypted_key)
#         print(f"   Project ID: {key_data.get('project_id')}")
#         print(f"   Service Account: {key_data.get('client_email')}")
        
#         with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
#             json.dump(key_data, f)
#             temp_key_file = f.name
        
#         try:
#             # Test GCS client creation
#             print("\n☁️  Creating GCS client...")
#             client = storage.Client.from_service_account_json(temp_key_file)
            
#             # List accessible buckets
#             print("📊 Listing accessible buckets...")
#             buckets = list(client.list_buckets())
#             print(f"✅ Found {len(buckets)} accessible buckets:")
            
#             for bucket in buckets:
#                 print(f"   • {bucket.name}")
            
#             # Test specific bucket access
#             print(f"\n🎯 Testing access to target bucket: {gcs_bucket}")
#             target_bucket = client.bucket(gcs_bucket)
            
#             if target_bucket.exists():
#                 print(f"✅ Bucket '{gcs_bucket}' exists and is accessible")
                
#                 # Check for the target file
#                 target_blob = target_bucket.blob(gcs_blob)
#                 if target_blob.exists():
#                     target_blob.reload()
#                     size_gb = target_blob.size / (1024**3)
#                     print(f"✅ Target file '{gcs_blob}' found")
#                     print(f"   Size: {size_gb:.2f} GB")
#                     print(f"   Last modified: {target_blob.updated}")
#                     print(f"   Content type: {target_blob.content_type}")
#                     print(f"   MD5 hash: {target_blob.md5_hash}")
                    
#                     # Test download capability
#                     print(f"\n📥 Testing download capability...")
#                     try:
#                         # Download first 1KB to test
#                         sample_data = target_blob.download_as_bytes(start=0, end=1023)
#                         print(f"✅ Download test successful ({len(sample_data)} bytes)")
                        
#                         # Show first few characters if it's text
#                         try:
#                             preview = sample_data.decode('utf-8')[:100]
#                             print(f"   Preview: {preview}...")
#                         except:
#                             print(f"   Binary file detected")
                            
#                     except Exception as download_error:
#                         print(f"❌ Download test failed: {download_error}")
                    
#                 else:
#                     print(f"❌ Target file '{gcs_blob}' not found in bucket")
                    
#                     # List some files to see what's available
#                     print("📁 Available files in bucket (first 10):")
#                     blobs = list(target_bucket.list_blobs(max_results=10))
#                     if blobs:
#                         for blob in blobs:
#                             print(f"   • {blob.name} ({blob.size / (1024*1024):.1f} MB)")
#                     else:
#                         print("   (bucket is empty)")
#             else:
#                 print(f"❌ Bucket '{gcs_bucket}' not found or not accessible")
            
#             print(f"\n🎉 GCS AUTHENTICATION TEST COMPLETE!")
#             print(f"   • Package installation: ✅ Success")
#             print(f"   • Key decryption: ✅ Working")
#             print(f"   • GCS authentication: ✅ Working") 
#             print(f"   • Bucket access: ✅ Working")
#             print(f"   • File detection: ✅ Working")
#             print(f"   • Download capability: ✅ Tested")
#             print(f"\n🚀 Ready for full data loading!")
            
#         finally:
#             # Clean up temporary file
#             try:
#                 os.unlink(temp_key_file)
#                 print("🧹 Cleaned up temporary key file")
#             except:
#                 pass
#     else:
#         print("❌ Failed to decrypt service account key")
#         print("💡 Check your password and encrypted key URL")

# except ImportError as e:
#     print(f"❌ Missing required package after installation: {e}")
#     print("💡 Try manually installing: pip install google-cloud-storage pycryptodome")

# except Exception as e:
#     print(f"❌ Authentication test failed: {e}")
#     print("💡 Check your configuration and try again")

# finally:
#     # Security cleanup
#     try:
#         gcs_password = 'X' * len(gcs_password)
#         del gcs_password
#         import gc
#         gc.collect()
#         print("🔒 Security cleanup completed")
#     except:
#         pass

# print("\n" + "=" * 50)
# print("Test completed! This cell is fully self-contained.")